In [1]:
#pandas
import pandas as pd
#datefor
import datetime
import sys

In [2]:
#mongo
from pymongo import MongoClient
import pymongo

In [3]:
#Visual
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.patches import Wedge
import matplotlib
#from matplotlib.widgets import RadioButtons
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk
import tkinter as tk
from copy import deepcopy
import numpy as np #para manipular os vetores

from matplotlib.backend_bases import MouseEvent
from tkinter import ttk, Menu, Toplevel, LabelFrame, BooleanVar, Listbox, END, Scrollbar,Frame
import tkinter.font as tkFont

#from tkinter import * 
#from tkinter import Tk, Frame, Menu
from wordcloud import WordCloud, STOPWORDS 
from matplotlib import ft2font
from matplotlib.font_manager import ttfFontProperty
import matplotlib.font_manager as fm
#matplotlib.use('TkAgg')0,,,,,,,,,,,,,,,,,,,,,,,,,,,
#matplotlib.rcParams['font.sans-serif'] = ['Segoe UI Emoji','Simbola','Source Han Sans TW', 'SimHei','Arial', 'Bitstream Vera Sans', 'Lucida Grande', 'Verdana', 'Geneva', 'Lucid', 'Helvetica', 'Avant Garde', 'sans-serif']
import webbrowser

In [4]:
fpath = 'C:\\Users\\Joana\\TwitterColorEmoji-SVGinOT-Win-13.0.1\\TwitterColorEmoji-SVGinOT.ttf'

fprop = fm.FontProperties(fname=fpath)
font = ft2font.FT2Font(fpath)
fprop = fm.FontProperties(fname=fpath)

ttfFontProp = ttfFontProperty(font)

fontprop = fm.FontProperties(family='sans-serif',
                            fname=ttfFontProp.fname,
                            size=15,
                            stretch=ttfFontProp.stretch,
                            style=ttfFontProp.style,
                            variant=ttfFontProp.variant,
                            weight=ttfFontProp.weight)

In [5]:
class Conexao(object):
    _db = None
    _client = None
    _dt = None
    def __init__(self, MONGO_HOST):
        #print(MONGO_HOST)
        self._client = MongoClient(MONGO_HOST)
        #print(MONGO_HOST[20:])
        self._db = self._client[MONGO_HOST[20:]]        
        collectionsList = self._db.list_collection_names ()
        #print(collectionsList)
    def Find(self, collection, queryDic = None,queryFields = None):
        retorno = pd.DataFrame()
        try:
            if queryDic == None:
                retorno =  pd.DataFrame(list(self._db[collection].find()))
            elif queryFields == None:
                retorno =  pd.DataFrame(list(self._db[collection].find(queryDic)))
            else:
                retorno =  pd.DataFrame(list(self._db[collection].find(queryDic,queryFields)))
        except Exception as e:
            print(e)
            retorno = pd.DataFrame()
        return retorno
    def Aggregate(self, collection, queryPipe):
        retorno = pd.DataFrame
        try:
            #aggregate(pipe, allowDiskUse=True)
            #,allowDiskUse=True
            retorno = pd.DataFrame(list(self._db[collection].aggregate(queryPipe)))
        except Exception as e:
            print(e)
            retorno = pd.DataFrame()
        return retorno
    def AggregatePipe(self, collection, projectList = None, groupList = None,sortList = None,matchListDic = None):
        projectDic,groupDic,sortDic, matchDic = PipeBuild ( projectList, groupList, sortList, matchListDic)
#         print(projectDic)
#         print(groupDic)
#         print(sortDic)
#         print(matchDic)
        retorno = pd.DataFrame()
        
        try: 
            if projectDic != None and groupDic != None and sortDic!= None and matchDic!= None:
                retorno = pd.DataFrame(list(self._db[collection].aggregate([matchDic,projectDic,groupDic,sortDic])))
            elif projectDic != None and groupDic != None and sortDic!= None:
                retorno = pd.DataFrame(list(self._db[collection].aggregate([projectDic,groupDic,sortDic])))
            elif projectDic != None and groupDic != None:
                retorno = pd.DataFrame(list(self._db[collection].aggregate([projectDic,groupDic])))
            elif projectDic != None and matchDic!= None:
                retorno = pd.DataFrame(list(self._db[collection].aggregate([matchDic,projectDic])))
            elif projectDic != None:
                retorno = pd.DataFrame(list(self._db[collection].aggregate([projectDic])))            
        except Exception as e:
            print(e)
            retorno = pd.DataFrame()
        return retorno

In [6]:
def PipeBuild(projectList,groupList,sortList,matchListDic):
    
    if matchListDic != None:
        matchDic      = {}
        matchAuxDic   = {}        
        for k,v in matchListDic.items():
            if k == "video_commentID" :            
                #matchAuxDic["$match"] = {'comment_videoID' : v}
                matchAuxDic["comment_videoID"] = v
            elif k == "instagram_id":
                matchAuxDic["instagram_id"] = v
        matchDic["$match"] = matchAuxDic
    else: 
        matchDic = None
     
    if projectList != None:        
        projectDic    = {}
        projectAuxDic = {}
        for pl in projectList:        
            if pl == "dayOfMonthS":
                projectAuxDic["dayOfMonthS"]    = {"$toString" : {"$dayOfMonth" : "$created_at_db"}} 
            elif pl == "monthS":                   
                projectAuxDic["monthS"]         = {"$toString" : {"$month"      : "$created_at_db"}}
            elif pl == "monthI":                   
                projectAuxDic["monthI"]         = {"$toInt"    : {"$month"      : "$created_at_db"}}  
            elif pl == "yearS":    
                projectAuxDic["yearS"]          = {"$toString" : {"$year"       : "$created_at_db"}} 
            elif pl == "yearI":    
                projectAuxDic["yearI"]          = {"$toInt"    : {"$year"       : "$created_at_db"}} 
            elif pl == "hourS":    
                projectAuxDic["hourS"]          = {"$toString" : {"$hour"       : "$created_at_db"}}
            elif pl == "minuteS":  
                projectAuxDic["minuteS"]        = {"$toString" : {"$minute"     : "$created_at_db"}}
            elif pl == "comment_text":    
                projectAuxDic["comment_text"]   = {"$toString" : "$comment_text"}
                
            elif pl == "text":
                projectAuxDic["text"]           = "$text"
            elif pl == "tweet":
                projectAuxDic["text"]           = "$tweet" 
            elif pl == "anoID":
                projectAuxDic["_id"]            = {'$toInt': {'$year': '$created_at_db'}}
            elif pl == "mesID":
                projectAuxDic["_id"]            = {"$concat" : [{"$toString" :{'$year': '$created_at_db'}},
                                                                "-",                                                                      
                                                                {"$cond" :  {"if" : { "$lt" : [ {'$month': '$created_at_db'}, 10 ] }, "then": { "$concat" : ["0",{"$toString": {'$month': '$created_at_db'}}]},"else": { "$toString": {'$month': '$created_at_db'}}}}
                                                                ]}
            elif pl == "diaID":
                projectAuxDic["_id"]            = {"$concat" : [{"$toString" :{'$year': '$created_at_db'}},
                                                                "-",
                                                                {"$cond" :  {"if" : { "$lt" : [ {'$month': '$created_at_db'}, 10 ] }, "then": { "$concat" : ["0",{"$toString": {'$month': '$created_at_db'}}]},"else": { "$toString": {'$month': '$created_at_db'}}}},
                                                                "-",
                                                                {"$cond" :  {"if" : { "$lt" : [ {'$dayOfMonth': '$created_at_db'}, 10 ] }, "then": { "$concat" : ["0",{"$toString": {'$dayOfMonth': '$created_at_db'}}]},"else": { "$toString": {'$dayOfMonth': '$created_at_db'}}}}
                                                                 ]}
            elif pl == "hourID":
                projectAuxDic["_id"]            = {"$concat" : [{"$toString" :{'$year': '$created_at_db'}},
                                                                "-",
                                                                {"$cond" :  {"if" : { "$lt" : [ {'$month': '$created_at_db'}, 10 ] }, "then": { "$concat" : ["0",{"$toString": {'$month': '$created_at_db'}}]},"else": { "$toString": {'$month': '$created_at_db'}}}},
                                                                "-",
                                                                {"$cond" :  {"if" : { "$lt" : [ {'$dayOfMonth': '$created_at_db'}, 10 ] }, "then": { "$concat" : ["0",{"$toString": {'$dayOfMonth': '$created_at_db'}}]},"else": { "$toString": {'$dayOfMonth': '$created_at_db'}}}},
                                                                " ",
                                                                {"$cond" :  {"if" : { "$lt" : [ {'$hour': '$created_at_db'}, 10 ] }, "then": { "$concat" : ["0",{"$toString": {'$hour': '$created_at_db'}}]},"else": { "$toString": {'$hour': '$created_at_db'}}}}
                                                                ]}
            elif pl == "minID":
                projectAuxDic["_id"]            = {"$concat" : [{"$toString" :{'$year': '$created_at_db'}},
                                                                "-",
                                                                {"$cond" :  {"if" : { "$lt" : [ {'$month': '$created_at_db'}, 10 ] }, "then": { "$concat" : ["0",{"$toString": {'$month': '$created_at_db'}}]},"else": { "$toString": {'$month': '$created_at_db'}}}},
                                                                "-",
                                                                {"$cond" :  {"if" : { "$lt" : [ {'$dayOfMonth': '$created_at_db'}, 10 ] }, "then": { "$concat" : ["0",{"$toString": {'$dayOfMonth': '$created_at_db'}}]},"else": { "$toString": {'$dayOfMonth': '$created_at_db'}}}},
                                                                " ",
                                                                {"$cond" :  {"if" : { "$lt" : [ {'$hour': '$created_at_db'}, 10 ] }, "then": { "$concat" : ["0",{"$toString": {'$hour': '$created_at_db'}}]},"else": { "$toString": {'$hour': '$created_at_db'}}}},
                                                                ":",
                                                                {"$cond" :  {"if" : { "$lt" : [ {'$minute': '$created_at_db'}, 10 ] }, "then": { "$concat" : ["0",{"$toString": {'$minute': '$created_at_db'}}]},"else": { "$toString": {'$minute': '$created_at_db'}}}},
                                                                ":00",
                                                                 ]} 
            elif pl == "idtweet":                
                projectAuxDic["idtweet"]         = "$id"            
            elif pl == "retweets":                
                projectAuxDic["retweets"]        = "$retweets_count"
            elif pl == "likes":
                projectAuxDic["likes"]           = "$likes_count"
            elif pl == "replies":
                projectAuxDic["replies"]         = "$replies_count"
            elif pl == "twitter_video":
                projectAuxDic["twitter_video"]   = "$video"
            elif pl == "twitter_text":
                projectAuxDic["twitter_text"]    = "$tweet"
            elif pl == "twitter_created_at_db" :               
                projectAuxDic["twitter_created_at_db"] = "$created_at_db"
            elif pl == "video_viewCount" :
                projectAuxDic["video_viewCount"] = {"$toInt"    : "$video_viewCount"} 
            elif pl == "video_likeCount" :
                projectAuxDic["video_likeCount"] = {"$toInt"    : "$video_likeCount"} 
            elif pl == "video_dislikeCount" :
                projectAuxDic["video_dislikeCount"] = {"$toInt" : "$video_dislikeCount"} 
            elif pl == "video_commentCount" :
                projectAuxDic["video_commentCount"] = {"$toInt" : "$video_commentCount_total"}
            elif pl == "youtube_created_at_db" :               
                projectAuxDic["youtube_created_at_db"] = "$created_at_db"
            elif pl == "video_title" :
                projectAuxDic["text"] = "$video_title"
            elif pl == "video_id" :
                projectAuxDic["video_id"] = "$video_id" 
            elif pl == "video_definition" :
                projectAuxDic["video_definition"] = "$video_definition"
            elif pl == "video_caption" :
                projectAuxDic["video_caption"] = "$video_caption"
            elif pl == "video_duration" :
                projectAuxDic["video_duration"] = "$video_duration"
            elif pl == "overall" :
                projectAuxDic["overall"] = "$overall"
            elif pl == "comment_youtube_likes" :
                projectAuxDic["comment_likeCount"] = "$comment_likeCount"
            elif pl == "comment_instagram_likes" :                
                projectAuxDic["likes"] = "$likes"
            elif pl == "comment_videoID":
                projectAuxDic["comment_videoID"] = "$comment_videoID"
            elif pl == "instagram_id" :
                projectAuxDic["instagram_id"] = "$instagram_id"
            elif pl == "instagram_text" :
                projectAuxDic["text"] = "$text"
            elif pl == "instagram_commentCount" :
                projectAuxDic["instagram_comment"] = {"$toInt" : "$comment_count"}
            elif pl == "instagram_likeCount" :
                projectAuxDic["instagram_like"] = {"$toInt" : "$like_count"}
            elif pl == "instagram_created_at" :
                projectAuxDic["instagram_created_at"] = "$created_at"
            elif pl == "instagram_text" :
                projectAuxDic["instagram_text"] = "$text"
            elif pl == "instagram_video" :
                projectAuxDic["instagram_video"] = "$video"
            elif pl == "instagram_created_at_db" :               
                projectAuxDic["instagram_created_at_db"] = "$created_at_db"
            
        projectDic["$project"] = projectAuxDic
    else: 
        projectDic = None
        
    if groupList != None:
        groupDic      = {}
        groupAuxDic   = {}        
        for gl in groupList:
            #group by year
            if gl == "year" :
                groupAuxDic["_id"]           = "$yearI"
            #group by Mes
            if gl == "month" :
                groupAuxDic["_id"]           = { "$cond" : {"if" : { "$lt" : [ "$monthI", 10 ] }, "then": {"$concat" : ["$yearS","-","0","$monthS"]} ,"else":{"$concat" : ["$yearS","-","$monthS"]}}}
            #group by dia
            if gl == "day" :
                groupAuxDic["_id"]           = { "$toString" : {"$toDate" : {"$concat": ["$yearS","-", "$monthS", "-", "$dayOfMonthS"]}}}
            #group by hora
            if gl == "hour" :
                groupAuxDic["_id"]           = { "$substr": [{ "$toDate" :{"$concat": ["$yearS","-", "$monthS","-","$dayOfMonthS"," ","$hourS",":00:00"]}},0,13]}
            if gl == "hourComp" :
                groupAuxDic["_id"]           = { "$toDate" :{"$concat": ["$yearS","-", "$monthS","-","$dayOfMonthS"," ","$hourS",":00:00"]}}        
            #group by minute 
            if gl == "minute" :
                groupAuxDic["_id"]           = { "$toDate" :{"$concat": ["$yearS","-", "$monthS","-","$dayOfMonthS"," ","$hourS",":","$minuteS", ":00"]}}             
            if gl == "instagramid" :
                groupAuxDic["_id"]           = "$instagram_id"
            if gl == "count" :
                groupAuxDic["count"]         = { "$sum": 1 }
            if gl == "retweets" :
                groupAuxDic["retweets"]      = { "$max": "retweets"}                                
            if gl == "likes" :
                groupAuxDic["likes"]         = { "$max": "$likes"} 
            if gl == "replies" :
                groupAuxDic["replies"]       = { "$max": "$replies"} 
            if gl == "text" :
                 groupAuxDic["text"]         = "$text"
            
            if gl == "comment_videoid" :
                groupAuxDic["_id"]           = "$comment_videoID"
            if gl == "video_viewCount" :
                groupAuxDic["video_viewCount"] = { "$max": "$video_viewCount"} 
            if gl == "video_likeCount" :
                groupAuxDic["video_likeCount"] = { "$max": "$video_likeCount"} 
            if gl == "video_dislikeCount" :
                groupAuxDic["video_dislikeCount"] = { "$max": "$video_dislikeCount"} 
            if gl == "video_commentCount" :
                groupAuxDic["video_commentCount"] = { "$max": "$video_commentCount"} 
            if gl == "video_viewCount" :
                groupAuxDic["video_viewCount"] = { "$max": "$video_viewCount"} 
            if gl == "video_likeCount" :
                groupAuxDic["video_likeCount"] = { "$max": "$video_likeCount"} 
            if gl == "video_dislikeCount" :
                groupAuxDic["video_dislikeCount"] = { "$max": "$video_dislikeCount"} 
            if gl == "video_commentCount" :
                groupAuxDic["video_commentCount"] = { "$max": "$video_commentCount"} 
            if gl == "video_commentCount_positives" :
                groupAuxDic["positive"] = {'$sum' : {'$cond' : [{'$eq':['$overall','Positive']},1,0]}}
            if gl == "video_commentCount_negatives" :
                groupAuxDic["negative"] = {'$sum' : {'$cond' : [{'$eq':['$overall','Negative']},1,0]}}
            if gl == "video_commentCount_neutral" :
                groupAuxDic["neutral"] = {'$sum' : {'$cond' : [{'$eq':['$overall','Neutral']},1,0]}}
            if gl == "video_commentCount_negatives" :
                groupAuxDic["negative"] = {'$sum' : {'$cond' : [{'$eq':['$overall','Negative']},1,0]}}
            if gl == "video_commentCount_neutral" :
                groupAuxDic["neutral"] = {'$sum' : {'$cond' : [{'$eq':['$overall','Neutral']},1,0]}}
            if gl == "instagram_commentCount_positives" :
                groupAuxDic["positive"] = {'$sum' : {'$cond' : [{'$eq':['$overall','Positive']},1,0]}}
            if gl == "instagram_commentCount_negatives" :
                groupAuxDic["negative"] = {'$sum' : {'$cond' : [{'$eq':['$overall','Negative']},1,0]}}
            if gl == "instagram_commentCount_neutral" :
                groupAuxDic["neutral"] = {'$sum' : {'$cond' : [{'$eq':['$overall','Neutral']},1,0]}}
            
                    
        groupDic["$group"] = groupAuxDic
    else: 
        groupDic = None

        
    if sortList != None:
        sortDic       = {}
        sortAuxDic    = {}        
        for sl in sortList:     
            if sl == "_id":
                sortAuxDic["_id"] = 1 
            elif sl == "_idR":
                sortAuxDic["_id"] = -1
            elif sl == "instagram_text":
                sortAuxDic["text"] = 1
            elif sl == "youtube_text":
                sortAuxDic["video_title"] = 1
        sortDic["$sort"] = sortAuxDic 
    else: 
        sortDic = None
    return(projectDic,groupDic,sortDic,matchDic)

In [7]:
class myDictCoords:
        
    def AddItems(self, actual_dict, mapper):

        self.dict = actual_dict
        self.map  = mapper
    
    def __getitem__(self, key):
        try:
            if type(key) is str:
                key = float(key)
                
            if type(key) is not float:
                raise Exception("Invalid Index {}!".format(key))

            f = None
            for k,v in self.map.items():
                lower, higher = v
                if key >= lower and key <= higher:
                    f = k
            if f == None:
                raise Exception("Couldn't find {}!".format(key))
            return self.dict[f]
        except Exception as e:
            print(e)
            return -1
        
    def GetItemCoords(self, value):
        try:
            ret = None
            key = None
            for v,v1 in self.dict.items():
                if v1 == value:
                    key = v
                    break
            ret = self.map[key]
            if ret == None:
                raise Exception("Couldn't find {}!".format(key))
            return ret
        except Exception as e:
            print(e)
            return -1
        

In [8]:
def dfCommentsSent(id_busca, socialMedia):
        
    mtdic = {}
    dfAux = pd.DataFrame()
    if socialMedia == 'Instagram':
        mtdic["instagram_id"] = id_busca 
        dfComments  = con.AggregatePipe("instagram_comments",["yearI","yearS","monthS","monthI","dayOfMonthS","video_id","overall"],
                                          ["day","instagram_commentCount_negatives","instagram_commentCount_positives","instagram_commentCount_neutral"],["instagram_text"],mtdic)
        
    else:
        mtdic["video_commentID"] = id_busca #'bnBM2Zd54cw'
        dfComments  = con.AggregatePipe("youtube_comments",["yearI","yearS","monthS","monthI","dayOfMonthS","video_id","overall"],
                                          ["day","video_commentCount_negatives","video_commentCount_positives","video_commentCount_neutral"],["youtube_text"],mtdic)
    #if dfComments is not None:
    if not dfComments.empty:            
        dfComments.columns = ColumnsRename('dfCommentsSent',list(dfComments.columns))#['id', 'Negative','Neutro','Positive']   
        dfAux = dfComments.copy()
        if len(dfAux) >=50:
            dfAux["id"] = [d[1][0][:4] + 
              ((str(datetime.date(int(d[1][0][:4]), int(d[1][0][5:7]), int(d[1][0][8:10])).isocalendar()[1])) 
               if len(str(datetime.date(int(d[1][0][:4]), int(d[1][0][5:7]), int(d[1][0][8:10])).isocalendar()[1]))>=2 else ('0%s' % str(datetime.date(int(d[1][0][:4]), int(d[1][0][5:7]), int(d[1][0][8:10])).isocalendar()[1])))
              for d in dfAux.iterrows()]
            #dfAux["id"] = [str(datetime.date(int(d[1][0][:4]), int(d[1][0][5:7]), int(d[1][0][8:10])).isocalendar()[1]) +'-'+ d[1][0][:4] for d in dfAux.iterrows()]
            dfAux = dfAux.groupby("id").sum()
            dfAux = dfAux.reset_index()
            if len(dfAux) >=50:                
                l = list(dfAux['id'])
                l2 = []
                #for i in range(0,len(l),2):
                i = 0    
                while i<= len(l)-1:
                    try:
                        if int(l[i]) + 1 == int(l[i+1]):
                            #print(l[i][-2:] + '-' + l[i+1][-2:])
                            l2.append( int(l[i][:4] + l[i][-2:] +  l[i+1][-2:]))
                            l2.append( int(l[i][:4] + l[i][-2:] +  l[i+1][-2:]))
                            i+=2
                        else:
                            l2.append( int(l[i][:4] + l[i][-2:] +  l[i][-2:]))
                            i+=1
                    except:
                        l2.append( int(l[i][:4] + l[i][-2:] +  l[i][-2:]))
                        i+=1

                dfAux["id"] = l2
                dfAux = dfAux.groupby("id").sum()                            
                dfAux = dfAux.reset_index()
                if len(dfAux) >=50:
                    dfAux = dfComments.copy()                
                    dfAux["id"] = [d[1][0][:4] + 
                                  ((str(datetime.date(int(d[1][0][:4]), int(d[1][0][5:7]), int(d[1][0][8:10])).month)) 
                                   if len(str(datetime.date(int(d[1][0][:4]), int(d[1][0][5:7]), int(d[1][0][8:10])).month))>=2 else ('0%s' % str(datetime.date(int(d[1][0][:4]), int(d[1][0][5:7]), int(d[1][0][8:10])).month)))
                                  for d in dfAux.iterrows()]
                    dfAux = dfAux.groupby("id").sum()
                    dfAux = dfAux.reset_index()
                    if len(dfAux) >=50:                
                        l = list(dfAux['id'])
                        l2 = []
                        #for i in range(0,len(l),2):
                        i = 0    
                        while i<= len(l)-1:
                            try:
                                if int(l[i]) + 1 == int(l[i+1]):
                                    l2.append( int(l[i][:4] + l[i][-2:] +  l[i+1][-2:]))
                                    l2.append( int(l[i][:4] + l[i][-2:] +  l[i+1][-2:]))
                                    i+=2
                                else:
                                    l2.append( int(l[i][:4] + l[i][-2:] +  l[i][-2:]))
                                    i+=1
                            except:
                                l2.append( int(l[i][:4] + l[i][-2:] +  l[i][-2:]))
                                i+=1

                        dfAux["id"] = l2
                        dfAux = dfAux.groupby("id").sum()                            
                        dfAux = dfAux.reset_index()

    dfAux = dfAux.sort_values("id")
    dfAux = dfAux.reset_index()
    dfAux = dfAux.drop('index',axis = 1)    

    return dfAux

In [9]:
def dfCommentsText(id_busca,socialmedia):
    mtdic = {}
    if socialmedia == "Instagram":
        mtdic["instagram_id"] = id_busca
        commentField = "text"
        df = con.AggregatePipe("instagram_comments",[commentField,"overall","comment_instagram_likes"],None, None,mtdic)
        if df.empty:
            return df, [], []
        df.columns = ColumnsRename('dfCommentsText',list(df.columns),socialmedia)#['id', 'comment_likeCount','overall','comment_text']
        #field = "text"            
        #df.columns = ['id', 'comment_likeCount','comment_text','overall']
    else:
        mtdic["video_commentID"] = id_busca
        commentField = "comment_text"
        df = con.AggregatePipe("youtube_comments",[commentField,"overall","comment_youtube_likes"],None,None,mtdic)
        if df.empty:
            return df, [], []
        df.columns = ColumnsRename('dfCommentsText',list(df.columns),socialmedia)#['id', 'comment_likeCount','comment_text','overall']
        #field = "comment_text"    
    
    
        
    dfPos = df.loc[df['overall'] == 'Positive']
    comment_wordsPos = ''
    commentsListPos = list(dfPos['comment_text'])
    
    for comment in commentsListPos:
        comment = str(comment) 
        tokens = comment.split() 
        comment_wordsPos += " ".join(tokens)+" "    
    
    dfNeg = df.loc[df['overall'] == 'Negative']
    comment_wordsNeg = ''
    commentsListNeg = list(dfNeg['comment_text'])
    
    for comment in commentsListNeg:
        comment = str(comment) 
        tokens = comment.split() 
        comment_wordsNeg += " ".join(tokens)+" "

    return df, comment_wordsPos, comment_wordsNeg

    # iterate through the csv file 
    

In [10]:
def ColumnsRename(orig, columns, socialmedia = None):
    columnsList = []
    if orig == 'dataFrameRetrieveCount':
        for c in columns:
            if c == "_id":
                columnsList.append("id")
            elif c == "count":
                columnsList.append("Count")
            elif c == "socialmedia":
                columnsList.append("Social Media")
    elif orig == 'dataFrameRetrieveMaxTwitter':
        for c in columns:
                if c=="_id":
                    columnsList.append('id')
                elif c=="text":
                    columnsList.append('Text')
                elif c=="retweets":
                    columnsList.append('Retweets')                    
                elif c=='likes':
                    columnsList.append('Likes')                    
                elif c=='replies':
                    columnsList.append('Replies')
                elif c=="idtweet":
                    columnsList.append("idtweet")
                elif c=="socialmedia":
                    columnsList.append('Social Media')
                    
    elif orig == 'dataFrameRetrieveMaxYouTube':         
        for c in columns:            
            if c == "_id":
                columnsList.append('id')
            elif c == "text":
                columnsList.append('Text')
            elif c == "video_viewCount":
                columnsList.append('Views')
            elif c == "video_likeCount":
                columnsList.append('Likes')
            elif c == "video_dislikeCount":
                columnsList.append('Dislikes',)
            elif c == "video_commentCount":
                columnsList.append('Comments')
            elif c == "video_id":
                columnsList.append('Video_id')
            elif c == "socialmedia":
                columnsList.append('Social Media')                
    
    elif orig == 'dataFrameRetrieveMaxInstagram':
        for c in columns:
            if c=="_id":
                columnsList.append('id')
            elif c=="text":
                columnsList.append('Text')
            elif c=="instagram_like":
                columnsList.append('Instagram_Like')
            elif c=="instagram_comment":
                columnsList.append('Instagram_Comment')
            elif c=="instagram_id":
                columnsList.append('Instagram_ID')
            elif c=="socialmedia":
                columnsList.append('Social Media')
    elif orig == 'dfCommentsText':
        if socialmedia == "Instagram":
            for c in columns:
                if c=="_id":
                    columnsList.append('id')
                elif c=="text":
                    columnsList.append('comment_text')                    
                elif c=="overall":
                    columnsList.append('overall') 
                elif c=="likes":
                    columnsList.append('comment_likeCount')
        else:
            for c in columns:
                if c =="_id":
                    columnsList.append('id')
                elif c == "comment_text":
                    columnsList.append('comment_text')                    
                elif c == "overall":
                    columnsList.append('overall')
                elif c == "comment_likeCount":
                    columnsList.append('comment_likeCount')
    elif orig == 'dfCommentsSent':
        for c in columns:
            if c == "_id":
                columnsList.append('id')
            elif c == "negative":
                columnsList.append('Negative')
            elif c == "positive" :
                columnsList.append('Positive')
            elif c == "neutral":
                columnsList.append('Neutral')
    return columnsList

In [11]:
def dataFrameRetrieveCount():
    
    #count
    dfAnoCountOrigTwitter  = con.AggregatePipe("twitter",["yearI"],
                                      ["year","count"],["_id"],None)
    dfAnoCountOrigTwitter = dfAnoCountOrigTwitter.dropna()
    dfAnoCountOrigTwitter["socialmedia"] = ['Twitter' for d in dfAnoCountOrigTwitter.iterrows()]
    
    dfAnoCountOrigYouTube  = con.AggregatePipe("youtube_videos",["yearI"],
                                      ["year","count"],["_id"],None)
    dfAnoCountOrigYouTube = dfAnoCountOrigYouTube.dropna()
    dfAnoCountOrigYouTube["socialmedia"] = ['YouTube' for d in dfAnoCountOrigYouTube.iterrows()]
    
    dfAnoCountOrigInstagram  = con.AggregatePipe("instagram",["yearI"],
                                      ["year","count"],["_id"],None)
    dfAnoCountOrigInstagram = dfAnoCountOrigInstagram.dropna()
    dfAnoCountOrigInstagram["socialmedia"] = ['Instagram' for d in dfAnoCountOrigInstagram.iterrows()]
    
    dfMesCountOrigTwitter  = con.AggregatePipe("twitter",["yearI","yearS","monthS","monthI"],
                                      ["month","count"],["_id"],None)
    dfMesCountOrigTwitter = dfMesCountOrigTwitter.dropna()
    dfMesCountOrigTwitter["socialmedia"] = ['Twitter' for d in dfMesCountOrigTwitter.iterrows()]
    
    dfMesCountOrigYouTube  = con.AggregatePipe("youtube_videos",["yearI","yearS","monthS","monthI"],
                                      ["month","count"],["_id"],None)
    dfMesCountOrigYouTube = dfMesCountOrigYouTube.dropna()
    dfMesCountOrigYouTube["socialmedia"] = ['YouTube' for d in dfMesCountOrigYouTube.iterrows()]
    
    dfMesCountOrigInstagram  = con.AggregatePipe("instagram",["yearI","yearS","monthS","monthI"],
                                      ["month","count"],["_id"],None)
    dfMesCountOrigInstagram = dfMesCountOrigInstagram.dropna()
    dfMesCountOrigInstagram["socialmedia"] = ['Instagram' for d in dfMesCountOrigInstagram.iterrows()]
    
    dfDiaCountOrigTwitter  = con.AggregatePipe("twitter",["yearI","yearS","monthS","monthI","dayOfMonthS"],
                                      ["day","count"],["_id"],None)
    dfDiaCountOrigTwitter = dfDiaCountOrigTwitter.dropna()
    dfDiaCountOrigTwitter["socialmedia"] = ['Twitter' for d in dfDiaCountOrigTwitter.iterrows()]
    dfDiaCountOrigTwitter["_id"] = dfDiaCountOrigTwitter["_id"].apply(lambda x: str(x[0:10]))
    
    dfDiaCountOrigYouTube  = con.AggregatePipe("youtube_videos",["yearI","yearS","monthS","monthI","dayOfMonthS"],
                                      ["day","count"],["_id"],None)
    dfDiaCountOrigYouTube = dfDiaCountOrigYouTube.dropna()

    dfDiaCountOrigYouTube["socialmedia"] = ['YouTube' for d in dfDiaCountOrigYouTube.iterrows()]
    dfDiaCountOrigYouTube["_id"] = dfDiaCountOrigYouTube["_id"].apply(lambda x: str(x[0:10]))
    
    dfDiaCountOrigInstagram  = con.AggregatePipe("instagram",["yearI","yearS","monthS","monthI","dayOfMonthS"],
                                      ["day","count"],["_id"],None)
    dfDiaCountOrigInstagram = dfDiaCountOrigInstagram.dropna()
    dfDiaCountOrigInstagram["socialmedia"] = ['Instagram' for d in dfDiaCountOrigInstagram.iterrows()]
    dfDiaCountOrigInstagram["_id"] = dfDiaCountOrigInstagram["_id"].apply(lambda x: str(x[0:10]))

    
    dfHoraCountOrigTwitter = con.AggregatePipe("twitter",["yearI","yearS","monthS","monthI","dayOfMonthS","hourS"],
                                      ["hour","count"],["_id"],None)
    dfHoraCountOrigTwitter = dfHoraCountOrigTwitter.dropna()
    dfHoraCountOrigTwitter["socialmedia"] = ['Twitter' for d in dfHoraCountOrigTwitter.iterrows()]
    
    dfHoraCountOrigTwitter["_id"] = dfHoraCountOrigTwitter["_id"].apply(lambda x: str(x.replace("T"," ") ))
    
    dfHoraCountOrigYouTube = con.AggregatePipe("youtube_videos",["yearI","yearS","monthS","monthI","dayOfMonthS","hourS"],
                                      ["hour","count"],["_id"],None)
    dfHoraCountOrigYouTube = dfHoraCountOrigYouTube.dropna()
    dfHoraCountOrigYouTube["socialmedia"] = ['YouTube' for d in dfHoraCountOrigYouTube.iterrows()]
    
    dfHoraCountOrigYouTube["_id"] = dfHoraCountOrigYouTube["_id"].apply(lambda x: str(x.replace("T"," ") ))
    
    dfHoraCountOrigInstagram = con.AggregatePipe("instagram",["yearI","yearS","monthS","monthI","dayOfMonthS","hourS"],
                                      ["hour","count"],["_id"],None)
    dfHoraCountOrigInstagram = dfHoraCountOrigInstagram.dropna()
    dfHoraCountOrigInstagram["socialmedia"] = ['Instagram' for d in dfHoraCountOrigInstagram.iterrows()]

    dfHoraCountOrigInstagram["_id"] = dfHoraCountOrigInstagram["_id"].apply(lambda x: str(x.replace("T"," ") ))    
    
    dfMinCountOrigTwitter  = con.AggregatePipe("twitter",["yearI","yearS","monthS","monthI","dayOfMonthS","hourS","minuteS"],
                                        ["minute","count"],["_id"],None)
    dfMinCountOrigTwitter = dfMinCountOrigTwitter.dropna()
    dfMinCountOrigTwitter["socialmedia"] = ['Twitter' for d in dfMinCountOrigTwitter.iterrows()]
    
    dfMinCountOrigYouTube  = con.AggregatePipe("youtube_videos",["yearI","yearS","monthS","monthI","dayOfMonthS","hourS","minuteS"],
                                        ["minute","count"],["_id"],None)
    dfMinCountOrigYouTube = dfMinCountOrigYouTube.dropna()
    dfMinCountOrigYouTube["socialmedia"] = ['YouTube' for d in dfMinCountOrigYouTube.iterrows()]
    
    dfMinCountOrigInstagram  = con.AggregatePipe("instagram",["yearI","yearS","monthS","monthI","dayOfMonthS","hourS","minuteS"],
                                        ["minute","count"],["_id"],None)
    dfMinCountOrigInstagram = dfMinCountOrigInstagram.dropna()
    dfMinCountOrigInstagram["socialmedia"] = ['Instagram' for d in dfMinCountOrigInstagram.iterrows()]

    dfAnoCountOrig = dfAnoCountOrigTwitter.append(dfAnoCountOrigYouTube.append(dfAnoCountOrigInstagram))
    dfMesCountOrig = dfMesCountOrigTwitter.append(dfMesCountOrigYouTube.append(dfMesCountOrigInstagram))
    dfDiaCountOrig = dfDiaCountOrigTwitter.append(dfDiaCountOrigYouTube.append(dfDiaCountOrigInstagram))
    dfHoraCountOrig = dfHoraCountOrigTwitter.append(dfHoraCountOrigYouTube.append(dfHoraCountOrigInstagram))
    dfMinCountOrig = dfMinCountOrigTwitter.append(dfMinCountOrigYouTube.append(dfMinCountOrigInstagram))
    
    
    
    dfAnoCountOrig = dfAnoCountOrig.reset_index()
    dfAnoCountOrig = dfAnoCountOrig.drop('index',axis = 1)
    dfMesCountOrig = dfMesCountOrig.reset_index()
    dfMesCountOrig = dfMesCountOrig.drop('index',axis = 1)
    dfDiaCountOrig = dfDiaCountOrig.reset_index()
    dfDiaCountOrig = dfDiaCountOrig.drop('index',axis = 1)
    dfHoraCountOrig = dfHoraCountOrig.reset_index()
    dfHoraCountOrig = dfHoraCountOrig.drop('index',axis = 1)
    dfMinCountOrig = dfMinCountOrig.reset_index()
    dfMinCountOrig = dfMinCountOrig.drop('index',axis = 1)
        
    dfAnoCountOrig.columns = ColumnsRename('dataFrameRetrieveCount',list(dfAnoCountOrig.columns))#['id', 'Count','Social Media']

    dfMesCountOrig.columns = ColumnsRename('dataFrameRetrieveCount',list(dfMesCountOrig.columns))#['id', 'Count','Social Media']

    dfDiaCountOrig.columns = ColumnsRename('dataFrameRetrieveCount',list(dfDiaCountOrig.columns))#['id', 'Count','Social Media']

    dfHoraCountOrig.columns = ColumnsRename('dataFrameRetrieveCount',list(dfHoraCountOrig.columns))#['id', 'Count','Social Media']

    dfMinCountOrig.columns = ColumnsRename('dataFrameRetrieveCount',list(dfMinCountOrig.columns))#['id', 'Count','Social Media']
    
    #Maxretweets
    return(dfAnoCountOrig, dfMesCountOrig, dfDiaCountOrig, dfHoraCountOrig, dfMinCountOrig) 

In [12]:
def dataFrameRetrieveMaxTwitter():
    
    dfAnoMaxOrigTwitter  = con.AggregatePipe("twitter",["anoID",'tweet','retweets',"likes","replies","idtweet"],None,None,None)
    dfAnoMaxOrigTwitter["socialmedia"] = ['Twitter' for d in dfAnoMaxOrigTwitter.iterrows()]
    
    dfMesMaxOrigTwitter  = con.AggregatePipe("twitter",["mesID",'tweet','retweets',"likes","replies","idtweet"],None,None,None)
                              
    dfMesMaxOrigTwitter["socialmedia"] = ['Twitter' for d in dfMesMaxOrigTwitter.iterrows()]
    
    dfDiaMaxOrigTwitter  = con.AggregatePipe("twitter",["diaID",'tweet','retweets',"likes","replies","idtweet"]
                                      ,None,None,None)

    dfDiaMaxOrigTwitter["socialmedia"] = ['Twitter' for d in dfDiaMaxOrigTwitter.iterrows()]

    dfHoraMaxOrigTwitter = con.AggregatePipe("twitter",["hourID",'tweet','retweets',"likes","replies","idtweet"]
                                      ,None,None,None)
    dfHoraMaxOrigTwitter["socialmedia"] = ['Twitter' for d in dfHoraMaxOrigTwitter.iterrows()]
    
    dfMinMaxOrigTwitter  = con.AggregatePipe("twitter",["minID",'tweet','retweets',"likes","replies","idtweet"]
                                        ,None,None,None)
    
    dfMinMaxOrigTwitter["socialmedia"] = ['Twitter' for d in dfMinMaxOrigTwitter.iterrows()]    

    dfAnoMaxOrigTwitter.columns =  ColumnsRename('dataFrameRetrieveMaxTwitter',list(dfAnoMaxOrigTwitter.columns))#['id', "idtweet", 'Likes', 'Replies', 'Retweets', 'Text', 'Social Media']

    dfMesMaxOrigTwitter.columns =  ColumnsRename('dataFrameRetrieveMaxTwitter',list(dfMesMaxOrigTwitter.columns))#['id', "idtweet", 'Likes', 'Replies', 'Retweets', 'Text', 'Social Media'] 

    dfDiaMaxOrigTwitter.columns =  ColumnsRename('dataFrameRetrieveMaxTwitter',list(dfDiaMaxOrigTwitter.columns))#['id', "idtweet", 'Likes', 'Replies', 'Retweets', 'Text', 'Social Media']

    dfHoraMaxOrigTwitter.columns = ColumnsRename('dataFrameRetrieveMaxTwitter',list(dfHoraMaxOrigTwitter.columns))#['id', "idtweet", 'Likes', 'Replies', 'Retweets', 'Text', 'Social Media']

    dfMinMaxOrigTwitter.columns =  ColumnsRename('dataFrameRetrieveMaxTwitter',list(dfMinMaxOrigTwitter.columns))#['id', "idtweet", 'Likes', 'Replies', 'Retweets', 'Text', 'Social Media']
    
    return(dfAnoMaxOrigTwitter, dfMesMaxOrigTwitter, dfDiaMaxOrigTwitter, dfHoraMaxOrigTwitter, dfMinMaxOrigTwitter)

In [13]:
def dataFrameRetrieveMaxYouTube():
    
    dfAnoMaxOrigYouTube  = con.AggregatePipe("youtube_videos",["anoID",'video_title','video_viewCount','video_likeCount','video_dislikeCount','video_commentCount','video_id'],None,None,None)
    dfAnoMaxOrigYouTube = dfAnoMaxOrigYouTube.dropna()
    dfAnoMaxOrigYouTube["socialmedia"] = ['youtube' for d in dfAnoMaxOrigYouTube.iterrows()]
    
    dfMesMaxOrigYouTube  = con.AggregatePipe("youtube_videos",["mesID",'video_title','video_viewCount','video_likeCount','video_dislikeCount','video_commentCount','video_id'],None,None,None)                              
    dfMesMaxOrigYouTube = dfMesMaxOrigYouTube.dropna()
    dfMesMaxOrigYouTube["socialmedia"] = ['YouTube' for d in dfMesMaxOrigYouTube.iterrows()]
    
    dfDiaMaxOrigYouTube  = con.AggregatePipe("youtube_videos",["diaID",'video_title','video_viewCount', 'video_likeCount','video_dislikeCount','video_commentCount','video_id']
                                      ,None,None,None)
    dfDiaMaxOrigYouTube = dfDiaMaxOrigYouTube.dropna()
    dfDiaMaxOrigYouTube["socialmedia"] = ['YouTube' for d in dfDiaMaxOrigYouTube.iterrows()]

    dfHoraMaxOrigYouTube = con.AggregatePipe("youtube_videos",["hourID",'video_title','video_viewCount','video_likeCount','video_dislikeCount','video_commentCount','video_id']
                                      ,None,None,None)
    dfHoraMaxOrigYouTube = dfHoraMaxOrigYouTube.dropna()
    dfHoraMaxOrigYouTube["socialmedia"] = ['YouTube' for d in dfHoraMaxOrigYouTube.iterrows()]
    
    dfMinMaxOrigYouTube  = con.AggregatePipe("youtube_videos",["minID", 'video_title','video_viewCount','video_likeCount','video_dislikeCount','video_commentCount','video_id']
                                        ,None,None,None)
    dfMinMaxOrigYouTube = dfMinMaxOrigYouTube.dropna()
    dfMinMaxOrigYouTube["socialmedia"] = ['YouTube' for d in dfMinMaxOrigYouTube.iterrows()]
        
    dfAnoMaxOrigYouTube.columns = ColumnsRename('dataFrameRetrieveMaxYouTube',list(dfAnoMaxOrigYouTube.columns))#['id', 'Text', 'Views', 'Likes', 'Dislikes', 'Comments', 'Video_id',   'Social Media']#['id', 'Text', 'Comments', 'Dislikes', 'Video_id', 'Likes', 'Views', 'Social Media']

    dfMesMaxOrigYouTube.columns = ColumnsRename('dataFrameRetrieveMaxYouTube',list(dfMesMaxOrigYouTube.columns))#['id', 'Text', 'Views', 'Likes', 'Dislikes', 'Comments', 'Video_id',   'Social Media']#['id', 'Text', 'Comments', 'Dislikes', 'Video_id', 'Likes', 'Views', 'Social Media']

    dfDiaMaxOrigYouTube.columns = ColumnsRename('dataFrameRetrieveMaxYouTube',list(dfDiaMaxOrigYouTube.columns))#['id', 'Text', 'Views', 'Likes', 'Dislikes', 'Comments', 'Video_id',   'Social Media']#['id', 'Text', 'Comments', 'Dislikes', 'Video_id', 'Likes', 'Views', 'Social Media']

    dfHoraMaxOrigYouTube.columns = ColumnsRename('dataFrameRetrieveMaxYouTube',list(dfHoraMaxOrigYouTube.columns))#['id', 'Text', 'Views', 'Likes', 'Dislikes', 'Comments', 'Video_id',   'Social Media']#['id', 'Text', 'Comments', 'Dislikes', 'Video_id', 'Likes', 'Views', 'Social Media']

    #dfMinMaxOrigYouTube.columns = ['id', 'Text', 'Views', 'Likes', 'Dislikes', 'Comments', 'Video_id',   'Social Media']#['id', 'Text', 'Comments', 'Dislikes', 'Video_id', 'Likes', 'Views', 'Social Media']
    dfMinMaxOrigYouTube.columns = ColumnsRename('dataFrameRetrieveMaxYouTube',list(dfMinMaxOrigYouTube.columns))
    
    return(dfAnoMaxOrigYouTube, dfMesMaxOrigYouTube, dfDiaMaxOrigYouTube,
           dfHoraMaxOrigYouTube, dfMinMaxOrigYouTube)

In [14]:
def dataFrameRetrieveMaxInstagram():
    
    dfAnoMaxOrigInstagram  = con.AggregatePipe("instagram",["anoID",'instagram_text','instagram_likeCount','instagram_commentCount','instagram_id'],None,None,None)
    dfAnoMaxOrigInstagram["socialmedia"] = ['Instagram' for d in dfAnoMaxOrigInstagram.iterrows()]
    
    dfMesMaxOrigInstagram  = con.AggregatePipe("instagram",["mesID",'instagram_text','instagram_likeCount','instagram_commentCount','instagram_id'],None,None,None)
                              
    dfMesMaxOrigInstagram["socialmedia"] = ['Instagram' for d in dfMesMaxOrigInstagram.iterrows()]
    
    dfDiaMaxOrigInstagram  = con.AggregatePipe("instagram",["diaID",'instagram_text','instagram_likeCount','instagram_commentCount','instagram_id']
                                      ,None,None,None)

    dfDiaMaxOrigInstagram["socialmedia"] = ['Instagram' for d in dfDiaMaxOrigInstagram.iterrows()]

    dfHoraMaxOrigInstagram = con.AggregatePipe("instagram",["hourID",'instagram_text','instagram_likeCount','instagram_commentCount','instagram_id']
                                      ,None,None,None)
    dfHoraMaxOrigInstagram["socialmedia"] = ['Instagram' for d in dfHoraMaxOrigInstagram.iterrows()]
    
    dfMinMaxOrigInstagram  = con.AggregatePipe("instagram",["minID", 'instagram_text','instagram_likeCount','instagram_commentCount','instagram_id']
                                        ,None,None,None)
    
    dfMinMaxOrigInstagram["socialmedia"] = ['Instagram' for d in dfMinMaxOrigInstagram.iterrows()]
    
    dfAnoMaxOrigInstagram.columns = ColumnsRename('dataFrameRetrieveMaxInstagram',list(dfAnoMaxOrigInstagram.columns))#['id', 'Text', 'Instagram_Like', 'Instagram_Comment', 'Instagram_ID', 'Social Media']#['id', 'Instagram_Comment', 'Instagram_ID', 'Instagram_Like', 'Text', 'Social Media']

    dfMesMaxOrigInstagram.columns = ColumnsRename('dataFrameRetrieveMaxInstagram',list(dfMesMaxOrigInstagram.columns))#['id', 'Text', 'Instagram_Like', 'Instagram_Comment', 'Instagram_ID', 'Social Media']#['id', 'Instagram_Comment', 'Instagram_ID', 'Instagram_Like', 'Text', 'Social Media']

    dfDiaMaxOrigInstagram.columns = ColumnsRename('dataFrameRetrieveMaxInstagram',list(dfDiaMaxOrigInstagram.columns))#['id', 'Text', 'Instagram_Like', 'Instagram_Comment', 'Instagram_ID', 'Social Media']#['id', 'Instagram_Comment', 'Instagram_ID', 'Instagram_Like', 'Text', 'Social Media']

    dfHoraMaxOrigInstagram.columns = ColumnsRename('dataFrameRetrieveMaxInstagram',list(dfHoraMaxOrigInstagram.columns))#['id', 'Text', 'Instagram_Like', 'Instagram_Comment', 'Instagram_ID', 'Social Media']#['id', 'Instagram_Comment', 'Instagram_ID', 'Instagram_Like', 'Text', 'Social Media']

    dfMinMaxOrigInstagram.columns = ColumnsRename('dataFrameRetrieveMaxInstagram',list(dfMinMaxOrigInstagram.columns))#['id', 'Text', 'Instagram_Like', 'Instagram_Comment', 'Instagram_ID', 'Social Media']#['id', 'Instagram_Comment', 'Instagram_ID', 'Instagram_Like', 'Text', 'Social Media']
    
    return(dfAnoMaxOrigInstagram, dfMesMaxOrigInstagram, dfDiaMaxOrigInstagram,
           dfHoraMaxOrigInstagram, dfMinMaxOrigInstagram)

In [15]:
def draw_barPlot(df, graphName, ax1, canvas, filtroDate = "", gui = None):
               

    yticklabelsList = []
    xticklabelsList = []    
    if not df.empty:
        
        if graphName != 'BARSENT':
            byTitle = ""
            if filtroDate == 'ANO':
                yticklabelsList = sorted(set(list(df['id'])))
                xticklabelsList = sorted(set(list(df['id'])))
                byTitle = "Year"
            elif filtroDate == 'MES': 
                yticklabelsList = [mes[x[len(x)-2:]] for x in df['id'] ]
                xticklabelsList = sorted(set(list([x[len(x)-2:] for x in df['id'] ])))
                xticklabelsList = [mes[x] for x in xticklabelsList]
                byTitle = "Month"
            elif filtroDate == 'DIA':
                yticklabelsList = sorted(set(list([ x[8:] for x in df['id']])))
                xticklabelsList = sorted(set(list([ x[8:] for x in df['id']])))
                byTitle = "Day"
            elif filtroDate == 'HORA':
                yticklabelsList = sorted(set(list([ x[11:]+":00" for x in df['id']])))
                xticklabelsList = sorted(set(list([ x[11:]+":00" for x in df['id']])))
                byTitle = "Hour"
            elif filtroDate == 'MIN':
                yticklabelsList = sorted(set(list([ str(x)[11:16] for x in df['id']])))
                xticklabelsList = sorted(set(list([ str(x)[11:16] for x in df['id']])))
                byTitle = "Minute"
            #else:
                #yticklabelsList = list(df['id'])
                #byTitle = ""
                #xticklabelsList = [ x[11:]+":00" for x in df['id']]
 
        if graphName == 'TWITTER_BAR_RET':
            orient = 'v'
            ax1 = sns.barplot(x ="id", y="Retweets", data=df, picker=True, orient=orient, ax=ax1, color='#1DA1F2')
            title ='Totals Retweets by ' + byTitle
            ax1.set (xticklabels=xticklabelsList, title = title)
            ax1.set_ylabel("")
            ax1.set_xlabel("")
            coordName = 'TWITTER_BAR'
        elif graphName == 'TWITTER_BAR_LIKE':            
            orient = 'v'
            ax1 = sns.barplot(x ="id", y="Likes", data=df, picker=True, orient=orient, ax=ax1, color='#1DA1F2')
            title ='Totals Likes by ' + byTitle
            ax1.set (xticklabels=xticklabelsList, title = title)        
            ax1.set_ylabel("")
            ax1.set_xlabel("")            
            coordName = 'TWITTER_BAR'
            count_df = 0
        elif graphName == 'TWITTER_BAR_REP':            
            orient = 'v'
            ax1 = sns.barplot(x ="id", y="Replies", data=df, picker=True, orient=orient, ax=ax1, color='#1DA1F2')
            title ='Totals Replies by ' + byTitle
            ax1.set (xticklabels=xticklabelsList, title = title)        
            ax1.set_ylabel("")
            ax1.set_xlabel("")
            coordName = 'TWITTER_BAR'
        elif graphName == 'YOUTUBE_BAR_VIEW':
            orient = 'v'
            #ax = sns.barplot()
            ax1 = sns.barplot(x ="id", y="Views", data=df, picker=True, orient=orient, ax=ax1, color="#FF0000")
            title ='Totals Views by ' + byTitle
            ax1.set (xticklabels=xticklabelsList, title = title)
            ax1.set_ylabel("")
            ax1.set_xlabel("")
            coordName = 'YOUTUBE_BAR'
            
        elif graphName == 'YOUTUBE_BAR_LIKE':
            orient = 'v'
            ax1 = sns.barplot(x ="id", y="Likes", data=df, picker=True, orient=orient, ax=ax1, color="#FF0000")
            title ='Totals Likes by ' + byTitle
            ax1.set (xticklabels=xticklabelsList, title = title)
            ax1.set_ylabel("")
            ax1.set_xlabel("")
            coordName = 'YOUTUBE_BAR'
        elif graphName == 'YOUTUBE_BAR_DISLIKE':
            orient = 'v'
            ax1 = sns.barplot(x ="id", y="Dislikes", data=df, picker=True, orient=orient, ax=ax1, color="#FF0000")
            title ='Totals Dislikes by ' + byTitle
            ax1.set (xticklabels=xticklabelsList, title = title)
            ax1.set_ylabel("")
            ax1.set_xlabel("")
            coordName = 'YOUTUBE_BAR'
        elif graphName == 'YOUTUBE_BAR_COMMENT':
            orient = 'v'
            ax1 = sns.barplot(x ="id", y="Comments", data=df, picker=True, orient=orient, ax=ax1, color="#FF0000")
            title ='Totals Comments by ' + byTitle
            ax1.set (xticklabels=xticklabelsList, title = title)        
            ax1.set_ylabel("")
            ax1.set_xlabel("")
            coordName = 'YOUTUBE_BAR'
        elif graphName == 'INSTAGRAM_BAR_LIKE':
            orient = 'v'
            ax1 = sns.barplot(x ="id", y="Instagram_Like", data=df, picker=True, orient=orient, ax=ax1, color="#AC2CB5")
            title ='Totals Likes by ' + byTitle
            ax1.set (xticklabels=xticklabelsList, title = title)            
            ax1.set_ylabel("")
            ax1.set_xlabel("")
            coordName = 'INSTAGRAM_BAR'
        elif graphName == 'INSTAGRAM_BAR_COMMENT':
            orient = 'v'
            ax1 = sns.barplot(x ="id", y="Instagram_Comment", data=df, picker=True, orient=orient, ax=ax1, color="#AC2CB5")
            title ='Totals Comments by' + byTitle
            ax1.set (xticklabels=xticklabelsList, title = title)        
            ax1.set (title = title)        
            ax1.set_ylabel("")
            ax1.set_xlabel("")
            coordName = 'INSTAGRAM_BAR'
        elif graphName == 'GERAL_BAR':
            orient = 'v'
            socialmediasList = set(list(df['Social Media']))
            if 'Instagram' in socialmediasList and 'Twitter' in socialmediasList and 'YouTube' in socialmediasList:
                clrs = ['#AC2CB5','#1DA1F2','#FF0000']
            elif ('Instagram' in socialmediasList) and ('Twitter' not in socialmediasList and 'YouTube' not in socialmediasList):
                clrs = ['#AC2CB5']
            elif ('Twitter' in socialmediasList) and ('Instagram' not in socialmediasList and 'YouTube' not in socialmediasList):
                clrs = ['#1DA1F2']
            elif ('YouTube' in socialmediasList) and ('Instagram' not in socialmediasList and 'Twitter' not in socialmediasList):
                clrs = ['#FF0000']
            ax1 = sns.barplot(x ="id", y="Count", hue = "Social Media", data=df, picker=True,orient=orient, ax=ax1,palette=clrs)
            title ='Social Media Posts by ' + byTitle
            ax1.set (title = title)
            #print(xticklabelsList)
            ax1.set (xticklabels=xticklabelsList)
            ax1.legend(fontsize = 'small')
            ax1.set_ylabel("")
            ax1.set_xlabel("")
            coordName = 'GERAL_BAR'
            
        elif graphName == 'BARSENT': 
            
            orient = 'v'
            totals = [i+j+k for i,j,k in zip(df['Positive'], df['Negative'], df['Neutral'])]
            
            df['positive_percent'] = [i / j * 100 for i,j in zip(df['Positive'], totals)]
            df['negative_percent'] = [i / j * 100 for i,j in zip(df['Negative'], totals)]
            df['neutral_percent'] = [i / j * 100 for i,j in zip(df['Neutral'], totals)] 
            
            print(df['positive_percent'])
            print(df['negative_percent'])
            print(df['neutral_percent']) 
            ax1 = sns.barplot(x="id", y='negative_percent', data=df, label="negative_percent", color="r",orient = 'v',ax=ax1)
                        
            count = 0
            count_df = 0
            for p in ax1.patches:
                if p.get_height() > 0:
                    ax1.annotate('{:d}'.format(df['Negative'][count_df]), (p.get_x() + p.get_width()/2, p.get_height()/2),
                       ha='center', va='bottom',
                       color= 'black')
                count_df +=1
            
            ax1 = sns.barplot(x="id", y='neutral_percent', data=df, label="neutral_percent",bottom=df['negative_percent'], color="b",orient = 'v',ax=ax1)
            count = 0
            count_df = 0
            dflen = len(df['Neutral'])
            for p in ax1.patches:
                if count >= dflen:
                    if p.get_height() > 0.0:
                        ax1.annotate('{:d}'.format(df['Neutral'][count_df]), (p.get_x() + p.get_width()/2, p.get_y() + p.get_height()/2),
                           ha='center', va='bottom',
                           color= 'black')
                    count_df += 1
                count += 1
            
            ax1 = sns.barplot(x="id", y='positive_percent', data=df, label="positive_percent",bottom=[i+j for i,j in zip(df['neutral_percent'], df['negative_percent'])], color="g",orient = 'v',ax=ax1)
            count = 0
            count_df = 0
            for p in ax1.patches:
                if count >= dflen *2:
                    if p.get_height() > 0.0:
                        ax1.annotate('{:d}'.format(df['Positive'][count_df]), (p.get_x() + p.get_width()/2, p.get_y() + p.get_height()/2),
                           ha='center', va='bottom',
                           color= 'black')
                    count_df += 1
                count += 1
            if len(str(df['id'][0])) == 6:
                xticklabelsList = [x[0:4] + '/' + x[4:6] for x in df['id']]
            elif len(str(df['id'][0])) == 8:
                xticklabelsList = [str(x)[0:4] + '/' + str(x)[4:6] + '/' + str(x)[6:8] for x in df['id']]            
            else:
                xticklabelsList = [x[8:10] + '/' + x[5:7] + '/' + x[2:4] for x in df['id']]
            
            ax1.set_title ('Comments Evolution')
            ax1.set_xticklabels(labels=xticklabelsList, rotation=90)
            ax1.set_ylabel("")
            ax1.set_xlabel("")  
        
        elif graphName == 'YOUTUBE_BAR_COMMENT_LIKE':
            orient = 'h'
            clrs = []
            for i in df['overall']:
                if i == 'Positive':
                    clrs.append('green')
                elif i == 'Neutral':
                    clrs.append('blue')
                else:
                    clrs.append('red')
            ax1 = sns.barplot(x ="comment_likeCount", y="id", data=df, picker=True, orient=orient, ax=ax1, palette=clrs)
            title ='Most liked Comments by ' + byTitle            
            ax1.set (yticklabels="", title = title)
            ax1.set_ylabel("")
            ax1.set_xlabel("")
            coordName = 'YOUTUBE_BAR_COMMENT_LIKE'
            
        elif graphName == 'INSTAGRAM_BAR_COMMENT_LIKE':
            orient = 'h'
            clrs = []
            for i in df['overall']:
                if i == 'Positive':
                    clrs.append('green')
                elif i == 'Neutral':
                    clrs.append('blue')
                else:
                    clrs.append('red')
            #ax1 = sns.barplot(x ="id", y="likes", data=df, picker=True, orient=orient, ax=ax1, color="#AC2CB5")
            ax1 = sns.barplot(x ="comment_likeCount", y="id", data=df, picker=True, orient=orient, ax=ax1, palette=clrs)
            title ='Most liked Comments by ' + byTitle
            ax1.set (yticklabels="", title = title)
            ax1.set_ylabel("")
            ax1.set_xlabel("")
            coordName = 'INSTAGRAM_BAR_COMMENT_LIKE'
        

        ax1.tick_params(axis='x', which='major', pad=0)
        ax1.figure.tight_layout() #aqui#

        
        if graphName != 'BARSENT':
        
            countb = 0
            barsPos = {}
            barsMap = {}        
            barsMapV = {}
            if orient == 'h':
                count = df.index.min()
                for r in ax1.patches:
                    if isinstance(r, Rectangle):
                        barsPos[count] = df.loc[count]["id"]
                        y = round(r.get_y(),2)
                        y2 = round(y + r.get_height(),2)
                        barsMap[count] = (y,y2) 
                        #vertical
                        x = round(r.get_x(),2)
                        x2 = round(x + r.get_width(),2)
                        barsMapV[count] = (x,x2)         
                        count +=1                      
            elif orient == 'v':                
                count = df.index.min()
                for r in ax1.patches:
                    if isinstance(r, Rectangle):
                        barsPos[count] = df.loc[count]["id"]
                        x = round(r.get_x(),2)
                        x2 = round(x + r.get_width(),2)
                        barsMap[count] = (x,x2)
                        #vertical
                        y = round(r.get_y(),2)
                        y2 = round(y + r.get_height(),2)
                        barsMapV[count] = (y,y2)         
                        count +=1      

            dicBars = myDictCoords()
            dicBars.AddItems(barsPos, barsMap)
            #my_gui.dicCoordsChange(coordName, dicBars)
            gui.dicCoordsChange(coordName, dicBars)
            #dicCoordsChange(coordName, dicBars)
            
            dicBarsV = myDictCoords()
            dicBarsV.AddItems(barsPos, barsMapV)
            #my_gui.dicCoordsChange(coordName, dicBarsV,'V')
            gui.dicCoordsChange(coordName, dicBarsV,'V')
            #dicCoordsChange(coordName, dicBarsV,'V')
            
            barsHoriz = {}
            barsHMap = {}

            sns.despine(ax=ax1, right=True)
        
    else:

        ax1 = sns.barplot(ax=ax1)
#         dicBars = myDictCoords()
#         dicBars.AddItems({}, {})

# #        my_gui.dicCoordsChange(graphName, dicBars)
#         gui.dicCoordsChange(graphName, dicBars)
    canvas.draw()

In [16]:
def draw_heatPlot(df, graphName, ax1, canvas, radioOption, gui =None):#, countP = 0):#filterDate = ""):
#ToDO            
    if not df.empty:        
        if graphName == 'TWITTER_HEAT':# or graphName == 'TWITTER_HEAT_FULL':
            df = df.set_index('Text')
            if radioOption == "TWITTER_BAR_LIKE":
                fieldsListOrder = ["Likes", "Retweets", "Replies"]
                title = "Most Liked Tweets"
            elif radioOption == "TWITTER_BAR_RET":
                fieldsListOrder = ["Retweets", "Replies", "Likes"]
                title = "Most Retweeted Tweets"
            else: #radioOption == "TWITTER_BAR_REP":
                fieldsListOrder = ["Replies", "Likes", "Retweets"]
                title = "Most Replied Tweets"
            
            #df[['Retweets', 'Favorites']] = df[['Retweets', 'Favorites']]/10000            

            vmin = min(df[[fieldsListOrder[0], fieldsListOrder[1], fieldsListOrder[2]]].min())
            vmax = max(df[[fieldsListOrder[0], fieldsListOrder[1], fieldsListOrder[2]]].max())

            cmap = sns.color_palette("Blues")
            x_axis_labels = [fieldsListOrder[0], fieldsListOrder[1], fieldsListOrder[2]] # labels for x-axis
            #y_axis_labels = [x[:15] for x in list(df.index) ]
         
            ax1 = sns.heatmap(df[[fieldsListOrder[0], fieldsListOrder[1], fieldsListOrder[2]]],ax=ax1,cbar=False, vmin=vmin, vmax=vmax, annot=True, annot_kws={"size": 10},linewidths=0.1,cmap =cmap,xticklabels=x_axis_labels,fmt=",d")           
            #title = "Top
            ax1.set (title = title)

        elif graphName == 'YOUTUBE_HEAT':
            
            df = df.set_index('Text')
        
            if radioOption == "YOUTUBE_BAR_LIKE": 
                fieldsListOrder = ['Likes','Dislikes','Views','Comments']
                title = "Most Liked Videos"
            elif radioOption == "YOUTUBE_BAR_DISLIKE":
                fieldsListOrder = ['Dislikes','Views','Comments','Likes']
                title = "Most Disliked Videos"
            elif radioOption == "YOUTUBE_BAR_VIEW":
                fieldsListOrder = ['Views','Comments','Likes','Dislikes']
                title = "Most Views Videos"
            else: #radioOption == "TWITTER_BAR_REP":                
                fieldsListOrder = ['Comments','Likes','Dislikes','Views']
                title = "Most Commented Videos"
            
            #df[['Video_CommentCount','Video_DislikeCount','Video_LikeCount','Video_ViewCount']] = df[['Video_CommentCount','Video_DislikeCount','Video_LikeCount','Video_ViewCount']]/10000
            
            vmin = min(df[[fieldsListOrder[0], fieldsListOrder[1], fieldsListOrder[2], fieldsListOrder[3]]].min())
            vmax = max(df[[fieldsListOrder[0], fieldsListOrder[1], fieldsListOrder[2], fieldsListOrder[3]]].max())
            
            x_axis_labels = [fieldsListOrder[0], fieldsListOrder[1], fieldsListOrder[2], fieldsListOrder[3]] # labels for x-axis
            #y_axis_labels = [x[:15] for x in list(df.index) ]
            
            cmap = sns.color_palette("Reds")            
            ax1 = sns.heatmap(df[[fieldsListOrder[0], fieldsListOrder[1], fieldsListOrder[2], fieldsListOrder[3]]],ax=ax1,cbar=False, vmin=vmin, vmax=vmax, annot=True, annot_kws={"size": 10},linewidths=0.1,cmap =cmap, xticklabels=x_axis_labels,fmt=",d")
            ax1.set(title = title)
        
        elif graphName == 'INSTAGRAM_HEAT':

            df = df.set_index('Text')
            
            if radioOption == "INSTAGRAM_BAR_LIKE":
                fieldsListOrder = ['Instagram_Like','Instagram_Comment']
                title = "Most Liked Posts"
                x_axis_labels = ["Likes", "Comments"] # labels for x-axis
            else: #radioOption == "INSTAGRAM_COMMENT":
                fieldsListOrder = ['Instagram_Comment','Instagram_Like']
                title = "Most Commented Posts"
                x_axis_labels = ["Comments","Likes"]
            
            vmin = min(df[[fieldsListOrder[0], fieldsListOrder[1]]].min())
            vmax = max(df[[fieldsListOrder[0], fieldsListOrder[1]]].max())
            
            
            #y_axis_labels = [x[:15] for x in list(df.index) ]
            
            cmap = sns.color_palette("Purples")
            ax1 = sns.heatmap(df[[fieldsListOrder[0], fieldsListOrder[1]]],ax=ax1,cbar=False, vmin=vmin, vmax=vmax, annot=True, annot_kws={"size": 10},linewidths=0.1,cmap =cmap,xticklabels=x_axis_labels,fmt=",d")
            ax1.set (title = title)
        
        #if not "FULL" in graphName:
        ax1.tick_params(labelleft =False, rotation=0)
        ax1.tick_params(axis='x', which='major', pad=0)
        ax1.set_ylabel("")
        ax1.autoscale(True)
        #ax1.figure.tight_layout()

        heatPos = {}
        heatMap = {} 
        count = 0
        countb = 0
        if graphName == 'YOUTUBE_HEAT':
            colnumber = 4            
        elif graphName == 'TWITTER_HEAT':
            colnumber = 3
        else:
            colnumber = 2

        for t in ax1.texts:
            if count % colnumber == 0:              
                x,y = t.get_position()
                heatPos[countb] = countb#df.loc[count]["index"]
                y1 = y - 0.5 
                y2 = y + 0.5 
                heatMap[countb] = (y1,y2)        
                countb +=1
            count +=1    
        dicHeat = myDictCoords()
        dicHeat.AddItems(heatPos, heatMap)
        gui.dicCoordsChange(graphName, dicHeat)   

    else:

        df = pd.DataFrame()
        #ax = sns.heatmap(data=df,ax=ax)
        dicBars = myDictCoords()
        dicBars.AddItems({}, {})

        gui.dicCoordsChange(graphName, dicBars)
        
    canvas.draw()
    return len(ax1.texts) 

In [17]:
def draw_linePlot(df, graphName, ax1, canvas, filtroDate = ""):
#     print('draw_linePlot')
    yticklabelsList = []
    xticklabelsList = []    
    if not df.empty:
            
        byTitle = ""
        if filtroDate == 'ANO':
            yticklabelsList = list(df['id'])
            xticklabelsList = list(df['id'])
            byTitle = "Year"
        elif filtroDate == 'MES':   
            yticklabelsList = [mes[x[len(x)-2:]] for x in df['id'] ]
            xticklabelsList = [mes[x[len(x)-2:]] for x in df['id'] ]
            byTitle = "Month"
        elif filtroDate == 'DIA':
            yticklabelsList = [ x[8:] for x in df['id']]
            xticklabelsList = [ x[8:] for x in df['id']]
            byTitle = "Day"
        elif filtroDate == 'HORA':
            yticklabelsList = [ x[11:]+":00" for x in df['id']]
            xticklabelsList = [ x[11:]+":00" for x in df['id']]
            byTitle = "Hour"
        elif filtroDate == 'MIN':
            yticklabelsList = [ str(x)[11:16] for x in df['id']]
            xticklabelsList = [ str(x)[11:16] for x in df['id']]
            byTitle = "Minute"
        #else:
            #yticklabelsList = list(df['id'])
            #byTitle = ""
            #xticklabelsList = [ x[11:]+":00" for x in df['id']]
        df = df.drop("index", axis = 1)
        df = df.set_index("id")
        
        #clrDic = {'Instagram':'#AC2CB5', 'YouTube':'#FF0000', 'Twitter':'#1DA1F2'}
        
        if graphName == 'TWITTER_LINE':
            #ax1 = sns.barplot(x ="id", y="Retweets", data=df, picker=True, orient=orient, ax=ax1, color='#1DA1F2')
            ax1 = sns.lineplot(data=df, ax=ax1, palette = ['#1DA1F2','#1DA1F2','#1DA1F2'])#, picker=True, orient=orient, ax=ax1, color='#1DA1F2')
            #ax1 = sns.lineplot(data=df)
            title ='Totals Interactions by ' + byTitle
            #ax1.set (xticks=np.arange(0, 12, 5))
            #df.index.values
            ax1.set (xticks=list(df.index.values),xticklabels=xticklabelsList)
            
            ax1.set (title = title)
            ax1.set_ylabel("")
            ax1.set_xlabel("")
            
        elif graphName == 'YOUTUBE_LINE':
            #ax = sns.barplot()
            #ax1 = sns.barplot(x ="id", y="Views", data=df, picker=True, orient=orient, ax=ax1, color="#FF0000")            
            ax1 = sns.lineplot(data=df, ax=ax1, palette=['#FF0000','#FF0000','#FF0000','#FF0000'])
            title ='Totals Interactions by ' + byTitle
            ax1.set (xticks=list(df.index.values),xticklabels=xticklabelsList)
            ax1.set (title = title)
            ax1.set_ylabel("")
            ax1.set_xlabel("")
                    
        elif graphName == 'INSTAGRAM_LINE':
            #ax1 = sns.barplot(x ="id", y="Instagram_Comment", data=df, picker=True, orient=orient, ax=ax1, color="#AC2CB5")
            df.columns = ['Comments','Likes']
            ax1 = sns.lineplot(data=df, ax=ax1, palette=['#AC2CB5','#AC2CB5'])
            title ='Totals Interactions by ' + byTitle
            ax1.set (xticks=list(df.index.values),xticklabels=xticklabelsList)
            ax1.set (title = title)               
            ax1.set_ylabel("")
            ax1.set_xlabel("")
            
        sns.despine(ax=ax1, right=True)
        
    else:

        ax1 = sns.barplot(ax=ax1)
#         dicBars = myDictCoords()
#         dicBars.AddItems({}, {})

        #gui.dicCoordsChange(graphName, dicBars)
    canvas.draw()

In [18]:
def func(pct, allvals):
    absolute = int(round(pct/100*np.sum(allvals)))
    return "{:.1f}%\n({:d})".format(pct, absolute)

In [19]:
# wedges, texts, autotexts = ax.pie(data, autopct=lambda pct: func(pct, data),
#                                   textprops=dict(color="w"))

def draw_piePlot(df, graphName, ax1, canvas, gui = None):#, countP = 0):#filterDate = ""):
    if not df.empty:
        if graphName == 'GERAL_PIE':
            
            clrDic = {'Instagram':'#AC2CB5', 'YouTube':'#FF0000', 'Twitter':'#1DA1F2'}
            
            dfTotals = df.sort_values(['Social Media'])
            #dfTotals = dfTotals.set_index('Social Media').sum(level = 'Social Media')
            dfTotals = dfTotals.set_index('Social Media').groupby(level = 'Social Media').sum()
            clrs = []
            
            i=0
            labels = list(dfTotals.index)
            for l in labels:
                clrs.append(clrDic[l])
                labels[i] = "%s%s" % (l[0].upper(),l[1:])
                i +=1
            wedges, plt_labels, texts = ax1.pie(list(dfTotals['Count']),labels = labels,colors =clrs, autopct=lambda pct: func(pct, list(dfTotals['Count'])))
#             ax1.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%',
#         shadow=True, startangle=90)
            title = 'Social Media Data from %s' % db_name
            ax1.set(title = title)
            
        elif graphName == 'SENT_PIE':

            #clrDic = {'Negative':'#AC2CB5', 'Neutro':'#FF0000', 'Positive':'#1DA1F2'}
            clrs = ['r', 'g', 'b']
            df = df.sum() 
 
            labels = list(df.index[1:])
            values = list(df.values[1:])
            
            wedges, plt_labels, texts = ax1.pie(values, labels = labels,colors =clrs,autopct=lambda pct: func(pct, values))
            #title = 'Social Media Data from %s' % db_name
            title = 'Comments Percentage'
            ax1.set(title = title)

        #ax1.figure.tight_layout()
        ax1.figure.subplots_adjust(bottom=0.15, top = 0.87)
        
        if graphName == 'GERAL_PIE':
            
            make_picker(wedges)

            count=0
            piePos = {}
            pieMap = {}

            for r in ax1.patches:

                piePos[count] = labels[count]
                y = round(r.theta1,2)
                y2 = round(r.theta2,2)

                pieMap[count] = (y,y2)
                count +=1          

            dicPies = myDictCoords()
            dicPies.AddItems(piePos, pieMap)

            gui.dicCoordsChange(graphName, dicPies)   
            

    else:
        wedges, plt_labels = ax1.pie([0])        
        
#         dicBars = myDictCoords()
#         dicBars.AddItems({}, {})
#         gui.dicCoordsChange(graphName, dicPies)    
        
    canvas.draw()


        #return

In [20]:
def make_picker(wedges):

    for wedge in wedges:
            wedge.set_picker(True)

In [21]:
def fillDataFrameCount (df):
    socialMediaList = ['Twitter','YouTube','Instagram']
    tamSocialMediaList = len(socialMediaList)
    idList = set(list(df['id']))
    for idl in idList:
        if len(df.loc[df['id'] == idl]) != tamSocialMediaList:
                dfAux = pd.DataFrame(df.loc[df['id'] == idl])
                dfSMList = list(dfAux['Social Media'])
                for l in socialMediaList:
                    if l not in dfSMList:
                        df = df.append({'id' : dfAux['id'].values[0] , 'Count' : 0, 'Social Media' : l} , ignore_index=True)

    df = df.sort_values(['Social Media','id']) 
    df = df.reset_index()
    df = df.drop('index',axis=1)
    
    return df 

In [22]:
def TopFilter(df, graphName = 'TWITTER_BAR_RET', ini = 0, fim = 10, radioOption = ""):
    ini -= 1 
    
    if graphName == 'TWITTER_BAR_RET':
        field = ['id',"Retweets"]
    elif graphName == 'TWITTER_BAR_LIKE':
        field = ['id',"Likes"]
    elif graphName == 'TWITTER_BAR_REP':
        field = ['id',"Replies"]
    elif graphName == 'INSTAGRAM_BAR_LIKE':
        field = ['id','Instagram_Like']
    elif graphName == 'INSTAGRAM_BAR_COMMENT':
         field = ['id','Instagram_Comment']
    elif graphName == 'YOUTUBE_BAR_VIEW':
        field = ['id','Views']
    elif graphName == 'YOUTUBE_BAR_LIKE':
        field = ['id','Likes']
    elif graphName == 'YOUTUBE_BAR_DISLIKE':
        field = ['id','Dislikes']
    elif graphName == 'YOUTUBE_BAR_COMMENT':
        field = ['id','Comments']

    elif graphName == 'TWITTER_HEAT':
        if radioOption == "TWITTER_BAR_LIKE":
            field = ["Likes"]                
        elif radioOption == "TWITTER_BAR_RET":
            field = ["Retweets"]
        else: #radioOption == "TWITTER_BAR_REP":
            field = ["Replies"]
        #field = ['Retweets','Likes','Replies']
    elif graphName == 'YOUTUBE_HEAT':
        if radioOption == "YOUTUBE_BAR_LIKE": 
            field = ['Likes']
        elif radioOption == "YOUTUBE_BAR_DISLIKE":
            field = ['Dislikes']
        elif radioOption == "YOUTUBE_BAR_VIEW":
            field = ['Views']
        else: #radioOption == "TWITTER_BAR_REP":
            field = ['Comments']
    elif graphName == 'INSTAGRAM_HEAT':            
        if radioOption == "INSTAGRAM_BAR_LIKE":
            field = ['Instagram_Like']
        else: #radioOption == "INSTAGRAM_COMMENT":
            field = ['Instagram_Comment']
            
    elif graphName == 'YOUTUBE_BAR_COMMENT_LIKE':
        field = ['comment_likeCount']
    elif graphName == 'INSTAGRAM_BAR_COMMENT_LIKE':
        field = ['comment_likeCount']
    if not df.empty:            
        df = df.sort_values(by = field,ascending=False)
        listID = sorted(set(list(df['id']))) 

        dfAux = pd.DataFrame()
        
        if graphName == 'YOUTUBE_BAR_COMMENT_LIKE' or graphName == 'INSTAGRAM_BAR_COMMENT_LIKE':
            lin = 0
            for i in range(ini,fim):
                dfAuxLoop = df[lin:lin+1]
                dfAux = dfAux.append(dfAuxLoop,ignore_index=True)
                lin +=1
        else:
            for l in listID:
                dfAuxLoop = df.loc[df["id"] == l][ini:fim]
                dfAux = dfAux.append(dfAuxLoop,ignore_index=True)
        return dfAux
    else :
        return df
    

In [23]:
def SumTotals(df):#, graphName = 'TWITTER_BAR_RET', option = "Likes"):
    if not df.empty:
        df = df.groupby("id").sum()
        df = df.reset_index()
    return df

In [24]:
def MeanTotals(df):#, graphName = 'TWITTER_BAR_RET', option = "Likes"):
    if not df.empty:
        df = df.groupby("id").mean()
        df = df.reset_index() 
    return df

In [25]:
# mes = {'01': 'Jan','02': 'Feb','03': 'Mar','04': 'Apr','05': 'May','06': 'Jun',
#        '07': 'Jul','08': 'Aug','09': 'Sep','10': 'Oct','11': 'Nov','12': 'Dez'}
# mesNome = {'Jan' : '01','Feb':'02','Mar':'03','Apr':'04','May':'05','Jun':'06',
#         'Jul':'07','Aug':'08','Sep':'09','Oct':'10','Nov':'11','Dez':'12'}

In [26]:
mes = {'01': 'JAN','02': 'FEB','03': 'MAR','04': 'APR','05': 'MAY','06': 'JUN',
       '07': 'JUL','08': 'AUG','09': 'SEP','10': 'OCT','11': 'NOV','12': 'DEC'}
mesNome = {'JAN' : '01','FEB':'02','MAR':'03','APR':'04','MAY':'05','JUN':'06',
        'JUL':'07','AUG':'08','SEP':'09','OCT':'10','NOV':'11','DEC':'12'}

In [27]:
def draw_wordcloudPlot(comment_words,ax1, canvas, tipo):
    
    if tipo == 'POS':
        colormap = 'Greens'
        title = "Positive Word Cloud"
    else: 
        colormap = 'Reds'
        title = "Negative Word Cloud"

    stopwords = set(STOPWORDS)     
        
    if len(comment_words) > 0:

        x, y = np.ogrid[:700, :1800]
        a = 350  # x center, half width                                       
        b = 900 

        ellipse = ((x-a)/a)**2 + ((y-b)/b)**2 >= 1  # True for points inside the ellipse        
        ellipse = 255 * ellipse.astype(int)
        
        wordcloud = WordCloud(width = 1800, height = 700, 
                        background_color ='white', 
                        stopwords = stopwords,colormap = colormap, 
                        min_font_size = 10, mask=ellipse).generate(comment_words) 

        ax1.imshow(wordcloud, interpolation='bilinear')
        ax1.axis("off")
        ax1.set_title(title)
    
    canvas.draw()

    #plt.show() 

In [28]:
# client = ('mongodb://localhost/socialmediadb_amazonprime')
# #client = ('mongodb://localhost/socialmediadb_netflix2')
# con = Conexao (client)  
# dfAnoCountOrig, dfMesCountOrig, dfDiaCountOrig,  dfHoraCountOrig, dfMinCountOrig = dataFrameRetrieveCount()
# dfAnoMaxOrigTwitter, dfMesMaxOrigTwitter, dfDiaMaxOrigTwitter, dfHoraMaxOrigTwitter, dfMinMaxOrigTwitter = dataFrameRetrieveMaxTwitter()
# dfAnoMaxOrigYouTube, dfMesMaxOrigYouTube, dfDiaMaxOrigYouTube, dfHoraMaxOrigYouTube, dfMinMaxOrigYouTube = dataFrameRetrieveMaxYouTube()
# dfAnoMaxOrigInstagram, dfMesMaxOrigInstagram, dfDiaMaxOrigInstagram, dfHoraMaxOrigInstagram, dfMinMaxOrigInstagram = dataFrameRetrieveMaxInstagram()

In [29]:
class TKInterSentAnalisys():
    def __init__(self, master, dicGraphicsFilters):# newWindow):
        self.master = master        
        self.dicGraphicsFilters = dicGraphicsFilters
        self.df = pd.DataFrame()
        self.socialmedia_title = self.dicGraphicsFilters['SOCIALMEDIATITLE']
        self.socialmedia = self.dicGraphicsFilters['SOCIALMEDIA']
        self.master.title("%s Sentiment Analysis %s" % (self.socialmedia_title,self.socialmedia))
        self.dpi = 100
        self.dic_sent_anal = {}
        self.dfcommentlikes = pd.DataFrame()
        #self.dic_word_cloud = {}
        self.postID = ""
        self.master.state('zoomed')
        self.dicCoords = {}
        self.dicCoordsV = {}
        
        sns.set_style("darkgrid", {"axes.edgecolor":  ".8", 'axes.facecolor': 'white', 'grid.color': '.8'})

#       frame filtros

        self.frameCombosFiltros = tk.Label(self.master)#,bg = 'blue')        
        self.frameCombosFiltros.pack(side="top",expand=False,fill='none', pady=2)       
        
        #INICIO Combos filtros
        
        self.label_comboGeralYear = tk.Label(self.frameCombosFiltros, text = 'Year:')
        self.label_comboGeralYear.pack(side="left",expand=False, fill='none')
                
        self.comboGeralYear = ttk.Combobox(self.frameCombosFiltros,state="readonly", values= "", width=10)

        self.comboGeralYear.size()
        self.comboGeralYear.pack(side="left",expand=False, fill='x')

        self.label_comboGeralMonth = tk.Label(self.frameCombosFiltros, text = 'Month:')
        self.label_comboGeralMonth.pack(side="left",expand=False, fill='none')

        self.comboGeralMonth = ttk.Combobox(self.frameCombosFiltros,state="readonly", values="", width=10)
        self.comboGeralMonth.size()
        self.comboGeralMonth.pack(side="left",expand=False, fill='x')

        self.label_comboGeralDay = tk.Label(self.frameCombosFiltros, text = 'Day:')
        self.label_comboGeralDay.pack(side="left",expand=False, fill='x')

        self.comboGeralDay = ttk.Combobox(self.frameCombosFiltros,state="readonly", values="", width=10)
        self.comboGeralDay.size()
        self.comboGeralDay.pack(side="left",expand=False, fill='x')
        
        #######
        #FIM Combos filtros
        
        #Frame Esq 
        self.frameEsqL1 = tk.Label(self.master)#, bg = 'pink')        
        self.frameEsqL1.pack(side="left",expand=False,fill='y', pady=2, anchor = 'nw') 
        
        #list box
        
        self.frameListPost = tk.Label(self.frameEsqL1)        
        self.frameListPost.pack(side="top",expand=False,fill='x', anchor = 'n')
    
        if self.socialmedia == "Instagram":
            comboname = "Posts"
            listAux = sorted(list(set(dfAnoMaxOrigInstagram['id'])))
        else:
            comboname = "Videos"
            listAux = sorted(list(set(dfAnoMaxOrigYouTube['id'])))

        
        self.label_frameListPost = tk.Label(self.frameListPost, text = comboname + ":")
        self.label_frameListPost.pack(side="top",expand=False, fill='none', anchor = 'w')        
       
        scrollbar = Scrollbar(self.frameListPost, orient="v")
        self.listPost = Listbox(self.frameListPost, yscrollcommand=scrollbar.set, height = 15, width = 80, selectmode = 'single')
        #listbox = Listbox(frame, yscrollcommand=scrollbar.set)
        scrollbar.config(command=self.listPost.yview)
        scrollbar.pack(side="right", fill="y", anchor = "w")
        #listbox.pack(side=LEFT, fill=BOTH, expand=1)
        self.listPost.pack(side="left", expand = True, fill = 'both')
        
        self.listCombosGerais = [self.comboGeralYear, self.comboGeralMonth, self.comboGeralDay] 
                
        self.frameGraphPieSent = tk.Frame(self.frameEsqL1)          
        self.frameGraphPieSent.pack(side="top",expand=True,fill='both', pady=2, anchor = 'w')  

        self.figures_pie_sent = plt.Figure(figsize=(1,1), dpi=self.dpi)
        self.figures_pie_sent.patch.set_facecolor('#f0f0f0')
        self.ax_pie_sent = self.figures_pie_sent.add_subplot(111)

        self.canvas_pie_sent = FigureCanvasTkAgg(self.figures_pie_sent, master=self.frameGraphPieSent)
        self.canvas_pie_sent.get_tk_widget().pack(side="left",fill='both',expand=True)
        
        self.framebarcommentslike = tk.Frame(self.frameEsqL1)#, bg = 'green')          
        self.framebarcommentslike.pack(side="top",expand=True,fill='both', pady=2, anchor = 'w')  
        
        self.figures_bar_comments_likes = plt.Figure(figsize=(1,1), dpi=self.dpi)
        self.figures_bar_comments_likes.patch.set_facecolor('#f0f0f0')
        self.ax_bar_comments_likes = self.figures_bar_comments_likes.add_subplot(111)

        self.canvas_bar_comments_likes = FigureCanvasTkAgg(self.figures_bar_comments_likes, master=self.framebarcommentslike)
        self.canvas_bar_comments_likes.get_tk_widget().pack(side="left",fill='both',expand=True)
        self.dicCoords[self.socialmedia.upper() + '_BAR_COMMENT_LIKE'] = {}
        self.dicCoordsV[self.socialmedia.upper() + '_BAR_COMMENT_LIKE'] = {}
        
        #Frame Dir 
        #Frame linha1 Geral
        self.framelistSent = tk.Label(self.master)        
        self.framelistSent.pack(side="left",expand=True,fill='both', pady=2)

        self.frameDirL1 = tk.Label(self.framelistSent, height=100)        
        self.frameDirL1.pack(side="left",expand=True,fill='both', pady=2)       
        
        self.frameGraphSent = tk.Frame(self.frameDirL1)          
        self.frameGraphSent.pack(side="top",expand=True,fill='both', pady=2)  

        self.figures_sent_anal = plt.Figure(figsize=(1,1), dpi=self.dpi)
        self.figures_sent_anal.patch.set_facecolor('#f0f0f0')
        self.ax_sent_anal = self.figures_sent_anal.add_subplot(111)

        self.canvas_sent_anal = FigureCanvasTkAgg(self.figures_sent_anal, master=self.frameGraphSent)
        self.canvas_sent_anal.get_tk_widget().pack(side="left",fill='both',expand=True)
        
#         ############word cloud
        self.frameGraphWordCloud = tk.Frame(self.frameDirL1)#, bg = 'brown')          
        self.frameGraphWordCloud.pack(side="top",expand=True,fill='both', pady=2)        
            
        self.frameGraphWordCloudPos = tk.Frame(self.frameGraphWordCloud)         
        self.frameGraphWordCloudPos.pack(side="left",expand=True,fill='both', pady=2)        

        self.figures_word_cloudPos = plt.Figure(figsize=(1,1), dpi=self.dpi)
        self.figures_word_cloudPos.patch.set_facecolor('#f0f0f0')
        self.ax_word_cloudPos = self.figures_word_cloudPos.add_subplot(111)

        self.canvas_word_cloudPos = FigureCanvasTkAgg(self.figures_word_cloudPos, master=self.frameGraphWordCloudPos)
        self.canvas_word_cloudPos.get_tk_widget().pack(side="left",fill='both',expand=True)
        
        self.frameGraphWordCloudNeg = tk.Frame(self.frameGraphWordCloud)
        self.frameGraphWordCloudNeg.pack(side="left",fill='both',expand=True, pady=2) 

        self.figures_word_cloudNeg = plt.Figure(figsize=(1,1), dpi=self.dpi)
        self.figures_word_cloudNeg.patch.set_facecolor('#f0f0f0')
        self.ax_word_cloudNeg = self.figures_word_cloudNeg.add_subplot(111)

        self.canvas_word_cloudNeg = FigureCanvasTkAgg(self.figures_word_cloudNeg, master=self.frameGraphWordCloudNeg)
        self.canvas_word_cloudNeg.get_tk_widget().pack(side="left",fill='both',expand=True)
        self.connect()       
                
        self.DATATIPO_orig = self.dicGraphicsFilters['DATATIPO'] 
        self.DATAVAL_orig = self.dicGraphicsFilters['DATAVAL'] 
        self.dicGraphicsFilters['DATATIPO'] = "ANO"
        self.dicGraphicsFilters['DATAVAL'] = ""        
        self.filterDate()   
        self.update_comboGerais_values()
        self.comboGeralYear.current(0)        
        if self.DATATIPO_orig == "MES":         
            self.dicGraphicsFilters['DATATIPO'] = self.DATATIPO_orig
            self.dicGraphicsFilters['DATAVAL'] = self.DATAVAL_orig
            self.filterDate()   
            self.update_comboGerais_values()
            try:
                self.cur = self.comboGeralYear['values'].index(self.DATAVAL_orig[0:4])
            except: 
                self.cur = 0
            self.comboGeralYear.current(self.cur)
            #elf.cur = self.comboGeralYear['values'].index(dicGraphicsFilters['DATAVAL'][0:4])
            
        elif self.DATATIPO_orig == "DIA":
            self.dicGraphicsFilters['DATATIPO'] = "MES"
            self.dicGraphicsFilters['DATAVAL'] = self.DATAVAL_orig[0:4]
            self.filterDate()   
            self.update_comboGerais_values()
            self.cur = self.comboGeralYear['values'].index(self.DATAVAL_orig[0:4])
            self.comboGeralYear.current(self.cur)
                                        
            self.dicGraphicsFilters['DATATIPO'] = self.DATATIPO_orig
            self.dicGraphicsFilters['DATAVAL'] = self.DATAVAL_orig
            self.filterDate()   
            self.update_comboGerais_values()
            try:
                self.cur = self.comboGeralMonth['values'].index(mes[self.dicGraphicsFilters['DATAVAL'][5:7]])
            except: 
                self.cur = 0
            self.comboGeralMonth.current(self.cur)
        
        elif self.DATATIPO_orig == "HORA":   
            self.dicGraphicsFilters['DATATIPO'] = "MES"
            self.dicGraphicsFilters['DATAVAL'] = self.DATAVAL_orig[0:4]
            self.filterDate()   
            self.update_comboGerais_values()
            try:
                self.cur = self.comboGeralYear['values'].index(self.DATAVAL_orig[0:4])
            except: 
                self.cur = 0
            self.comboGeralYear.current(self.cur)
                                        
            self.dicGraphicsFilters['DATATIPO'] = "DIA"
            self.dicGraphicsFilters['DATAVAL'] = self.DATAVAL_orig[:7]
            self.filterDate()   
            self.update_comboGerais_values()
            try:
                self.cur = self.comboGeralMonth['values'].index(mes[self.dicGraphicsFilters['DATAVAL'][5:7]])
            except: 
                self.cur = 0
            self.comboGeralMonth.current(self.cur)
            
            self.dicGraphicsFilters['DATATIPO'] = self.DATATIPO_orig
            self.dicGraphicsFilters['DATAVAL'] = self.DATAVAL_orig
            self.filterDate()               
            self.update_comboGerais_values()
            try:
                self.cur = self.comboGeralDay['values'].index(self.dicGraphicsFilters['DATAVAL'][8:10])
            except: 
                self.cur = 0
            self.comboGeralDay.current(self.cur)
            
        self.dicGraphicsFilters['DATATIPO'] = self.DATATIPO_orig
        self.dicGraphicsFilters['DATAVAL'] = self.DATATIPO_orig
                    
    def connect(self):
        
        self.comboGeralYear.bind('<<ComboboxSelected>>', self.comboGeralYear_select)
        self.comboGeralMonth.bind('<<ComboboxSelected>>', self.comboGeralMonth_select)
        self.comboGeralDay.bind('<<ComboboxSelected>>', self.comboGeralDay_select)
        self.listPost.bind('<<ListboxSelect>>', self.ListPost_select)
        self.canvas_bar_comments_likes.mpl_connect('motion_notify_event', self.on_mouse_move_BAR_COMMENTS_LIKE)        
        
    def on_mouse_move_BAR_COMMENTS_LIKE(self, event):        
        #print('on_mouse_move_BAR_COMMENTS_LIKE')          
        if not self.dfcommentlikes.empty :
            if event.inaxes:            
                posX = event.xdata
                posY = event.ydata
                if ((posX !=None) and (posY !=None)):                
                    self.on_mouse_move(posX,posY, self.socialmedia.upper() + '_BAR_COMMENT_LIKE')
            else:
                self.on_mouse_move(0,0,self.socialmedia.upper() + '_BAR_COMMENT_LIKE', True)

    def comboGeralYear_select(self, event):        
        filterDate = self.comboGeralYear.get()
        if not filterDate == "":
            self.filterDateSet(filterDate)
        self.filterDate()
        self.update_comboGerais_values()
        
    def comboGeralMonth_select(self, event):
        if self.comboGeralMonth.get() == '':
            filterDate = "%s" % (self.comboGeralYear.get())
        else:
            filterDate = "%s-%s" % (self.comboGeralYear.get(),mesNome[self.comboGeralMonth.get()])
        self.filterDateSet(filterDate)
        self.filterDate()
        self.update_comboGerais_values()
    
    def comboGeralDay_select(self, event):
        if self.comboGeralDay.get() == '':
            filterDate = "%s-%s" % (self.comboGeralYear.get(),mesNome[self.comboGeralMonth.get()])
        else:
            filterDate = "%s-%s-%s" % (self.comboGeralYear.get(),mesNome[self.comboGeralMonth.get()],self.comboGeralDay.get())        
        self.filterDateSet(filterDate)
        self.filterDate()
        self.update_comboGerais_values()
    
    def ListPost_select(self, event): 
        item = self.listPost.get('anchor')
        values = list(self.listPost.get(0,'end'))
        item = values.index(item) 
        if self.socialmedia == "Instagram":
            self.postID = self.df.iloc[item].Instagram_ID            
        else:            
            self.postID = self.df.iloc[item].Video_id  
        self.Graphics()
                
    def filterDateSet(self, filterDate):
        print('filterDateSet INI')
        print(filterDate)
        if len(filterDate) == 0: #Ano
            self.dicGraphicsFilters['DATATIPO'] = 'ANO'            
        elif len(filterDate) == 4: #mes
            self.dicGraphicsFilters['DATATIPO'] = 'MES'
        elif len(filterDate) == 7: #dia
            self.dicGraphicsFilters['DATATIPO'] = 'DIA'
        elif len(filterDate) == 10: #hora            
            self.dicGraphicsFilters['DATATIPO'] = 'HORA'
        elif len(filterDate) == 13: #mim
            self.dicGraphicsFilters['DATATIPO'] = 'MIN'
        self.dicGraphicsFilters['DATAVAL'] = filterDate
#        print('filterDateSet FIM')
            
    def filterDate(self):        
#        print('filterDate INI')
        if self.dicGraphicsFilters['DATATIPO'] == 'ANO':
            if self.socialmedia == 'Instagram':
                self.df = dfAnoMaxOrigInstagram
            else:                
                self.df = dfAnoMaxOrigYouTube            
            
        if self.dicGraphicsFilters['DATATIPO'] == 'MES':
            if self.socialmedia == 'Instagram':                
                self.df = dfMesMaxOrigInstagram.loc[(dfMesMaxOrigInstagram['id']>= self.dicGraphicsFilters['DATAVAL'] +'-01') & 
                                                                    (dfMesMaxOrigInstagram['id']<=self.dicGraphicsFilters['DATAVAL'] +"-12")]
            else:
                self.df = dfMesMaxOrigYouTube.loc[(dfMesMaxOrigYouTube['id']>= self.dicGraphicsFilters['DATAVAL'] +'-01') &
                                                                  (dfMesMaxOrigYouTube['id']<=self.dicGraphicsFilters['DATAVAL'] +"-12")]                        
            
        elif self.dicGraphicsFilters['DATATIPO'] == 'DIA': #dia
            if self.socialmedia == 'Instagram':                                                      
                self.df = dfDiaMaxOrigInstagram.loc[(dfDiaMaxOrigInstagram['id']>=self.dicGraphicsFilters['DATAVAL'] + '-01') & 
                                                                    (dfDiaMaxOrigInstagram['id']<=self.dicGraphicsFilters['DATAVAL'] + "-31")]            
            else:
                self.df = dfDiaMaxOrigYouTube.loc[(dfDiaMaxOrigYouTube['id']>=self.dicGraphicsFilters['DATAVAL'] + '-01') & (dfDiaMaxOrigYouTube['id']<=self.dicGraphicsFilters['DATAVAL'] + "-31")]            

        elif self.dicGraphicsFilters['DATATIPO'] == 'HORA': #hora
            if self.socialmedia == 'Instagram': 
                self.df = dfHoraMaxOrigInstagram.loc[(dfHoraMaxOrigInstagram['id']>=self.dicGraphicsFilters['DATAVAL'] + ' 01') & 
                                                                     (dfHoraMaxOrigInstagram['id']<=self.dicGraphicsFilters['DATAVAL'] + " 23")]            
            else:
                self.df = dfHoraMaxOrigYouTube.loc[(dfHoraMaxOrigYouTube['id']>=self.dicGraphicsFilters['DATAVAL'] + ' 01') & 
                                                                   (dfHoraMaxOrigYouTube['id']<=self.dicGraphicsFilters['DATAVAL'] + " 23")]            
    def update_comboGerais_values(self):
        #print('update_comboGerais_values INI')
        if self.dicGraphicsFilters['DATATIPO'] == 'ANO':
            self.listCombosGerais[0]['values'] = [''] + sorted(list(set(self.df['id'])))   
            self.listCombosGerais[0].current(0)
            self.listCombosGerais[1]['values'] = ['']
            self.listCombosGerais[1].current(0)
            self.listCombosGerais[2]['values'] = ['']
            self.listCombosGerais[2].current(0)
        elif self.dicGraphicsFilters['DATATIPO'] == 'MES':
            self.update_comboGerais_values_indiv()
            self.listCombosGerais[1].current(0)
            self.listCombosGerais[2]['values'] = ['']
            self.listCombosGerais[2].current(0)
        elif self.dicGraphicsFilters['DATATIPO'] == 'DIA':
            self.update_comboGerais_values_indiv()
            self.listCombosGerais[2].current(0)
        elif self.dicGraphicsFilters['DATATIPO'] == 'HORA':
            self.update_comboGerais_values_indiv()
        #print('update_comboGerais_values FIM')#  
    
    def update_comboGerais_values_indiv(self):
        listElement = []
        for l in sorted(list(set(self.df['id']))):
            if self.dicGraphicsFilters['DATATIPO'] == 'MES':
                listElement.append(mes[l[-2:]])
                self.listCombosGerais[1]['values'] = [''] + listElement                   
            elif self.dicGraphicsFilters['DATATIPO'] == 'DIA':
                listElement.append(l[-2:])                
                self.listCombosGerais[2]['values'] = [''] + listElement
        self.update_list_post_values()               
        
    def update_list_post_values(self):  
        self.listPost.delete(0, END)
        for item in self.df['Text']:
            char_list = [item[j] for j in range(len(item)) if ord(item[j]) in range(65536)]
            itemAux = ''
            for j in char_list:
                itemAux = itemAux+j
            self.listPost.insert(END, itemAux)
    
    def on_mouse_move(self, posx, posy, graphName,limpar = False):
        #print('on_mouse_move')
        self.ax_bar_comments_likes.texts = []
        value = self.dicCoords[graphName][float(posy)]
        if not limpar:
            if value !=-1:
                v1,v2 = self.dicCoordsV[graphName].GetItemCoords(value)
                if posx <= v2:                                        
                    ind = self.dfcommentlikes.loc[(self.dfcommentlikes['id']==value)]["comment_text"].index        
                    text = ''
                    if ind.size > 0:            
                        ind = int(ind[0])
                        text = self.dfcommentlikes.loc[ind]["comment_text"]                
                    self.ax_bar_comments_likes.annotate(text, xy=(posx, posy), xytext=(posx, posy),  # 3 points vertical offset
                                textcoords="data",
                                bbox=dict(boxstyle="round", facecolor='yellow', alpha=0.8, edgecolor='black'),
                                ha='center', va='bottom', wrap=True)
        self.canvas_bar_comments_likes.draw_idle()
            
    def dicCoordsChange(self, dicCoordName,dic,orient = 'h'):   
        if orient == 'h':
            self.dicCoords[dicCoordName] = dic
        else:
            self.dicCoordsV[dicCoordName] = dic
       
        
    def GraphicPrepare(self):

        self.dic_sent_anal =  dfCommentsSent(self.postID,self.socialmedia)
        self.dfcommentlikes, self.word_cloud_list_pos, self.word_cloud_list_neg =  dfCommentsText(self.postID,self.socialmedia)

        
    def Graphics(self):
        
        self.GraphicPrepare()
        self.ax_pie_sent.cla()
        self.ax_bar_comments_likes.cla()
        self.ax_sent_anal.cla()        
        self.ax_word_cloudPos.cla()
        self.ax_word_cloudNeg.cla()
        draw_piePlot(self.dic_sent_anal,'SENT_PIE',self.ax_pie_sent, self.canvas_pie_sent,self) #,my_gui.newWindow)
        if self.socialmedia == 'Instagram':
            self.dfcommentlikes = TopFilter(self.dfcommentlikes, 'INSTAGRAM_BAR_COMMENT_LIKE', 1, 10)
            #draw_barPlot(self.dfcommentlikes,'INSTAGRAM_BAR_COMMENT_LIKE',self.ax_bar_comments_likes, self.canvas_bar_comments_likes,my_gui.sentanalysisgui,self)            
            draw_barPlot(self.dfcommentlikes,'INSTAGRAM_BAR_COMMENT_LIKE',self.ax_bar_comments_likes, self.canvas_bar_comments_likes,"",self)            
        else:
            self.dfcommentlikes = TopFilter(self.dfcommentlikes, 'YOUTUBE_BAR_COMMENT_LIKE', 1, 10)
            draw_barPlot(self.dfcommentlikes,'YOUTUBE_BAR_COMMENT_LIKE',self.ax_bar_comments_likes, self.canvas_bar_comments_likes,"", self)
            #draw_barPlot(self.dfcommentlikes,'YOUTUBE_BAR_COMMENT_LIKE',self.ax_bar_comments_likes, self.canvas_bar_comments_likes,my_gui.sentanalysisgui, self)
        #draw_barPlot(self.dic_sent_anal,'BARSENT',self.ax_sent_anal, self.canvas_sent_anal,my_gui.sentanalysisgui)        
        draw_barPlot(self.dic_sent_anal,'BARSENT',self.ax_sent_anal, self.canvas_sent_anal,"")        
        draw_wordcloudPlot(self.word_cloud_list_pos,self.ax_word_cloudPos,self.canvas_word_cloudPos,'POS')
        draw_wordcloudPlot(self.word_cloud_list_neg,self.ax_word_cloudNeg,self.canvas_word_cloudNeg,'NEG')
        
    
        
# dicData = {'DATATIPO'        : 'MES',
#            'DATAVAL'         : '2019',
#            'SOCIALMEDIA'     : 'Instagram',
#                   }        
        
        
# root = tk.Tk()
# my_gui = TKInterSentAnalisys(root, dicData)
# root.mainloop()        

In [30]:
class TKInterStatistics():
    def __init__(self, master,dicData):
        self.master = master
        self.dicData = dicData
        self.socialmedia_title = self.dicData['SOCIALMEDIATITLE']
        #print(self.dicData['SOCIALMEDIATITLE'])
        self.master.title("%s Statistics" % self.socialmedia_title)
        #self.master.title("Sentiment Analysis %s" % self.socialmedia) 
        if self.dicData['SOCIALMEDIATITLE'] == 'Nile Wilson':
            #print(1)
            self.twitter_following    = '406'
            self.twitter_followers    = '54.962'
            self.twitter_start_date   = "21/12/2011" 
            self.twitter_end_date     = "26/10/2020" 
            self.instagram_followers  = "497.430"
            self.instagram_following  =  "772"          
            self.instagram_start_date = "21/07/2013"
            self.instagram_end_date   = "09/11/2020"
            self.youtube_subscribers  = "1.430.000"                        
            self.youtube_start_date   = "17/11/2009"
            self.youtube_end_date     = "01/11/2020"
        elif self.dicData['SOCIALMEDIATITLE'] == 'Netflix':
            #print(2)
            self.twitter_following    = "1.649"
            self.twitter_followers    = "10.124.048"           
            self.twitter_start_date   = "03/10/2008" 
            self.twitter_end_date     = "15/09/2020" 
            self.instagram_followers  = "25.727.317"
            self.instagram_following  = "956"
            self.instagram_start_date = "13/08/2012"
            self.instagram_end_date   = "03/11/2020"            
            self.youtube_subscribers  = "18.200.00"                                    
            self.youtube_start_date   = "13/08/2012"
            self.youtube_end_date     = "02/11/2020"
        elif self.dicData['SOCIALMEDIATITLE'] == 'Amazon Prime Video' :
            #print(3)
            self.twitter_following    = "715"
            self.twitter_followers    = "1.773.927"           
            self.twitter_start_date   = "07/11/2008" 
            self.twitter_end_date     = "29/08/2020" 
            self.instagram_followers  = "1.421.339"
            self.instagram_following  = "514"
            self.instagram_start_date = "16/11/2016"
            self.instagram_end_date   = "06/09/2020"            
            self.youtube_subscribers  = "829.000"                        
            self.youtube_start_date   = "06/02/2014"
            self.youtube_end_date     = "08/09/2020"
        
        if self.dicData['DATATIPO'] == 'ANO':
            self.filtrotxt = ''            
        elif self.dicData['DATATIPO'] == 'MES':
            self.filtrotxt = ' Year ' + self.dicData['DATAVAL']
        elif self.dicData['DATATIPO'] == 'DIA':
            self.filtrotxt = ' Year ' + self.dicData['DATAVAL'][:4] + ' Month ' + mes[self.dicData['DATAVAL'][5:7]]
        elif self.dicData['DATATIPO'] == 'HORA':
            self.filtrotxt = ' Year ' + self.dicData['DATAVAL'][:4] + ' Month ' + mes[self.dicData['DATAVAL'][5:7]] + ' Day ' + self.dicData['DATAVAL'][8:10]
        elif self.dicData['DATATIPO'] == 'MIN':
            self.filtrotxt = ' Year ' + self.dicData['DATAVAL'][:4] + ' Month ' + mes[self.dicData['DATAVAL'][5:7]] + ' Day ' + self.dicData['DATAVAL'][8:10] + ' Hour ' + self.dicData['DATAVAL'][11:13]
            
        self.dicTwitterTextS = {'Followers:'                : self.twitter_followers,
                                'Following:'                : self.twitter_following,
                                'Tweets Posts Total:'       :'{:,}'.format(len(dfAnoMaxOrigTwitter)).replace(",","."),                                                           
                                'Tweets Likes Total:'       :'{:,}'.format(dfAnoMaxOrigTwitter['Likes'].sum()).replace(",","."),                        
                                'Tweets Likes Mean:'        :'{:,}'.format(round(dfAnoMaxOrigTwitter['Likes'].mean())).replace(",", "X").replace(".", ",").replace("X", "."),                           
                                'Tweets Replies Total:'     :'{:,}'.format(dfAnoMaxOrigTwitter['Replies'].sum()).replace(",","."),                        
                                'Tweets Replies Mean:'      :'{:,}'.format(round(dfAnoMaxOrigTwitter['Replies'].mean())).replace(",", "X").replace(".", ",").replace("X", "."),                            
                                'Retweets Total:'           :'{:,}'.format(dfAnoMaxOrigTwitter['Retweets'].sum()).replace(",","."),
                                'Retweets Mean:'            :'{:,}'.format(round(dfAnoMaxOrigTwitter['Retweets'].mean())).replace(",", "X").replace(".", ",").replace("X", "."),
                                'filler1'                   :'',                                
                                'filler2'                   :'',                                
                                'Start Date:'               : self.twitter_start_date, 
                                'End Date:'                 : self.twitter_end_date 
                               }

        self.dicInstagramTextS = {'Followers:'                : self.instagram_followers,
                                  'Following:'                : self.instagram_following,
                                  'Instagram Posts Totals:'   : '{:,}'.format(len(dfAnoMaxOrigInstagram)).replace(",","."),
                                  'Instagram Likes Total:'    : '{:,}'.format(dfAnoMaxOrigInstagram['Instagram_Like'].sum()).replace(",","."),
                                  'Instagram Likes Mean :'    : '{:,}'.format(round(dfAnoMaxOrigInstagram['Instagram_Like'].mean())).replace(",", "X").replace(".", ",").replace("X", "."),
                                  'Instagram Comments Total:' : '{:,}'.format(dfAnoMaxOrigInstagram['Instagram_Comment'].sum()).replace(",","."),
                                  'Instagram Comments Mean:'  : '{:,}'.format(round(dfAnoMaxOrigInstagram['Instagram_Comment'].mean())).replace(",", "X").replace(".", ",").replace("X", "."),
                                  'filler1'                   :'',
                                  'filler2'                   :'',
                                  'filler3'                   :'',                                  
                                  'filler4'                   :'',                                  
                                  'Start Date:'               : self.instagram_start_date, 
                                  'End Date:'                 : self.instagram_end_date
                                } 
        
        self.dicYouTubeTextS = {'Subscribers:'                : self.youtube_subscribers,
                                'filler1'                      :'',
                                'YouTube Posts Totals'         :'{:,}'.format(len(dfAnoMaxOrigYouTube)).replace(",","."),                                
                                'YouTube Likes Total:'         :'{:,}'.format(dfAnoMaxOrigYouTube['Likes'].sum()).replace(",","."),                           
                                'YouTube Likes Mean:'          :'{:,}'.format(round(dfAnoMaxOrigYouTube['Likes'].mean   ())).replace(",", "X").replace(".", ",").replace("X", "."),          
                                'YouTube Comments Total:'      :'{:,}'.format(dfAnoMaxOrigYouTube['Comments'].sum ()).replace(",","."),                           
                                'YouTube Comments Mean:'       :'{:,}'.format(round(dfAnoMaxOrigYouTube['Comments'].mean())).replace(",", "X").replace(".", ",").replace("X", "."),
                                'YouTube Views Total:'         :'{:,}'.format(dfAnoMaxOrigYouTube['Views'].sum()).replace(",","."),
                                'YouTube Views Total Mean:'    :'{:,}'.format(round(dfAnoMaxOrigYouTube['Views'].mean())).replace(",", "X").replace(".", ",").replace("X", "."),
                                'YouTube Dislikes Total:'      :'{:,}'.format(dfAnoMaxOrigYouTube['Dislikes'].sum ()).replace(",","."),                           
                                'YouTube Dislikes Mean:'       :'{:,}'.format(round(dfAnoMaxOrigYouTube['Dislikes'].mean())).replace(",", "X").replace(".", ",").replace("X", "."),                                                            
                                'Start Date:'                  : self.youtube_start_date, 
                                'End Date:'                    : self.youtube_end_date
                                }
        
        if not (self.dicData['TWITTER_STATS'].empty) :
            self.dicTwitterTextR = {
                                'Tweets Posts Total:'       :'{:,}'.format(len(self.dicData['TWITTER_STATS'])).replace(",","."),                                                           
                                'Tweets Likes Total:'       :'{:,}'.format(self.dicData['TWITTER_STATS']['Likes'].sum()).replace(",","."),                        
                                'Tweets Likes Mean:'        :'{:,}'.format(round(self.dicData['TWITTER_STATS']['Likes'].mean())).replace(",", "X").replace(".", ",").replace("X", "."),
                                'Tweets Replies Total:'     :'{:,}'.format(self.dicData['TWITTER_STATS']['Replies'].sum()).replace(",","."),                        
                                'Tweets Replies Mean:'      :'{:,}'.format(round(self.dicData['TWITTER_STATS']['Replies'].mean())).replace(",", "X").replace(".", ",").replace("X", "."),
                                'Retweets Total:'           :'{:,}'.format(self.dicData['TWITTER_STATS']['Retweets'].sum()).replace(",","."),
                                'Retweets Mean:'            :'{:,}'.format(round(self.dicData['TWITTER_STATS']['Retweets'].mean())).replace(",", "X").replace(".", ",").replace("X", "."),                        
                                'filler1'                   :'',
                                'filler2'                   :''
                                 }
        else:
            self.dicTwitterTextR = {
                                'Tweets Posts Total:'       :'',                                                   
                                'Tweets Likes Total:'       :'',
                                'Tweets Likes Mean:'        :'',
                                'Tweets Replies Total:'     :'',
                                'Tweets Replies Mean:'      :'',
                                'Retweets Total:'           :'',
                                'Retweets Mean:'            :'',
                                'filler1'                   :'',
                                'filler2'                   :'',
                                'filler3'                   :'',                                  
                                'filler4'                   :''
                                }
            
        if not (self.dicData['INSTAGRAM_STATS'].empty) :
            self.dicInstagramTextR = {
                                'Instagram Posts Totals:'   : '{:,}'.format(len(self.dicData['INSTAGRAM_STATS'])).replace(",","."),
                                'Instagram Likes Total:'    : '{:,}'.format(self.dicData['INSTAGRAM_STATS']['Instagram_Like'].sum()).replace(",","."),
                                'Instagram Likes Mean :'    : '{:,}'.format(round(self.dicData['INSTAGRAM_STATS']['Instagram_Like'].mean())).replace(",", "X").replace(".", ",").replace("X", "."),                                                    
                                'Instagram Comments Total:' : '{:,}'.format(self.dicData['INSTAGRAM_STATS']['Instagram_Comment'].sum()).replace(",","."),
                                'Instagram Comments Mean:'  : '{:,}'.format(round(self.dicData['INSTAGRAM_STATS']['Instagram_Comment'].mean())).replace(",", "X").replace(".", ",").replace("X", "."),                                
                                'filler1'                   :'',
                                'filler2'                   :'',
                                'filler3'                   :'',
                                'filler4'                   :''
                                }
        else:
            self.dicInstagramTextR = {
                                'Instagram Posts Totals:'   : '',
                                'Instagram Likes Total:'    : '',
                                'Instagram Likes Mean :'    : '',
                                'Instagram Comments Total:' : '',
                                'Instagram Comments Mean:'  : '',
                                'filler1'                   :'',
                                'filler2'                   :'',
                                'filler3'                   :'',                                  
                                'filler4'                   :''
                                } 
        
        if not (self.dicData['YOUTUBE_STATS'].empty) :
            self.dicYouTubeTextR = {
                                'YouTube Total Posted Videos'  :'{:,}'.format(len(self.dicData['YOUTUBE_STATS'])).replace(",","."),                                                                
                                'YouTube Likes Total:'         :'{:,}'.format(self.dicData['YOUTUBE_STATS']['Likes'].sum()).replace(",","."),                           
                                'YouTube Likes Mean:'          :'{:,}'.format(round(self.dicData['YOUTUBE_STATS']['Likes'].mean   ())).replace(",", "X").replace(".", ",").replace("X", "."),          
                                'YouTube Comments Total:'      :'{:,}'.format(self.dicData['YOUTUBE_STATS']['Comments'].sum ()).replace(",","."),                
                                'YouTube Comments Mean:'       :'{:,}'.format(round(self.dicData['YOUTUBE_STATS']['Comments'].mean())).replace(",", "X").replace(".", ",").replace("X", "."),
                                'YouTube Views Total:'         :'{:,}'.format(self.dicData['YOUTUBE_STATS']['Views'].sum()).replace(",","."),
                                'YouTube Views Total Mean:'    :'{:,}'.format(round(self.dicData['YOUTUBE_STATS']['Views'].mean())).replace(",", "X").replace(".", ",").replace("X", "."),
                                'YouTube Dislikes Total:'      :'{:,}'.format(self.dicData['YOUTUBE_STATS']['Dislikes'].sum ()).replace(",","."),                           
                                'YouTube Dislikes Mean:'       :'{:,}'.format(round(self.dicData['YOUTUBE_STATS']['Dislikes'].mean())).replace(",", "X").replace(".", ",").replace("X", ".")

                                   }
        else:
            self.dicYouTubeTextR = {
                                'YouTube Total Posted Videos:' :'',                                
                                'YouTube Likes Total:'         :'',                           
                                'YouTube Likes Mean:'          :'',                                          
                                'YouTube Comments Total:'      :'',
                                'YouTube Comments Mean:'       :'',
                                'YouTube Views Total:'         :'',
                                'YouTube Views Total Mean:'    :'',
                                'YouTube Dislikes Total:'      :'',                           
                                'YouTube Dislikes Mean:'       :''
                                }
            
        #DBE9F6 blue
#A49FCE purple
#FEDBCC red
  
        self.frameStatistics = tk.Frame(self.master)#,bg = 'gray')
        self.frameStatistics.pack(side="top",expand=True,fill='both')#(column = 0, row
        
        self.frameStatistics_Rel = tk.Frame(self.master)#,bg = 'gray')
        self.frameStatistics_Rel.pack(side="top",expand=True,fill='both')#(column = 0, row
        
        self.frameStatistics_tw = tk.LabelFrame(self.frameStatistics,text = 'Twitter Totals', font = ('Helvetica', 10, 'bold'), bg = '#DBE9F6')        
        self.frameStatistics_tw.pack(side="left",anchor = 'nw', expand=True,fill='x')#(column = 0, row = 0, sticky='nwse')
        
        self.frameStatistics_in = tk.LabelFrame(self.frameStatistics,text = 'Instagram Totals', font = ('Helvetica', 10, 'bold'), bg = '#D3CCE3')
        self.frameStatistics_in.pack(side="left",anchor = 'nw',expand=True,fill='x')#(column = 1, row = 0, sticky="nwse")
        
        self.frameStatistics_yt = tk.LabelFrame(self.frameStatistics,text = 'Youtube Totals', font = ('Helvetica', 10, 'bold') , bg = '#FEDBCC')                    
        self.frameStatistics_yt.pack(side="left",anchor = 'nw',expand=True,fill='x')#(column = 2, row = 0, sticky="nwse")
        
        self.frameStatisticsRel_tw = tk.LabelFrame(self.frameStatistics_Rel,text = 'Twitter Filters' + self.filtrotxt, bg = '#DBE9F6',font = ('Helvetica', 10, 'bold'))
        self.frameStatisticsRel_tw.pack(side="left",anchor = 'nw',expand=True,fill='x')#(column = 0, row = 1, sticky="nwse")
        
        self.frameStatisticsRel_in = tk.LabelFrame(self.frameStatistics_Rel,text = 'Instagram Filters' + self.filtrotxt, bg = '#D3CCE3',font = ('Helvetica', 10, 'bold') )
        self.frameStatisticsRel_in.pack(side="left",anchor = 'nw',expand=True,fill='x')#(column = 1, row = 1, sticky="nwse")
        
        self.frameStatisticsRel_yt = tk.LabelFrame(self.frameStatistics_Rel,text = 'YouTube Filters' + self.filtrotxt, bg = '#FEDBCC',font = ('Helvetica', 10, 'bold'))
        self.frameStatisticsRel_yt.pack(side="left",anchor = 'nw',expand=True,fill='x')#(column = 2, row = 1, sticky="N")
                
        
        if dicData['SOCIALMEDIA'] == '' or dicData['SOCIALMEDIA'] == 'Twitter':
            for k,v in self.dicTwitterTextS.items():
                self.twitterFrameLine = tk.Frame(self.frameStatistics_tw, bg = '#DBE9F6')
                self.twitterFrameLine.pack(side="top",expand=True,fill='x', ipadx = 20) 
                if 'filler' in k:                    
                    self.LabelTwitterStatsL = tk.Label(self.twitterFrameLine, text = "", bg= "#DBE9F6").pack(side="left", anchor = 'w')#.grid(sticky="W", column = 0, row = row)                
                    self.LabelTwitterStatsT = tk.Label(self.twitterFrameLine, text = "", bg = '#DBE9F6',font = ('Helvetica', 10, 'bold')).pack(side="right", anchor = 'e')#.grid(sticky="E", column = 1, row = row)                
                else:
                    self.LabelTwitterStatsL = tk.Label(self.twitterFrameLine, text = k, bg= "#DBE9F6").pack(side="left", anchor = 'w')#.grid(sticky="W", column = 0, row = row)                
                    self.LabelTwitterStatsT = tk.Label(self.twitterFrameLine, text = v, bg = '#DBE9F6',font = ('Helvetica', 10, 'bold')).pack(side="right", anchor = 'e')#.grid(sticky="E", column = 1, row = row)
                
        if dicData['SOCIALMEDIA'] == '' or dicData['SOCIALMEDIA'] == 'Instagram':
            for k,v in self.dicInstagramTextS.items():
                self.instagramFrameLine = tk.Frame(self.frameStatistics_in, bg = '#D3CCE3')
                self.instagramFrameLine.pack(side="top",expand=True,fill='x', ipadx = 20) 
                if 'filler' in k:                    
                    self.LabelInstagramStatsL = tk.Label(self.instagramFrameLine, text = "", bg = '#D3CCE3').pack(side="left", anchor = 'w')#.grid(sticky="W", column = 0, row = row)
                    self.LabelInstagramStatsT = tk.Label(self.instagramFrameLine, text = "", bg = '#D3CCE3', font = ('Helvetica', 10, 'bold')).pack(side="right", anchor = 'e')#.grid
                else:
                    self.LabelInstagramStatsL = tk.Label(self.instagramFrameLine, text = k, bg = '#D3CCE3').pack(side="left", anchor = 'w')#.grid(sticky="W", column = 0, row = row)
                    self.LabelInstagramStatsT = tk.Label(self.instagramFrameLine, text = v, bg = '#D3CCE3', font = ('Helvetica', 10, 'bold')).pack(side="right", anchor = 'e')#.grid(sticky="E", column = 1, 

        if dicData['SOCIALMEDIA'] == '' or dicData['SOCIALMEDIA'] == 'YouTube':
            for k,v in self.dicYouTubeTextS.items():
                self.YTFrameLine = tk.Frame(self.frameStatistics_yt, bg = '#FEDBCC') 
                self.YTFrameLine.pack(side="top",expand=True,fill='x', ipadx = 20)
                if 'filler' in k:                    
                    self.LabelInstagramStatsL = tk.Label(self.YTFrameLine, text = '', bg = '#FEDBCC').pack(side="left", anchor = 'w')#.grid(sticky="W", column = 0, row = row)
                    self.LabelInstagramStatsT = tk.Label(self.YTFrameLine, text = '', bg = '#FEDBCC', font = ('Helvetica', 10, 'bold')).pack(side="right", anchor = 'e')#.grid
                else:
                    self.LabelInstagramStatsL = tk.Label(self.YTFrameLine, text = k, bg = '#FEDBCC').pack(side="left", anchor = 'w')#.grid(sticky="W", column = 0, row = row)
                    self.LabelInstagramStatsT = tk.Label(self.YTFrameLine, text = v, bg = '#FEDBCC', font = ('Helvetica', 10, 'bold')).pack(side="right", anchor = 'e')#.grid(sticky="E", column = 1,                     
#         fill = len(self.dicYouTubeTextS) - len(self.dicTwitterTextS)
#         if dicData['SOCIALMEDIA'] == '' or dicData['SOCIALMEDIA'] == 'Twitter':            
#             for i in range(0,fill):
#                 self.twitterFrameLine = tk.Frame(self.frameStatistics_tw, bg = "#DBE9F6")
#                 self.twitterFrameLine.pack(side="top",expand=True,fill='x', ipadx = 20) 
#                 self.LabelTwitterStatsL = tk.Label(self.twitterFrameLine, bg = '#DBE9F6', text = ' ', font = ('Helvetica', 10, 'bold')).pack(side="left", anchor = 'w')
        
#         fill = len(self.dicYouTubeTextS) - len(self.dicInstagramTextS)
        
#         if dicData['SOCIALMEDIA'] == '' or dicData['SOCIALMEDIA'] == 'Instagram':
#             for i in range(0,fill):
#                 self.intagramFrameLine = tk.Frame(self.frameStatistics_in, bg = '#D3CCE3')
#                 self.intagramFrameLine.pack(side="top",expand=True,fill='x', ipadx = 20 ) 
#                 self.LabelintagramStatsL = tk.Label(self.intagramFrameLine,  text = ' ', bg = '#D3CCE3',font = ('Helvetica', 10, 'bold')).pack(side="left", anchor = 'w')
            
#         row = 0
        if dicData['SOCIALMEDIA'] == '' or dicData['SOCIALMEDIA'] == 'Twitter':
            for k,v in self.dicTwitterTextR.items():
                self.twitterFrameLine = tk.Frame(self.frameStatisticsRel_tw, bg = '#DBE9F6')
                self.twitterFrameLine.pack(side="top",expand=True,fill='both', ipadx = 20)            
                if 'filler' in k:
                    self.LabelTwitterStatsL = tk.Label(self.twitterFrameLine, text = '', bg = '#DBE9F6').pack(side="left", fill='both', anchor = 'w')#.grid(sticky="W", column = 0, row = row)
                    self.LabelTwitterStatsT = tk.Label(self.twitterFrameLine, text = '', bg = '#DBE9F6', font = ('Helvetica', 10, 'bold')).pack(side="right", fill='both', anchor = 'e')#grid(
                else:
                    self.LabelTwitterStatsL = tk.Label(self.twitterFrameLine, text = k, bg = '#DBE9F6').pack(side="left", fill='both', anchor = 'w')#.grid(sticky="W", column = 0, row = row)
                    self.LabelTwitterStatsT = tk.Label(self.twitterFrameLine, text = v, bg = '#DBE9F6', font = ('Helvetica', 10, 'bold')).pack(side="right", fill='both', anchor = 'e')#grid(sticky="E", column = 1, row = row)

        if dicData['SOCIALMEDIA'] == '' or dicData['SOCIALMEDIA'] == 'Instagram':
            for k,v in self.dicInstagramTextR.items():
                self.instagramFrameLine = tk.Frame(self.frameStatisticsRel_in, bg = '#D3CCE3')
                self.instagramFrameLine.pack(side="top",expand=True,fill='x', ipadx = 20) 
                if 'filler' in k:
                    self.LabelInstagramStatsL = tk.Label(self.instagramFrameLine, text = '',  bg = '#D3CCE3').pack(side="left", anchor = 'w')#.grid(sticky="W", column = 0, row = row)
                    self.LabelInstagramStatsT = tk.Label(self.instagramFrameLine, text = '', bg = '#D3CCE3', font = ('Helvetica', 10, 'bold')).pack(side="right", anchor = 'e')#.grid(
                else:
                    self.LabelInstagramStatsL = tk.Label(self.instagramFrameLine, text = k,  bg = '#D3CCE3').pack(side="left", anchor = 'w')#.grid(sticky="W", column = 0, row = row)
                    self.LabelInstagramStatsT = tk.Label(self.instagramFrameLine, text = v, bg = '#D3CCE3', font = ('Helvetica', 10, 'bold')).pack(side="right", anchor = 'e')#.grid(sticky="E", column = 1, 

        if dicData['SOCIALMEDIA'] == '' or dicData['SOCIALMEDIA'] == 'YouTube':
            for k,v in self.dicYouTubeTextR.items():
                self.YTFrameLine = tk.Frame(self.frameStatisticsRel_yt, bg = '#FEDBCC')
                self.YTFrameLine.pack(side="top",expand=True,fill='x', ipadx = 20) 
                if 'filler' in k:
                    self.LabelInstagramStatsL = tk.Label(self.YTFrameLine, text = '', bg = '#FEDBCC').pack(side="left", anchor = 'w')#.grid(sticky="W", column = 0, row = row)
                    self.LabelInstagramStatsT = tk.Label(self.YTFrameLine, text = '', bg = '#FEDBCC', font = ('Helvetica', 10, 'bold')).pack(side="right", anchor = 'e')#.grid(sticky="E", column = 1, 
                else:
                    self.LabelInstagramStatsL = tk.Label(self.YTFrameLine, text = k, bg = '#FEDBCC').pack(side="left", anchor = 'w')#.grid(sticky="W", column = 0, row = row)
                    self.LabelInstagramStatsT = tk.Label(self.YTFrameLine, text = v, bg = '#FEDBCC', font = ('Helvetica', 10, 'bold')).pack(side="right", anchor = 'e')#.grid
                
                
#         fill = len(self.dicYouTubeTextR) - len(self.dicTwitterTextR)
#         if dicData['SOCIALMEDIA'] == '' or dicData['SOCIALMEDIA'] == 'Twitter':                        
#             for i in range(0,fill):
#                 self.twitterFrameLine = tk.Frame(self.frameStatisticsRel_tw, bg = '#DBE9F6')
#                 self.twitterFrameLine.pack(side="top",expand=True,fill='x', ipadx = 20) 
#                 self.LabelTwitterStatsL = tk.Label(self.twitterFrameLine, text = ' ', bg = '#DBE9F6', font = ('Helvetica', 10, 'bold')).pack(side="left", anchor = 'w')                
                    
#         if dicData['SOCIALMEDIA'] == '' or dicData['SOCIALMEDIA'] == 'Instagram':
#             fill = len(self.dicYouTubeTextR) - len(self.dicInstagramTextR)
#             for i in range(0,fill):
#                 self.intagramFrameLine = tk.Frame(self.frameStatisticsRel_in, bg = '#D3CCE3')
#                 self.intagramFrameLine.pack(side="top",expand=True,fill='x', ipadx = 20) 
#                 self.LabelintagramStatsL = tk.Label(self.intagramFrameLine,  text = ' ', bg = '#D3CCE3', font = ('Helvetica', 10, 'bold')).pack(side="left", anchor = 'w')
                        
# root = tk.Tk()
# my_gui = TKInterStatistics(root)#, figures_dic)
# root.mainloop()
        

In [32]:
#refactor 2
class TKInterGUI():
    
    def __init__(self, master, title):
        
        #Class variables
        self.master = master
        self.default_font = tkFont.nametofont("TkDefaultFont")
        self.default_font.configure(size=10)
        self.frames_dic = {}
        self.ax = {}
        self.canvas = {}
        self.dicCoords = {}
        self.dicCoordsV = {}
        self.dicDF = {}
        self.dicDF_StatsSent = {}
        self.dicCombosHeatGraph = {}
        self.listCombosGerais = []
        self.db_name = db_name
        #self.a = ''
        self.title = title

        #self.sentanalysisgui = ""
        self.dicGraphicsFilters = {'DATATIPO':'ANO','DATAVAL':'',
                                   'SOCIALMEDIA'   : '',
                                   'AMOUNTYEARS'   : 'ALL',
                                   'YEARSGRAPH'    : [],
                                   'GERAL_BAR'     : 'GERAL_BAR',
                                   'TWITTER_BAR'   : tk.StringVar(), 
                                   'INSTAGRAM_BAR' : tk.StringVar(),
                                   'YOUTUBE_BAR'   : tk.StringVar(),                                   
                                   'TWITTER_HEAT'  : "", 
                                   'INSTAGRAM_HEAT': "",
                                   'YOUTUBE_HEAT'  : "",
                                   'UPDATECOMBOS'  : True
                                  } 
        
        self.dicGraphicsFromTo = { 'TWITTER_HEAT'  : [1,11], 
                                   'INSTAGRAM_HEAT': [1,11],
                                   'YOUTUBE_HEAT'  : [1,11],
                                   'UPDATECOMBOS'  : True
                                  } 
        
        self.dicHeatCels = {'TWITTER_HEAT'  : 0, 
                            'INSTAGRAM_HEAT': 0,
                            'YOUTUBE_HEAT'  : 0}
        
        self.listdicGraphicsFilters = []
        self.pick_execute = 'home'
        #config
        self.master.state('zoomed')
        windowName = "%s Database Visualization" % self.title
        self.master.title(windowName)
        self.dpi = 100 
        
        #window        
        
        self.initScreen()
        self.initMenu()
        self.Connect()
        
        
    def initMenu(self):
        #MENU
        self.menubar = Menu(self.master)
        self.master.config(menu=self.menubar)
        self.menubar.add_command(label="Home", command = self.home)
        self.menubar.add_command(label="Statistics", command= self.show_statistics)
        
        self.geral_show = BooleanVar(self.master, True)
        self.twitter_show = BooleanVar(self.master, True)
        self.instagram_show = BooleanVar(self.master, True)
        self.youtube_show = BooleanVar(self.master, True)
        
        self.bar_line_show = BooleanVar(self.master, True)
        self.horiz_vert_show = BooleanVar(self.master, True)
        
        self.sent_anal_menubar = Menu(self.menubar, tearoff=0)
        self.sent_anal_menubar.add_command(label="Instagram", command=self.show_sent_anal_instagram)
        self.sent_anal_menubar.add_command(label="YouTube", command=self.show_sent_anal_youtube)
        self.menubar.add_cascade(label="Sentiment Analysis",menu=self.sent_anal_menubar)
        
        self.social_media_menubar = Menu(self.menubar, tearoff=0)
        self.social_media_menubar.add_radiobutton(label="Bars",  value=1, variable=self.bar_line_show, command=self.showLineBarsGraphs)
        self.social_media_menubar.add_radiobutton(label="Lines", value=0, variable=self.bar_line_show, command=self.showLineBarsGraphs)
        self.social_media_menubar.add_separator()
        self.social_media_menubar.add_radiobutton(label="Horizontal",  value=1, variable=self.horiz_vert_show, command=self.showHorizVert)
        self.social_media_menubar.add_radiobutton(label="Vertical", value=0, variable=self.horiz_vert_show, command=self.showHorizVert)
        self.social_media_menubar.add_separator()
        self.social_media_menubar.add_checkbutton(label="Geral", onvalue=1, offvalue=0, variable=self.geral_show, command=self.showhideGraphs)
        self.social_media_menubar.add_checkbutton(label="Twitter", onvalue=1, offvalue=0, variable=self.twitter_show, command=self.showhideGraphs)
        self.social_media_menubar.add_checkbutton(label="Instagram", onvalue=1, offvalue=0, variable=self.instagram_show,command=self.showhideGraphs)
        self.social_media_menubar.add_checkbutton(label="YouTube",  onvalue=1, offvalue=0, variable=self.youtube_show,command=self.showhideGraphs)
        self.menubar.add_cascade(label="Layout",menu=self.social_media_menubar)
        #FIM MENU
    
    def initScreen(self):#, show_twitter = True, show_instagram = True, show_youtube= True):       
                
        #Frames linhas principais - Master Child
        self.exec_pick = True

        self.frames_dic['GERAL_FILTER'] = tk.Frame(self.master)#,bg = 'orange')
        self.frames_dic['GERAL_FILTER'].pack(side="top",expand=False,fill='none', pady=2)

        self.frames_dic['GERAL'] = tk.Frame(self.master)#,bg = 'purple' )        

        self.frames_dic['TWITTER'] = tk.Frame(self.master)#,bg = 'blue')                                    
        self.frames_dic['INSTAGRAM'] = tk.Frame(self.master)#,bg = 'pink')            
        self.frames_dic['YOUTUBE'] = tk.Frame(self.master)#,bg = 'red')

        #FIM Frames linhas principais - Master Child

        #Frames linhas Geral Filter - GERAL_FILTER Child

        #years label combo
        self.label_qdt_years = tk.Label(self.frames_dic['GERAL_FILTER'], text = 'Years Quantity:')
        self.label_qdt_years.pack(side="left",expand=False, fill='x')            

        amountYearsValues = [str(i) for i in range(len(list(set(dfAnoCountOrig['id'])))-1,0,-1)]
        amountYearsValues.insert(0, 'ALL')

        self.comboQtdYears = ttk.Combobox(self.frames_dic['GERAL_FILTER'],state="readonly", values=amountYearsValues, width=10)
        self.comboQtdYears.current(0)
        self.comboQtdYears.size()
        self.comboQtdYears.pack(side="left",expand=False, fill='x')

        self.dicGraphicsFilters['YEARSGRAPH'] = sorted(list(set(dfAnoCountOrig['id'])),reverse = True)

        #Social Media label combo
        self.labelSocialMedia = tk.Label(self.frames_dic['GERAL_FILTER'], text = 'Social Media:')
        self.labelSocialMedia.pack(side="left",expand=False, fill='x')  

        self.comboSocialMedia = ttk.Combobox(self.frames_dic['GERAL_FILTER'],state="readonly", values=['','Twitter','Instagram','YouTube'], width=10)
        self.comboSocialMedia.current(0)
        self.comboSocialMedia.size()
        self.comboSocialMedia.pack(side="left",expand=False, fill='x')

        #Year label combo            
        self.label_comboGeralYear = tk.Label(self.frames_dic['GERAL_FILTER'], text = 'Year:')
        self.label_comboGeralYear.pack(side="left",expand=False, fill='none')

        listAux = [''] + sorted(list(set(dfAnoCountOrig['id'])))
        self.comboGeralYear = ttk.Combobox(self.frames_dic['GERAL_FILTER'],state="readonly", values= listAux, width=10)
        self.comboGeralYear.current(0)
        self.comboGeralYear.size()
        self.comboGeralYear.pack(side="left",expand=False, fill='x')

        #Month label combo            
        self.label_comboGeralMonth = tk.Label(self.frames_dic['GERAL_FILTER'], text = 'Month:')
        self.label_comboGeralMonth.pack(side="left",expand=False, fill='none')

        self.comboGeralMonth = ttk.Combobox(self.frames_dic['GERAL_FILTER'],state="readonly", values="", width=10)
        #self.comboGeralMonth.current(len(list(set(dfAnoMaxOrigTwitter['id'])))-1)
        self.comboGeralMonth.size()
        self.comboGeralMonth.pack(side="left",expand=False, fill='x')

        #Day label combo            
        self.label_comboGeralDay = tk.Label(self.frames_dic['GERAL_FILTER'], text = 'Day:')
        self.label_comboGeralDay.pack(side="left",expand=False, fill='x')

        self.comboGeralDay = ttk.Combobox(self.frames_dic['GERAL_FILTER'],state="readonly", values="", width=10)
        #self.comboGeralDay.current(len(list(set(dfAnoMaxOrigTwitter['id'])))-1)
        self.comboGeralDay.size()
        self.comboGeralDay.pack(side="left",expand=False, fill='x')

        #Min label combo
        self.label_comboGeralHour = tk.Label(self.frames_dic['GERAL_FILTER'], text = 'Hour:')
        self.label_comboGeralHour.pack(side="left",expand=False, fill='x')

        self.comboGeralHour = ttk.Combobox(self.frames_dic['GERAL_FILTER'],state="readonly", values="", width=10)
        self.comboGeralHour.size()
        self.comboGeralHour.pack(side="left",expand=False, fill='x')

        #FIM Frames linhas Geral Filter - GERAL_FILTER Child

        #Frames linhas Geral Graphics - GERAL Child

        self.frames_dic['GERAL_PIE'] = tk.Frame(self.frames_dic['GERAL'])#,bg = 'orange')
        
        self.frames_dic['GERAL_BAR'] = tk.Frame(self.frames_dic['GERAL'])#,bg = 'orange')

        #Fim Frames linhas Geral Graphics - GERAL_ Child

        #Frames linhas Twitter Graphics - TWITTER Child

        self.frames_dic['TWITTER_BAR'] = tk.Frame(self.frames_dic['TWITTER'])#,bg = 'gray')

        self.frames_dic['TWITTER_LINE'] = tk.Frame(self.frames_dic['TWITTER'])#,bg = 'gray')
        #self.frames_dic['TWITTER_LINE'].pack(side="left",expand=True,fill='both')

        self.frames_dic['TWITTER_HEAT'] = tk.Frame(self.frames_dic['TWITTER'])#,bg = 'gray')            

        self.frames_dic['TWITTER_FILTER'] = tk.LabelFrame(self.frames_dic['TWITTER'],text = 'Filters')#,bg = 'gray')


        #FIM Frames linhas Twitter Graphics - TWITTER Child

        #Frames linhas Instagram Graphics - INSTAGRAM Child

        self.frames_dic['INSTAGRAM_BAR'] = tk.Frame(self.frames_dic['INSTAGRAM'])#,bg = 'gray')
        
        self.frames_dic['INSTAGRAM_LINE'] = tk.Frame(self.frames_dic['INSTAGRAM'])#,bg = 'gray')

        self.frames_dic['INSTAGRAM_HEAT'] = tk.Frame (self.frames_dic['INSTAGRAM'])#,bg = 'gray')

        self.frames_dic['INSTAGRAM_FILTER'] = tk.LabelFrame(self.frames_dic['INSTAGRAM'],text = 'Filters')#, bg = 'gray')            

        #FIM Frames linhas Instagram Graphics - INSTAGRAM Child

        #Frames linhas Youtube Graphics - YOUTUBE Child
        self.frames_dic['YOUTUBE_BAR'] = tk.Frame(self.frames_dic['YOUTUBE'])#,bg = 'gray')
        
        self.frames_dic['YOUTUBE_LINE'] = tk.Frame(self.frames_dic['YOUTUBE'])#,bg = 'gray')

        self.frames_dic['YOUTUBE_HEAT'] = tk.Frame(self.frames_dic['YOUTUBE'])#,bg = 'gray')

        self.frames_dic['YOUTUBE_FILTER'] = tk.LabelFrame(self.frames_dic['YOUTUBE'],text = 'Filters')#, bg = 'gray')

        #FIM Frames linhas Youtube Graphics - YOUTUBE Child

        #Frames Radio Twitter - TWITTER_BAR Child
        self.frames_dic['TWITTER_RADIO'] = tk.LabelFrame(self.frames_dic['TWITTER_BAR'], text = 'Oprions')#,bg = 'gray')
        self.frames_dic['TWITTER_RADIO'].pack(side="left",expand=False,fill='both')

        self.radioTwitter = tk.Radiobutton(self.frames_dic['TWITTER_RADIO'], text="Likes   ", variable= self.dicGraphicsFilters['TWITTER_BAR'], value='TWITTER_BAR_LIKE', command = lambda : self.on_press_radio())
        self.radioTwitter.pack(side="top", anchor = 'w')

        self.radioTwitter = tk.Radiobutton(self.frames_dic['TWITTER_RADIO'], text="Retweets", variable= self.dicGraphicsFilters['TWITTER_BAR'], value='TWITTER_BAR_RET', command = lambda : self.on_press_radio()) 
        self.radioTwitter.pack(side="top", anchor = 'w')

        self.radioTwitter = tk.Radiobutton(self.frames_dic['TWITTER_RADIO'], text="Replies   ", variable= self.dicGraphicsFilters['TWITTER_BAR'], value='TWITTER_BAR_REP', command = lambda : self.on_press_radio())
        self.radioTwitter.pack(side="top", anchor = 'w')

        self.dicGraphicsFilters['TWITTER_BAR'].set('TWITTER_BAR_LIKE')
        
#         self.comboGraphValueOptTwitter = ttk.Combobox(self.frames_dic['TWITTER_RADIO'],state="readonly", values=["Totals","Means"], width=10)
#         self.comboGraphValueOptTwitter.current(0)
#         self.comboGraphValueOptTwitter.size()
#         self.comboGraphValueOptTwitter.pack(side="top",expand=False, fill='none', anchor = 'w')
#         self.dicGraphicsFilters['TWITTER_GRAPHVALUE'] = "TOTALS"

        #FIM Frames Radio Twitter - TWITTER_BAR Child

        #FIM Frames Combo Twitter - TWITTER_FILTER Child                       
        self.label_comboTopNTwitter = tk.Label(self.frames_dic['TWITTER_FILTER'], text = 'From - To:')
        self.label_comboTopNTwitter.pack(side="top",expand=False, fill='x', anchor = 'n')

        self.comboTopNTwitter = ttk.Combobox(self.frames_dic['TWITTER_FILTER'],state="readonly", values='', width=10)
        #self.update_comboTopN_values_indiv()
        self.comboTopNTwitter.current()
        self.comboTopNTwitter.size()
        self.comboTopNTwitter.pack(side="top",expand=False, fill='none', anchor = 'nw')

        self.label_comboHeatTwitter = tk.Label(self.frames_dic['TWITTER_FILTER'], text = 'Years:')
        self.label_comboHeatTwitter.pack(side="top",expand=False, fill='x', anchor = 'n')

        self.comboHeatTwitter = ttk.Combobox(self.frames_dic['TWITTER_FILTER'],state="readonly", values=sorted(list(set(dfAnoMaxOrigTwitter['id']))), width=10)
        self.comboHeatTwitter.current(len(list(set(dfAnoMaxOrigTwitter['id'])))-1)
        self.comboHeatTwitter.size()
        self.comboHeatTwitter.pack(side="top",expand=False, fill='none', anchor = 'nw')             
        self.dicGraphicsFilters['TWITTER_HEAT'] = self.comboHeatTwitter.get() 

        #FIM Frames Combo Twitter - TWITTER_FILTER Child

        #Frames Radio Instagram - INSTAGRAM_RADIO Child

        self.frames_dic['INSTAGRAM_RADIO'] = tk.LabelFrame(self.frames_dic['INSTAGRAM_BAR'], text = 'Oprions')#, text='Options', bg = 'gray')            
        self.frames_dic['INSTAGRAM_RADIO'].pack(side="left",expand=False,fill='both')

        self.radioInstagram = tk.Radiobutton(self.frames_dic['INSTAGRAM_RADIO'], text="Likes", variable= self.dicGraphicsFilters['INSTAGRAM_BAR'], value='INSTAGRAM_BAR_LIKE',command = lambda : self.on_press_radio()) 
        self.radioInstagram.pack(side="top", anchor = 'w')
        self.radioInstagram = tk.Radiobutton(self.frames_dic['INSTAGRAM_RADIO'], text="Comments", variable= self.dicGraphicsFilters['INSTAGRAM_BAR'], value='INSTAGRAM_BAR_COMMENT',command = lambda : self.on_press_radio())
        self.radioInstagram.pack(side="top", anchor = 'w')

        self.dicGraphicsFilters['INSTAGRAM_BAR'].set('INSTAGRAM_BAR_LIKE')

        #FIM Frames Radio Instagram - INSTAGRAM_RADIO Child

        #Frames Combo Instagram - INSTAGRAM_FILTER Child

        self.label_comboTopNInstagram = tk.Label(self.frames_dic['INSTAGRAM_FILTER'], text = 'From - To:')
        self.label_comboTopNInstagram.pack(side="top",expand=False, fill='x', anchor = 'n')

        self.comboTopNInstagram = ttk.Combobox(self.frames_dic['INSTAGRAM_FILTER'],state="readonly", values="", width=10)
        self.comboTopNInstagram.current()
        self.comboTopNInstagram.size()
        self.comboTopNInstagram.pack(side="top",expand=False, fill='none', anchor = 'nw') 


        self.label_comboHeatInstagram = tk.Label(self.frames_dic['INSTAGRAM_FILTER'], text = 'Years:')
        self.label_comboHeatInstagram.pack(side="top",fill='x',expand=False)

        self.comboHeatInstagram = ttk.Combobox(self.frames_dic['INSTAGRAM_FILTER'],state="readonly", values=sorted(list(set(dfAnoMaxOrigInstagram['id']))), width=10)
        self.comboHeatInstagram.current(len(list(set(dfAnoMaxOrigInstagram['id'])))-1)
        self.comboHeatInstagram.size()
        self.comboHeatInstagram.pack(side="top",fill='both',expand=False) 
        self.dicGraphicsFilters['INSTAGRAM_HEAT'] = self.comboHeatTwitter.get()

        #FIM Frames Combo Instagram - INSTAGRAM_FILTER Child

        #Frames Radio Youtube - YOUTUBE_RADIO Child


        self.frames_dic['YOUTUBE_RADIO'] = tk.LabelFrame(self.frames_dic['YOUTUBE_BAR'], text = 'Options')#, bg = 'gray')
        self.frames_dic['YOUTUBE_RADIO'].pack(side="left",expand=False,fill='both')

        self.radioYouTube = tk.Radiobutton(self.frames_dic['YOUTUBE_RADIO'], text="Likes", variable= self.dicGraphicsFilters['YOUTUBE_BAR'], value='YOUTUBE_BAR_LIKE',command = lambda : self.on_press_radio())  
        self.radioYouTube.pack(side="top", anchor = 'w')
        self.radioYouTube = tk.Radiobutton(self.frames_dic['YOUTUBE_RADIO'], text="Disikes", variable= self.dicGraphicsFilters['YOUTUBE_BAR'], value='YOUTUBE_BAR_DISLIKE',command = lambda : self.on_press_radio()) 
        self.radioYouTube.pack(side="top", anchor = 'w')
        self.radioYouTube = tk.Radiobutton(self.frames_dic['YOUTUBE_RADIO'], text="Views", variable= self.dicGraphicsFilters['YOUTUBE_BAR'], value='YOUTUBE_BAR_VIEW',command = lambda : self.on_press_radio()) 
        self.radioYouTube.pack(side="top", anchor = 'w')
        self.radioYouTube = tk.Radiobutton(self.frames_dic['YOUTUBE_RADIO'], text="Comments", variable= self.dicGraphicsFilters['YOUTUBE_BAR'], value='YOUTUBE_BAR_COMMENT',command = lambda : self.on_press_radio()) 
        self.radioYouTube.pack(side="top", anchor = 'w')             

        self.dicGraphicsFilters['YOUTUBE_BAR'].set('YOUTUBE_BAR_LIKE')

        #FIM Frames Radio Youtube - YOUTUBE_RADIO Child

        #Frames Combo Youtube - YOUTUBE_FILTER Child

        self.label_comboTopNYoutube = tk.Label(self.frames_dic['YOUTUBE_FILTER'], text = 'From - To:')
        self.label_comboTopNYoutube.pack(side="top",expand=False, fill='x', anchor = 'n')

        self.comboTopNYouTube = ttk.Combobox(self.frames_dic['YOUTUBE_FILTER'],state="readonly", values="", width=10)
        self.comboTopNYouTube.current()
        self.comboTopNYouTube.size()
        self.comboTopNYouTube.pack(side="top",expand=False, fill='none', anchor = 'nw')

        self.label_comboHeatYouTube = tk.Label(self.frames_dic['YOUTUBE_FILTER'], text = 'Years:       ')
        self.label_comboHeatYouTube.pack(side="top",fill='none',expand=False)

        self.comboHeatYouTube = ttk.Combobox(self.frames_dic['YOUTUBE_FILTER'],state="readonly", values=sorted(list(set(dfAnoMaxOrigYouTube['id']))), width=10)
        self.comboHeatYouTube.current(len(list(set(dfAnoMaxOrigYouTube['id'])))-1)
        self.comboHeatYouTube.size()
        self.comboHeatYouTube.pack(side="top",fill='none',expand=False)
        self.dicGraphicsFilters['YOUTUBE_HEAT'] = self.comboHeatYouTube.get()


        #Frames Combo Youtube - YOUTUBE_FILTER Child

        self.dicCombosHeatGraph = {'TWITTER_HEAT'   : [self.comboHeatTwitter,self.label_comboHeatTwitter], 
                                   'INSTAGRAM_HEAT' : [self.comboHeatInstagram,self.label_comboHeatInstagram],
                                   'YOUTUBE_HEAT'   : [self.comboHeatYouTube,self.label_comboHeatYouTube]}

        self.dicCombosTopNGraph = {'TWITTER_HEAT'   : [self.comboTopNTwitter], 
                                   'INSTAGRAM_HEAT' : [self.comboTopNInstagram],
                                   'YOUTUBE_HEAT'   : [self.comboTopNYouTube]}


        self.listCombosGerais = [self.comboGeralYear, self.comboGeralMonth, self.comboGeralDay, 
                                   self.comboGeralHour]

        #plt.rcParams['font.size'] = 10
        plt.rcParams['axes.titlesize'] = 11

        #plt.rc('figure', titlesize='x-small')
        for k,v in self.frames_dic.items():
            if  ('BAR' in k) or ('HEAT' in k) or ('PIE' in k) or ('LINE' in k):                    
                self.figure = plt.Figure(figsize=(1,1), dpi=self.dpi)
                self.figure.patch.set_facecolor('#f0f0f0')
                self.ax[k] = self.figure.add_subplot(111)

                self.canvas[k] = FigureCanvasTkAgg(self.figure, master=v )#,bg = 'orange')
                self.canvas[k].get_tk_widget().pack(side="left",fill='both',expand=True)

                self.dicCoords[k] = {}
                self.dicCoordsV[k] = {}
                self.dicDF[k] = pd.DataFrame()

        self.listdicGraphicsFilters.append(self.dicToList(self.dicGraphicsFilters))        

        self.frames_dic['GERAL'].pack(side="top",expand=True,fill='both', pady=2)
        self.frames_dic['GERAL_PIE'].pack(side="left",expand=True,fill='both')
        self.frames_dic['GERAL_BAR'].pack(side="left",expand=True,fill='both')

        self.frames_dic['TWITTER'].pack(side="top",expand=True,fill='both', pady=2)
        self.frames_dic['INSTAGRAM'].pack(side="top",expand=True,fill='both', pady=2)
        self.frames_dic['YOUTUBE'].pack(side="top",expand=True,fill='both', pady=2)

        self.frames_dic['TWITTER_BAR'].pack(side="left",expand=True,fill='both')
        self.frames_dic['TWITTER_HEAT'].pack(side="left",expand=True,fill='both')
        self.frames_dic['TWITTER_FILTER'].pack(side="top",expand=True,fill='both')

        self.frames_dic['INSTAGRAM_BAR'].pack(side="left",expand=True,fill='both')
        self.frames_dic['INSTAGRAM_HEAT'].pack(side="left",expand=True,fill='both')
        self.frames_dic['INSTAGRAM_FILTER'].pack(side="top",expand=True,fill='both') 

        self.frames_dic['YOUTUBE_BAR'].pack(side="left",expand=True,fill='both')
        self.frames_dic['YOUTUBE_HEAT'].pack(side="left",expand=True,fill='both')                            
        self.frames_dic['YOUTUBE_FILTER'].pack(side="top",expand=True,fill='both')
            
    def Connect(self):
        
        self.master.bind("<Button-3>", self.on_press_back) 
        
        self.canvas['GERAL_PIE'].mpl_connect('pick_event', self.on_pick_GERAL_PIE)
        #self.canvas['GERAL_PIE'].mpl_connect('button_press_event', self.on_press)
        
        self.canvas['GERAL_BAR'].mpl_connect('pick_event', self.on_pick_GERAL_BAR)
        #self.canvas['GERAL_BAR'].mpl_connect('button_press_event', self.on_press)        
        #self.canvas['GERAL_BAR'].mpl_connect('motion_notify_event', self.on_mouse_move_GERAL_BAR)
        
        a = self.canvas['TWITTER_BAR'].mpl_connect('pick_event', self.on_pick_TWITTER_BAR)
        #self.canvas['TWITTER_BAR'].mpl_connect('button_press_event', self.on_press)
        self.canvas['TWITTER_BAR'].mpl_connect('motion_notify_event', self.on_mouse_move_TWITTER_BAR)
        self.canvas['TWITTER_HEAT'].mpl_connect('motion_notify_event', self.on_mouse_move_TWITTER_HEAT)        
        self.canvas['TWITTER_HEAT'].mpl_connect('button_press_event', self.on_click_TWITTER_HEAT)
        
        self.canvas['INSTAGRAM_BAR'].mpl_connect('pick_event', self.on_pick_INSTAGRAM_BAR)
        self.canvas['INSTAGRAM_BAR'].mpl_connect('motion_notify_event', self.on_mouse_move_INSTAGRAM_BAR)
        self.canvas['INSTAGRAM_HEAT'].mpl_connect('motion_notify_event', self.on_mouse_move_INSTAGRAM_HEAT)
        self.canvas['INSTAGRAM_HEAT'].mpl_connect('button_press_event', self.on_click_INSTAGRAM_HEAT)
        

        self.canvas['YOUTUBE_BAR'].mpl_connect('pick_event', self.on_pick_YOUTUBE_BAR)
        self.canvas['YOUTUBE_BAR'].mpl_connect('motion_notify_event', self.on_mouse_move_YOUTUBE_BAR)
        self.canvas['YOUTUBE_HEAT'].mpl_connect('motion_notify_event', self.on_mouse_move_YOUTUBE_HEAT)
        self.canvas['YOUTUBE_HEAT'].mpl_connect('button_press_event', self.on_click_YOUTUBE_HEAT)        
#        self.canvas['YOUTUBE_BAR'].mpl_connect('motion_notify_event',)
#         self.canvas['YOUTUBE_BAR'].mpl_connect('button_press_event', self.on_press)
        #COMBOS
        self.comboQtdYears.bind     ('<<ComboboxSelected>>', self.comboQtdYears_select)
        self.comboGeralYear.bind    ('<<ComboboxSelected>>', self.comboGeralYear_select)
        self.comboGeralMonth.bind   ('<<ComboboxSelected>>', self.comboGeralMonth_select)
        self.comboGeralDay.bind     ('<<ComboboxSelected>>', self.comboGeralDay_select)
        self.comboGeralHour.bind    ('<<ComboboxSelected>>', self.comboGeralHour_select)
        self.comboSocialMedia.bind  ('<<ComboboxSelected>>', self.comboSocialMedia_select)
        
        self.comboHeatTwitter.bind  ('<<ComboboxSelected>>', self.comboHeatTwitter_select)
        self.comboTopNTwitter.bind  ('<<ComboboxSelected>>', self.comboTopNTwitter_select)
        
        self.comboHeatInstagram.bind('<<ComboboxSelected>>', self.comboHeatInstagram_select)
        self.comboTopNInstagram.bind('<<ComboboxSelected>>', self.comboTopNInstagram_select)
        
        self.comboHeatYouTube.bind  ('<<ComboboxSelected>>', self.comboHeatYouTube_select)
        self.comboTopNYouTube.bind  ('<<ComboboxSelected>>', self.comboTopNYouTube_select)
        
        #self.comboGraphValueOptTwitter.bind('<<ComboboxSelected>>', self.comboGraphValueOptTwitter_select)
        
    def dicCoordsChange(self, dicCoordName,dic,orient = 'h'):        
        if orient == 'h':
            self.dicCoords[dicCoordName] = dic
        else:
            self.dicCoordsV[dicCoordName] = dic
                

        
    def filterSocialMedia(self,df):   
#        print('filterSocialMedia')
        if self.dicGraphicsFilters['SOCIALMEDIA'] != '':
            df = df.loc[df['Social Media'] == self.dicGraphicsFilters['SOCIALMEDIA']]
        return df
    
    def filterDateSet(self, filterDate):       
        print('filterDateSET ini')
        print(filterDate)
        if len(filterDate) == 0: #Ano
            self.dicGraphicsFilters['DATATIPO'] = 'ANO'            
        elif len(filterDate) == 4: #mes
            self.dicGraphicsFilters['DATATIPO'] = 'MES'
        elif len(filterDate) == 7: #dia
            self.dicGraphicsFilters['DATATIPO'] = 'DIA'
        elif len(filterDate) == 10: #hora            
            self.dicGraphicsFilters['DATATIPO'] = 'HORA'
        elif len(filterDate) == 13: #mim
            self.dicGraphicsFilters['DATATIPO'] = 'MIN'
        self.dicGraphicsFilters['DATAVAL'] = filterDate
        print('filterDateSet FIM')
            
    def filterDate(self):        
#        print('filterDate INI')
        if self.dicGraphicsFilters['DATATIPO'] == 'ANO': #Ano
            self.dicDF['GERAL_BAR']     = dfAnoCountOrig
            self.dicDF['TWITTER_BAR']   = dfAnoMaxOrigTwitter
            self.dicDF['INSTAGRAM_BAR'] = dfAnoMaxOrigInstagram
            self.dicDF['YOUTUBE_BAR']   = dfAnoMaxOrigYouTube            
            
        elif self.dicGraphicsFilters['DATATIPO'] == 'MES':
            self.dicDF['GERAL_BAR']     = dfMesCountOrig.loc[(dfMesCountOrig['id']>=self.dicGraphicsFilters['DATAVAL'] +'-01') & 
                                                             (dfMesCountOrig['id']<=self.dicGraphicsFilters['DATAVAL'] +"-12")]
            self.dicDF['TWITTER_BAR']   = dfMesMaxOrigTwitter.loc[(dfMesMaxOrigTwitter['id']>=self.dicGraphicsFilters['DATAVAL'] +'-01') &
                                                                  (dfMesMaxOrigTwitter['id']<=self.dicGraphicsFilters['DATAVAL'] +"-12")]
            self.dicDF['INSTAGRAM_BAR'] = dfMesMaxOrigInstagram.loc[(dfMesMaxOrigInstagram['id']>= self.dicGraphicsFilters['DATAVAL'] +'-01') & 
                                                                    (dfMesMaxOrigInstagram['id']<=self.dicGraphicsFilters['DATAVAL'] +"-12")]
            self.dicDF['YOUTUBE_BAR']   = dfMesMaxOrigYouTube.loc[(dfMesMaxOrigYouTube['id']>= self.dicGraphicsFilters['DATAVAL'] +'-01') &
                                                                  (dfMesMaxOrigYouTube['id']<=self.dicGraphicsFilters['DATAVAL'] +"-12")]                        
            
        elif self.dicGraphicsFilters['DATATIPO'] == 'DIA': #dia
            self.dicDF['GERAL_BAR']     = dfDiaCountOrig.loc[(dfDiaCountOrig['id']>=self.dicGraphicsFilters['DATAVAL'] + '-01') & 
                                                             (dfDiaCountOrig['id']<=self.dicGraphicsFilters['DATAVAL'] + "-31")]            
            self.dicDF['TWITTER_BAR']   = dfDiaMaxOrigTwitter.loc[(dfDiaMaxOrigTwitter['id']>=self.dicGraphicsFilters['DATAVAL'] + '-01') & 
                                                                  (dfDiaMaxOrigTwitter['id']<=self.dicGraphicsFilters['DATAVAL'] + "-31")]
            self.dicDF['INSTAGRAM_BAR'] = dfDiaMaxOrigInstagram.loc[(dfDiaMaxOrigInstagram['id']>=self.dicGraphicsFilters['DATAVAL'] + '-01') & 
                                                                    (dfDiaMaxOrigInstagram['id']<=self.dicGraphicsFilters['DATAVAL'] + "-31")]            
            self.dicDF['YOUTUBE_BAR']   = dfDiaMaxOrigYouTube.loc[(dfDiaMaxOrigYouTube['id']>=self.dicGraphicsFilters['DATAVAL'] + '-01') & 
                                                                  (dfDiaMaxOrigYouTube['id']<=self.dicGraphicsFilters['DATAVAL'] + "-31")]            

        elif self.dicGraphicsFilters['DATATIPO'] == 'HORA': #hora
            self.dicDF['GERAL_BAR']     = dfHoraCountOrig.loc[(dfHoraCountOrig['id']>=self.dicGraphicsFilters['DATAVAL'] + ' 01') &  
                                                              (dfHoraCountOrig['id']<=self.dicGraphicsFilters['DATAVAL'] + " 23")]                         
            self.dicDF['TWITTER_BAR']   = dfHoraMaxOrigTwitter.loc[(dfHoraMaxOrigTwitter['id']>=self.dicGraphicsFilters['DATAVAL'] + ' 01') &
                                                                   (dfHoraMaxOrigTwitter['id']<=self.dicGraphicsFilters['DATAVAL'] + " 23")]
            self.dicDF['INSTAGRAM_BAR'] = dfHoraMaxOrigInstagram.loc[(dfHoraMaxOrigInstagram['id']>=self.dicGraphicsFilters['DATAVAL'] + ' 01') & 
                                                                     (dfHoraMaxOrigInstagram['id']<=self.dicGraphicsFilters['DATAVAL'] + " 23")]            
            self.dicDF['YOUTUBE_BAR']   = dfHoraMaxOrigYouTube.loc[(dfHoraMaxOrigYouTube['id']>=self.dicGraphicsFilters['DATAVAL'] + ' 01') & 
                                                                   (dfHoraMaxOrigYouTube['id']<=self.dicGraphicsFilters['DATAVAL'] + " 23")]            

        elif self.dicGraphicsFilters['DATATIPO'] == 'MIN': #mim
            self.dicDF['GERAL_BAR']     = dfMinCountOrig.loc[(dfMinCountOrig['id']>=self.dicGraphicsFilters['DATAVAL'] + ':00:00') & 
                                                             (dfMinCountOrig['id']<=self.dicGraphicsFilters['DATAVAL'] + ':59:59')]
            self.dicDF['TWITTER_BAR']   = dfMinMaxOrigTwitter.loc[(dfMinMaxOrigTwitter['id']>=self.dicGraphicsFilters['DATAVAL'] + ':00:00') & 
                                                                  (dfMinMaxOrigTwitter['id']<=self.dicGraphicsFilters['DATAVAL'] + ':59:59')]
            self.dicDF['INSTAGRAM_BAR'] = dfMinMaxOrigInstagram.loc[(dfMinMaxOrigInstagram['id']>=self.dicGraphicsFilters['DATAVAL'] + ':00:00') & 
                                                                    (dfMinMaxOrigInstagram['id']<=self.dicGraphicsFilters['DATAVAL'] + ':59:59')]            
            self.dicDF['YOUTUBE_BAR']   = dfMinMaxOrigYouTube.loc[(dfMinMaxOrigYouTube['id']>=self.dicGraphicsFilters['DATAVAL'] + ':00:00') & 
                                                                  (dfMinMaxOrigYouTube['id']<=self.dicGraphicsFilters['DATAVAL'] + ':59:59')]            
        if self.dicGraphicsFilters['DATATIPO'] != '':           
            self.dicDF['GERAL_BAR']     = self.dicDF['GERAL_BAR'].reset_index()
            self.dicDF['TWITTER_BAR']   = self.dicDF['TWITTER_BAR'].reset_index()
            self.dicDF['INSTAGRAM_BAR'] = self.dicDF['INSTAGRAM_BAR'].reset_index()
            self.dicDF['YOUTUBE_BAR']   = self.dicDF['YOUTUBE_BAR'].reset_index()
            self.dicDF['TWITTER_LINE']   = self.dicDF['TWITTER_BAR']
            self.dicDF['INSTAGRAM_LINE'] = self.dicDF['INSTAGRAM_BAR']
            self.dicDF['YOUTUBE_LINE']   = self.dicDF['YOUTUBE_BAR']
            self.dicDF['GERAL_PIE']      = self.dicDF['GERAL_BAR']
            self.dicDF['TWITTER_HEAT']   = self.dicDF['TWITTER_BAR']
            self.dicDF['INSTAGRAM_HEAT'] = self.dicDF['INSTAGRAM_BAR']
            self.dicDF['YOUTUBE_HEAT']   = self.dicDF['YOUTUBE_BAR']
            self.dicDF['GERAL_PIE']      = self.dicDF['GERAL_BAR']
    #print('filterDate FIM')
            
    def filterAmountYears(self):
        #print('filterAmountYears INI')
        if self.dicGraphicsFilters['AMOUNTYEARS'] != 'ALL':
            maxYear = self.dicGraphicsFilters['YEARSGRAPH'][0]
            minYear = self.dicGraphicsFilters['YEARSGRAPH'][int(self.dicGraphicsFilters['AMOUNTYEARS'])-1]
            for k,df in self.dicDF.items():                
                if self.dicGraphicsFilters['DATATIPO'] == 'ANO':
                    self.dicDF[k] = df.loc[(df['id']<=maxYear) & ( df['id']>=minYear)]  
                else:
                    self.dicDF[k] = df.loc[(df['id'].str[:4]<=str(maxYear)) & ( df['id'].str[:4]>=str(minYear))]
        #print('filterAmountYears FIM')
        
    def filterDatePlotHeat(self, comboName, dfName):
        #print('filterDatePlotHeat INI')
        if self.dicGraphicsFilters['DATATIPO'] == 'ANO':
            self.dicDF[dfName] = self.dicDF[dfName].loc[self.dicDF[dfName]['id']==int(self.dicGraphicsFilters[dfName])]                
        elif self.dicGraphicsFilters['DATATIPO'] == 'MES':
            heat_filter = self.dicGraphicsFilters['DATAVAL'] + "-" + mesNome[self.dicGraphicsFilters[dfName]]
            self.dicDF[dfName] = self.dicDF[dfName].loc[self.dicDF[dfName]['id']==heat_filter]            
        elif self.dicGraphicsFilters['DATATIPO'] == 'DIA':
            heat_filter = self.dicGraphicsFilters['DATAVAL'] + "-" + self.dicGraphicsFilters[dfName]
            self.dicDF[dfName] = self.dicDF[dfName].loc[self.dicDF[dfName]['id']==heat_filter]
        elif self.dicGraphicsFilters['DATATIPO'] == 'HORA':
            heat_filter =self.dicGraphicsFilters['DATAVAL'] + " " + self.dicGraphicsFilters[dfName][:2]
            self.dicDF[dfName] = self.dicDF[dfName].loc[self.dicDF[dfName]['id']==heat_filter]
        elif self.dicGraphicsFilters['DATATIPO'] == 'MIN':
            heat_filter = self.dicGraphicsFilters['DATAVAL'] + ":" + self.dicGraphicsFilters[dfName][-2:] +':00'
            self.dicDF[dfName] = self.dicDF[dfName].loc[self.dicDF[dfName]['id']==heat_filter]
        #print('filterDatePlotHeat FIM')
            
    def update_comboHeat_values(self,comboName,labelName,dfName): 
        #print('update_comboHeat_values INI')
        if self.dicGraphicsFilters['UPDATECOMBOS']:
            if self.dicGraphicsFilters['DATATIPO'] == 'ANO':
                comboName['values']=sorted(list(set(self.dicDF[dfName]['id'])))
                comboName.size ()
                comboName.current(len(list(set(self.dicDF[dfName]['id'])))-1)
                labelName['text'] = "Years:"
            else:
                listElement = []
                for l in sorted(list(set(self.dicDF[dfName]['id']))): 
                    if self.dicGraphicsFilters['DATATIPO'] == 'MES':
                        listElement.append(mes[l[-2:]])
                        labelName['text'] = "Months:"
                    elif self.dicGraphicsFilters['DATATIPO'] == 'DIA':
                        listElement.append(l[-2:])                
                        labelName['text'] = "Days:"
                    elif self.dicGraphicsFilters['DATATIPO'] == 'HORA':
                        listElement.append(l[-2:]+':00')                    
                        labelName['text'] = "Hours:"
                    elif self.dicGraphicsFilters['DATATIPO'] == 'MIN':                        
                        listElement.append(l[11:16])
                        labelName['text'] = "Minutes:"
                comboName['values']=listElement                
                if listElement != []:
                    comboName.current(0)                    
                else:
                    comboName.current()
            self.dicGraphicsFilters[dfName] = comboName.get()
        #print('update_comboHeat_values FIM')
                    
    def update_comboGerais_values(self):
        #print('update_comboGerais_values INI')
        if self.dicGraphicsFilters['DATATIPO'] == 'ANO':
            self.listCombosGerais[0]['values'] = [''] + sorted(list(set(self.dicDF['GERAL_BAR']['id'])))   
            self.listCombosGerais[0].current(0)
            self.listCombosGerais[1]['values'] = ['']
            self.listCombosGerais[1].current(0)
            self.listCombosGerais[2]['values'] = ['']
            self.listCombosGerais[2].current(0)
            self.listCombosGerais[3]['values'] = ['']
            self.listCombosGerais[3].current(0)
        elif self.dicGraphicsFilters['DATATIPO'] == 'MES':
            self.update_comboGerais_values_indiv()
            self.listCombosGerais[1].current(0)
            self.listCombosGerais[2]['values'] = ['']
            self.listCombosGerais[2].current(0)
            self.listCombosGerais[3]['values'] = ['']
            self.listCombosGerais[3].current(0)
        elif self.dicGraphicsFilters['DATATIPO'] == 'DIA':
            self.update_comboGerais_values_indiv()
            self.listCombosGerais[2].current(0)
            self.listCombosGerais[3]['values'] = ['']
            self.listCombosGerais[3].current(0)
        elif self.dicGraphicsFilters['DATATIPO'] == 'HORA':
            self.update_comboGerais_values_indiv()
            self.listCombosGerais[3].current(0)
        #print('update_comboGerais_values FIM')
        
    def update_comboGerais_socialMedia_qtd(self):
#   
        self.comboSocialMedia.current(self.comboSocialMedia['values'].index(self.dicGraphicsFilters['SOCIALMEDIA']))
        self.comboQtdYears.current(self.comboQtdYears['values'].index(self.dicGraphicsFilters['AMOUNTYEARS']))
#  
    
    def update_comboGerais_values_indiv(self):
        listElement = []
        for l in sorted(list(set(self.dicDF['GERAL_BAR']['id']))):
            if self.dicGraphicsFilters['DATATIPO'] == 'MES':
                listElement.append(mes[l[-2:]])
                self.listCombosGerais[1]['values'] = [''] + listElement                   
            elif self.dicGraphicsFilters['DATATIPO'] == 'DIA':
                listElement.append(l[-2:])                
                self.listCombosGerais[2]['values'] = [''] + listElement
            elif self.dicGraphicsFilters['DATATIPO'] == 'HORA':
                listElement.append(l[-2:]+':00')                    
                self.listCombosGerais[3]['values'] = [''] + listElement
                
    def update_comboTopN_values_indiv(self,comboName,dfName):         
        #print('update_comboTopN_values_indiv')
        if self.dicGraphicsFromTo['UPDATECOMBOS']:
            tam =  len(self.dicDF[dfName])
            size =  tam // 10
            listElement = []
            if tam >= 10:
                rest = tam % 10
                listElement = []
                linhas = 0
                for i in range(0,size):     
                    listElement.append("%s-%s" % (linhas+1,linhas+10))
                    linhas +=10
                if rest !=0:
                    listElement.append("%s-%s" % (linhas+1,linhas+rest))
            else:
                listElement.append("%s-%s" % (1,len(self.dicDF[dfName])))
            comboName['values']=listElement 
            comboName.current(0)
        
    def update_comboTopN_values(self):  
        for k,widgets in self.dicCombosTopNGraph.items():
            if not self.dicDF[k].empty:
                self.update_comboTopN_values_indiv(widgets[0],k)
                
    def GraphicPrepare(self): #PREPARE
        
        #print('GraficPrepare INI')        
        self.filterDate()
        for ax in self.ax:            
            self.ax[ax].cla()
        self.filterAmountYears()
        self.dicDF['GERAL_PIE'] = self.filterSocialMedia(self.dicDF['GERAL_PIE'])
        self.dicDF['GERAL_BAR'] = self.filterSocialMedia(self.dicDF['GERAL_BAR'])

        if self.dicGraphicsFilters['SOCIALMEDIA'] == 'Twitter':
            self.dicDF['INSTAGRAM_BAR'] = pd.DataFrame()
            self.dicDF['INSTAGRAM_HEAT'] = pd.DataFrame()
            self.dicDF['YOUTUBE_BAR'] = pd.DataFrame()
            self.dicDF['YOUTUBE_HEAT'] = pd.DataFrame()
        elif self.dicGraphicsFilters['SOCIALMEDIA'] == 'Instagram':
            self.dicDF['TWITTER_BAR'] = pd.DataFrame()
            self.dicDF['TWITTER_HEAT'] = pd.DataFrame()
            self.dicDF['YOUTUBE_BAR'] = pd.DataFrame()
            self.dicDF['YOUTUBE_HEAT'] = pd.DataFrame()              
        elif self.dicGraphicsFilters['SOCIALMEDIA'] == 'YouTube':
            self.dicDF['INSTAGRAM_BAR'] = pd.DataFrame()
            self.dicDF['INSTAGRAM_HEAT'] = pd.DataFrame()
            self.dicDF['TWITTER_BAR'] = pd.DataFrame()
            self.dicDF['TWITTER_HEAT'] = pd.DataFrame()
        else:
            self.dicDF['GERAL_BAR'] = fillDataFrameCount(self.dicDF['GERAL_BAR'])
        #self.comboGraphValueSet()
        self.loopComboValuesUpdate()        
        self.update_comboGerais_values()
        self.update_comboGerais_socialMedia_qtd()        
        self.update_comboTopN_values()
        self.dicGraphicsFilters['UPDATECOMBOS'] = True
        self.dicGraphicsFromTo['UPDATECOMBOS'] = True
                    
        #print('GraficPrepare FIM')
        
    def loopComboValuesUpdate(self):
        for k,widgets in self.dicCombosHeatGraph.items():
            if not self.dicDF[k].empty:
                self.update_comboHeat_values(widgets[0],widgets[1],k)
                self.filterDatePlotHeat(widgets[0],k)        
        
    def Graphics(self):
        #print('Graphics INI')
        self.draw_piePlot_all()
        self.draw_linePlot_all()
        self.draw_barPlot_all()
        self.draw_heatPlot_all()
        #print('Graphics FIM')
        
    def draw_barPlot_all(self):
        #print('draw_barPlot_all ini')        
        for k,v in self.frames_dic.items():         
            if ('BAR' in k):
                if 'GERAL' in k:
                    graphName = self.dicGraphicsFilters['GERAL_BAR']
                elif 'TWITTER' in k:                    
                    graphName = self.dicGraphicsFilters['TWITTER_BAR'].get()
                    self.dicDF_StatsSent[k] = self.dicDF[k]
                    self.dicDF[k] = SumTotals(self.dicDF[k])#, graphName)                       
                    
                elif 'INSTAGRAM' in k:
                    graphName = self.dicGraphicsFilters['INSTAGRAM_BAR'].get()
                    self.dicDF_StatsSent[k] = self.dicDF[k]
                    self.dicDF[k] = SumTotals(self.dicDF[k])#, graphName)
                                  
                elif 'YOUTUBE' in k:                
                    graphName = self.dicGraphicsFilters['YOUTUBE_BAR'].get()
                    self.dicDF_StatsSent[k] = self.dicDF[k]
                    self.dicDF[k] = SumTotals(self.dicDF[k])#, graphName)
                #talves o problema seja o canvas
                #draw_barPlot(self.dicDF[k],graphName, self.canvas[k],self.dicGraphicsFilters['DATATIPO'],my_gui)
                draw_barPlot(self.dicDF[k],graphName,self.ax[k], self.canvas[k],self.dicGraphicsFilters['DATATIPO'],my_gui)

        
        #print('draw_barPlot_all fim')        
                
    def draw_linePlot_all(self):
        for k,v in self.frames_dic.items():
            if ('LINE' in k):
#                 if 'GERAL' in k:
#                     graphName = self.dicGraphicsFilters['GERAL_BAR']             
                if 'TWITTER' in k:
                    graphName = 'TWITTER_LINE'
                    self.dicDF_StatsSent[k] = self.dicDF[k]
                    self.dicDF[k] = SumTotals(self.dicDF[k])#, graphName)                   
                    #self.dicDF[k] = TopFilter(self.dicDF[k], graphName)              
                elif 'INSTAGRAM' in k:
                    graphName = 'INSTAGRAM_LINE'
                    self.dicDF_StatsSent[k] = self.dicDF[k]
                    self.dicDF[k] = SumTotals(self.dicDF[k])#, graphName)
                    #self.dicDF[k] = TopFilter(self.dicDF[k], graphName)              
                elif 'YOUTUBE' in k:
                    graphName = 'YOUTUBE_LINE'
                    self.dicDF_StatsSent[k] = self.dicDF[k]
                    self.dicDF[k] = SumTotals(self.dicDF[k])#, graphName)
                    #self.dicDF[k] = TopFilter(self.dicDF[k], graphName)     
                draw_linePlot(self.dicDF[k],graphName,self.ax[k], self.canvas[k],self.dicGraphicsFilters['DATATIPO'])
    
    
    def draw_piePlot_all(self):
        draw_piePlot(self.dicDF['GERAL_PIE'],'GERAL_PIE',self.ax['GERAL_PIE'], self.canvas['GERAL_PIE'], my_gui)
    
    def draw_heatPlot_all(self):
        for k,v in self.frames_dic.items():
            if ('HEAT' in k):
                if 'TWITTER' in k:
                    graphName = 'TWITTER_HEAT'                    
                    #self.dicDF[k] = TopFilter(self.dicDF[k], graphName,10)  
                    radioOption = "TWITTER_BAR"
                elif 'INSTAGRAM' in k:
                    graphName = 'INSTAGRAM_HEAT'
                    #self.dicDF[k] = TopFilter(self.dicDF[k], graphName,10)                            
                    radioOption = "INSTAGRAM_BAR"
                elif 'YOUTUBE' in k:
                    graphName = 'YOUTUBE_HEAT'
                    #self.dicDF[k] = TopFilter(self.dicDF[k], graphName,10)              
                    radioOption = "YOUTUBE_BAR"
                self.dicDF[k] = TopFilter(self.dicDF[k], graphName,self.dicGraphicsFromTo[graphName][0],self.dicGraphicsFromTo[graphName][1],self.dicGraphicsFilters[radioOption].get())
                self.dicHeatCels[graphName] = draw_heatPlot(self.dicDF[k],graphName,self.ax[k], self.canvas[k], self.dicGraphicsFilters[radioOption].get(), my_gui)
        
    def on_press_radio(self):
        self.dicGraphicsFilters['UPDATECOMBOS'] = False
        self.on_press('radio')
        
    def on_press(self, event):
        print('self.on_press_ini')
        if event == 'combo' or event == 'radio' or event == 'comboheat' or event == 'pick': # or event == 'combographvalue':
            navigationFoward = True
            addlist = True        
        elif event == 'home' or event == 'combotop':
            navigationFoward = True
            addlist = False
        elif event == 'back' or event == 'pickback':
            navigationFoward = False 
        if navigationFoward:
            #print('on_press list size')  
            if addlist: 
                self.listdicGraphicsFilters.append(self.dicToList(self.dicGraphicsFilters))
            self.GraphicPrepare()
            self.Graphics()
        else:
            #print('on_press list size 3')
            if len(self.listdicGraphicsFilters) > 1:
                self.listdicGraphicsFilters.pop()
                self.dicGraphicsFiltersListToDic(self.listdicGraphicsFilters[-1])                
                self.GraphicPrepare()
                self.Graphics()
        print('self.on_press_fim')

            
    def on_press_back(self, event):
        #print('on_press_back')
        if self.pick_execute != 'pickback':
            self.on_press('back')
        self.pick_execute = 'back'        
        
    def on_mouse_move(self, posx, posy, graphName, field, limpar = False):
        self.ax[graphName].texts = []
        value = self.dicCoords[graphName][float(posx)]
        if not limpar:
            if value !=-1:
                v1,v2 = self.dicCoordsV[graphName].GetItemCoords(value)
                if posy <= v2:
                    ind = self.dicDF[graphName].loc[(self.dicDF[graphName]['id']==value)][field].index        
                    text = ''
                    if ind.size > 0:            
                        ind = int(ind[0])
                        text = self.dicDF[graphName].loc[ind][field]
                        text = '{:,.0f}'.format(int(text)).replace(".","%").replace(",",".").replace("%",",")
                    self.ax[graphName].annotate(text, xy=(posx, posy), xytext=(posx, posy),  # 3 points vertical offset
                                textcoords="data",
                                bbox=dict(boxstyle="round", facecolor='yellow', alpha=0.8, edgecolor='black'),
                                ha='center', va='bottom', wrap=True)
        self.canvas[graphName].draw_idle()
        
    def on_mouse_move_heat(self, posx, posy, graphName,limpar = False):
        #print(self.dicDF[graphName])
        value = self.dicCoords[graphName][float(posy)]
        lenFixText = self.dicHeatCels[graphName]
        if len(self.ax[graphName].texts) > lenFixText:
                self.ax[graphName].texts.pop()
        if not limpar:
            if value !=-1:
                text = self.dicDF[graphName].iloc[value]['Text']
                self.ax[graphName].annotate(text, xy=(posx, posy), xytext=(posx, posy),  # 3 points vertical offset
                                        textcoords="data",
                                        bbox=dict(boxstyle="round", facecolor='yellow', alpha=0.8, edgecolor='black'),
                                        ha='center', va='bottom', wrap=True)#, fontproperties=fprop)

        self.canvas[graphName].draw_idle()
        
    def on_click_heat(self, posx, posy, graphName):
        #print('onclick')
        value = self.dicCoords[graphName][float(posy)]
        if value !=-1:
            if "TWITTER" in graphName:
                id_url = self.dicDF[graphName].iloc[value]['idtweet']
                url = "https://twitter.com/" + profile + "/status/" + id_url
                webbrowser.open_new(url)
            elif "INSTAGRAM" in graphName:
                id_url = self.dicDF[graphName].iloc[value]['Instagram_ID']
                url = "https://www.instagram.com/p/" + id_url
                webbrowser.open_new(url)
            elif "YOUTUBE" in graphName:
                id_url = self.dicDF[graphName].iloc[value]['Video_id']
                url = "https://www.youtube.com/watch?v=" + id_url
                webbrowser.open_new(url)
        
    def comboQtdYears_select(self, event):     
        #print('comboQtdYears_select')
        self.dicGraphicsFilters['AMOUNTYEARS'] = self.comboQtdYears.get()
        self.dicGraphicsFilters['SOCIALMEDIA'] = ""
        self.filterDateSet("")
        self.pick_execute = 'combo'
        self.on_press('combo')
        
    def comboSocialMedia_select(self, event):
        #print('on_select_comboSocialMedia INI')        
        self.dicGraphicsFilters['SOCIALMEDIA'] = str(self.comboSocialMedia.get())
        self.pick_execute = 'combo'
        self.on_press('combo')
        #print('on_select_comboSocialMedia FIM')

    def comboGeralYear_select(self, event):
        #print('on_select_comboGeralYear INI') 
        self.filterDateSet(self.comboGeralYear.get())
        self.pick_execute = 'combo'
        self.on_press('combo')
        print(self.comboGeralYear['values'])
        #print('on_select_comboGeralYear FIM')
        
    def comboGeralMonth_select(self, event):
        #print('on_select_comboGeralMonth INI')        
        if self.comboGeralMonth.get() == '':
            filterDate = "%s" % (self.comboGeralYear.get())
        else:
            filterDate = "%s-%s" % (self.comboGeralYear.get(),mesNome[self.comboGeralMonth.get()])
        self.filterDateSet(filterDate)
        self.pick_execute = 'combo'
        self.on_press('combo')
        #print('on_select_comboGeralMonth FIM')
    
    def comboGeralDay_select(self, event):
        #print('on_select_comboGeralDay INI')        
        if self.comboGeralDay.get() == '':
            filterDate = "%s-%s" % (self.comboGeralYear.get(),mesNome[self.comboGeralMonth.get()])
        else:
            filterDate = "%s-%s-%s" % (self.comboGeralYear.get(),mesNome[self.comboGeralMonth.get()],self.comboGeralDay.get())
        self.filterDateSet(filterDate)
        self.pick_execute = 'combo'
        self.on_press('combo')
        #print('on_select_comboGeralDay FIM')
            
    def comboGeralHour_select(self, event):
        #print('on_select_comboGeralHour INI')        
        if self.comboGeralHour.get() == '':
            filterDate = "%s-%s-%s" % (self.comboGeralYear.get(),mesNome[self.comboGeralMonth.get()],self.comboGeralDay.get())
        else:
            filterDate = "%s-%s-%s %s" % (self.comboGeralYear.get(),mesNome[self.comboGeralMonth.get()],self.comboGeralDay.get(),self.comboGeralHour.get()[:2])
        self.filterDateSet(filterDate)        
        self.pick_execute = 'combo'
        self.on_press('combo')
        #print('on_select_comboGeralHour FIM')
        
    def adjustComboCurrent(self,value):        
        print('adjustComboCurrent INI')
        print(self.dicGraphicsFilters['DATATIPO'])
        print(value)
        if self.dicGraphicsFilters['DATATIPO'] == 'ANO':
            self.listCombosGerais[0].current(self.listCombosGerais[0]['values'].index(value))
        elif self.dicGraphicsFilters['DATATIPO'] == 'MES':
            value = mes[value[len(value)-2:]]           
            self.listCombosGerais[1].current(self.listCombosGerais[1]['values'].index(value))
        elif self.dicGraphicsFilters['DATATIPO'] == 'DIA':
            value = value[len(value)-2:]            
            self.listCombosGerais[2].current(self.listCombosGerais[2]['values'].index(value))
        elif self.dicGraphicsFilters['DATATIPO'] == 'HORA':
            value = value[len(value)-2:] + ":00"         
            self.listCombosGerais[3].current(self.listCombosGerais[3]['values'].index(value))
        #print('adjustComboCurrent FIM')

    def on_pick_GERAL_PIE(self, event):
        #print('onpick_GERAL_PIE INI')
        if event.mouseevent.button == 1:
            line = event.artist
            self.posX = event.artist.theta1
            self.posY = event.artist.theta2
            self.comboSocialMedia.current(self.comboSocialMedia['values'].index(str(self.dicCoords['GERAL_PIE'][float(round(self.posX,2))])))
            self.dicGraphicsFilters['SOCIALMEDIA'] = str(self.dicCoords['GERAL_PIE'][float(round(self.posX,2))])
            self.on_press('pick')    
        else:
            self.pick_execute = 'pickback'
            self.on_press('pickback')
            
        #print('onpick_GERAL_PIE FIM')

    def on_pick_GERAL_BAR(self, event):                
        print('on_pick_GERAL_BAR INI')
        print(event.artist)
#         if self.exec_pick:
#             self.exec_pick = False
        if self.dicGraphicsFilters['DATATIPO'] != 'MIN':            
            if event.mouseevent.button == 1:
                print('mouseevent')
                line = event.artist
                self.posX = round(line.get_x(),2)
                self.posY = round(line.get_y(),2)
                self.adjustComboCurrent(str(self.dicCoords['GERAL_BAR'][float(self.posX)]))
                self.filterDateSet(str(self.dicCoords['GERAL_BAR'][float(self.posX)])) 
                self.pick_execute = 'pick'
                self.on_press('pick')

            else:
                self.pick_execute = 'pickback'
                self.on_press('pickback')
#         else:
#             self.exec_pick = True
        #print('on_pick_GERAL_BAR FIM')

    def on_pick_TWITTER_BAR(self, event):       
        print('on_pick_TWITTER_BAR')        
        if self.dicGraphicsFilters['DATATIPO'] != 'MIN':
            if event.mouseevent.button == 1:
                line = event.artist
                self.posX = round(line.get_x(),2)
                self.posY = round(line.get_y(),2)
                #print(self.a)                                                      
                #if self.a != self.dicGraphicsFilters['DATATIPO']:
                self.adjustComboCurrent(str(self.dicCoords['TWITTER_BAR'][float(self.posX)]))
                self.filterDateSet(str(self.dicCoords['TWITTER_BAR'][float(self.posX)]))
                self.pick_execute = 'pick'
                self.on_press('pick')
                #self.a = self.filterDateSet(str(self.dicCoords['TWITTER_BAR'][float(self.posX)]))
            else:
                self.pick_execute = 'pickback'
                self.on_press('pickback')
        #print('on_pick_TWITTER_BAR FIM')
        
    def on_pick_INSTAGRAM_BAR(self, event):        
        #print('on_pick_INSTAGRAM_BAR')
        if self.dicGraphicsFilters['DATATIPO'] != 'MIN':
            if event.mouseevent.button == 1:
                line = event.artist
                self.posX = round(line.get_x(),2)
                self.posY = round(line.get_y(),2)
                self.adjustComboCurrent(str(self.dicCoords['INSTAGRAM_BAR'][float(self.posX)]))
                self.filterDateSet(str(self.dicCoords['INSTAGRAM_BAR'][float(self.posX)]))
                self.pick_execute = 'pick'
                self.on_press('pick')
            else:
                self.pick_execute = 'pickback'
                self.on_press('pickback')

    def on_pick_YOUTUBE_BAR(self, event):        
        print('on_pick_YOUTUBE_BAR')
        if self.dicGraphicsFilters['DATATIPO'] != 'MIN':
            if event.mouseevent.button == 1:
                line = event.artist
                self.posX = round(line.get_x(),2)
                self.posY = round(line.get_y(),2)
                self.adjustComboCurrent(str(self.dicCoords['YOUTUBE_BAR'][float(self.posX)]))
                self.filterDateSet(str(self.dicCoords['YOUTUBE_BAR'][float(self.posX)]))
                self.pick_execute = 'pick'
                self.on_press('pick')
            else:
                self.pick_execute = 'pickback'
                self.on_press('pickback')
    
    def on_mouse_move_GERAL_BAR(self, event):
        #print('on_mouse_move_GERAL_BAR inicio')
        if event.inaxes:
            posX = event.xdata
            posY = event.ydata
            if ((posX !=None) and (posY !=None)):
                self.on_mouse_move(posX,posY,'GERAL_BAR')
        else:
            self.on_mouse_move(0,0,'GERAL_BAR', True)
        #print('on_mouse_move_GERAL_BAR fim')
                
    def on_mouse_move_TWITTER_BAR(self, event):
        #print('on_mouse_move_TWITTER_BAR inicio')        
        if event.inaxes:
            posX = event.xdata
            posY = event.ydata
            if ((posX !=None) and (posY !=None)):
                if self.dicGraphicsFilters['TWITTER_BAR'].get() == 'TWITTER_BAR_LIKE':
                    field = 'Likes'
                elif self.dicGraphicsFilters['TWITTER_BAR'].get() == 'TWITTER_BAR_RET':
                    field = 'Retweets'
                else: #self.dicGraphicsFilters['TWITTER_BAR'] == 'TWITTER_BAR_REP'
                    field = 'Replies'
                self.on_mouse_move(posX,posY,'TWITTER_BAR',field)
        else:
            self.on_mouse_move(0,0,'TWITTER_BAR', '', True)    
            
    def on_mouse_move_INSTAGRAM_BAR(self, event):
        #print('on_mouse_move_INSTAGRAM_BAR inicio')
        if event.inaxes:
            posX = event.xdata
            posY = event.ydata
            if ((posX !=None) and (posY !=None)):
                if self.dicGraphicsFilters['INSTAGRAM_BAR'].get() == 'INSTAGRAM_BAR_LIKE':
                    field = "Instagram_Like"                      
                else: #self.dicGraphicsFilters['INSTAGRAM_BAR'], value='INSTAGRAM_BAR_COMMENT',command = lambda : self.on_press_radio())
                    field = "Instagram_Comment"
                self.on_mouse_move(posX,posY,'INSTAGRAM_BAR', field)
        else:
            self.on_mouse_move(0,0,'INSTAGRAM_BAR', '', True)
        #print('on_mouse_move_INSTAGRAM_BAR fim')
        
    def on_mouse_move_YOUTUBE_BAR(self, event):
        #print('on_mouse_move_YOUTUBE_BAR inicio')
        if event.inaxes:
            posX = event.xdata
            posY = event.ydata
            if ((posX !=None) and (posY !=None)):
                if self.dicGraphicsFilters['YOUTUBE_BAR'].get() == 'YOUTUBE_BAR_LIKE':
                    field = "Likes"
                elif self.dicGraphicsFilters['YOUTUBE_BAR'].get() == 'YOUTUBE_BAR_DISLIKE':
                    field = "Dislikes"
                elif self.dicGraphicsFilters['YOUTUBE_BAR'].get()  == 'YOUTUBE_BAR_VIEW':
                    field = "Views"
                else: #self.dicGraphicsFilters['YOUTUBE_BAR'], value='YOUTUBE_BAR_COMMENT'
                    field = "Comments"
                self.on_mouse_move(posX,posY,'YOUTUBE_BAR', field)
        else:
            self.on_mouse_move(0,0,'YOUTUBE_BAR', '', True)
        #print('on_mouse_move_YOUTUBE_BAR fim')        
        
    def on_mouse_move_TWITTER_HEAT(self, event):
        #print('on_mouse_move_TWITTER_MAP inicio')
        if event.inaxes:
            posX = event.xdata
            posY = event.ydata
            self.on_mouse_move_heat(posX,posY,'TWITTER_HEAT')
        else:
            self.on_mouse_move_heat(0,0,'TWITTER_HEAT', True)
        
        
    def on_click_TWITTER_HEAT(self, event):
        #print('on_mouse_move_TWITTER_MAP inicio')
        if event.inaxes:
            posX = event.xdata
            posY = event.ydata
            self.on_click_heat(posX,posY,'TWITTER_HEAT')        
            
    def on_mouse_move_INSTAGRAM_HEAT(self, event):
        #print('on_mouse_move_INSTAGRAM_HEAT inicio')
        if event.inaxes:
            posX = event.xdata
            posY = event.ydata
            self.on_mouse_move_heat(posX,posY,'INSTAGRAM_HEAT')
        else:
            self.on_mouse_move_heat(0,0,'INSTAGRAM_HEAT', True)
            
    def on_click_INSTAGRAM_HEAT(self, event):
        #print('on_mouse_move_TWITTER_MAP inicio')
        if event.inaxes:
            posX = event.xdata
            posY = event.ydata
            self.on_click_heat(posX,posY,'INSTAGRAM_HEAT')
            
    def on_mouse_move_YOUTUBE_HEAT(self, event):
        #print('on_mouse_move_YOUTUBE_HEAT inicio')
        if event.inaxes:
            posX = event.xdata
            posY = event.ydata
            self.on_mouse_move_heat(posX,posY,'YOUTUBE_HEAT')
        else:
            self.on_mouse_move_heat(0,0,'YOUTUBE_HEAT', True)
            
    def on_click_YOUTUBE_HEAT(self, event):
        #print('on_mouse_move_TWITTER_MAP inicio')
        if event.inaxes:
            posX = event.xdata
            posY = event.ydata
            self.on_click_heat(posX,posY,'YOUTUBE_HEAT')
            
    def comboHeatTwitter_select(self, event):
        #print(event)  
        self.dicGraphicsFilters['TWITTER_HEAT'] = self.comboHeatTwitter.get()
        self.dicGraphicsFilters['UPDATECOMBOS'] = False
        self.dicGraphicsFromTo['TWITTER_HEAT'] = [1,11]
        #self.dicGraphicsFromTo['UPDATECOMBOS'] = False
        self.pick_execute = 'comboheat'
        self.on_press('comboheat')
        
#     def comboGraphValueOptTwitter_select(self, event):
#         self.dicGraphicsFilters['TWITTER_GRAPHVALUE'] = str(self.comboGraphValueOptTwitter.get()).upper()
#         self.dicGraphicsFilters['UPDATECOMBOS'] = False
#         self.pick_execute = 'combographvalue'
#         self.on_press('combographvalue')
        
    def comboTopNTwitter_select(self, event):
        #print('topn')          
        fromToValues = self.comboTopNTwitter.get()
        self.dicGraphicsFromTo['TWITTER_HEAT'][0], self.dicGraphicsFromTo['TWITTER_HEAT'][1] = fromToValues.split("-")
        self.dicGraphicsFromTo['TWITTER_HEAT'][0] = int(self.dicGraphicsFromTo['TWITTER_HEAT'][0])
        self.dicGraphicsFromTo['TWITTER_HEAT'][1]  = int(self.dicGraphicsFromTo['TWITTER_HEAT'][1])
        self.dicGraphicsFilters['UPDATECOMBOS'] = False
        self.dicGraphicsFromTo['UPDATECOMBOS'] = False
        #self.pick_execute = 'comboheat'
        self.on_press('combotop')
        
    def comboHeatInstagram_select(self, event):        
        #print('comboHeatInstagram_select')
        self.dicGraphicsFilters['INSTAGRAM_HEAT'] = self.comboHeatInstagram.get()
        self.dicGraphicsFilters['UPDATECOMBOS'] = False
        self.dicGraphicsFromTo['INSTAGRAM_HEAT'] = [1,11]
        #self.dicGraphicsFromTo['UPDATECOMBOS'] = False
        self.pick_execute = 'comboheat'
        self.on_press('comboheat')
    
    def comboTopNInstagram_select(self, event):        
        #print('comboHeatInstagram_select')
        fromToValues = self.comboTopNInstagram.get()
        self.dicGraphicsFromTo['INSTAGRAM_HEAT'][0], self.dicGraphicsFromTo['INSTAGRAM_HEAT'][1] = fromToValues.split("-")
        self.dicGraphicsFromTo['INSTAGRAM_HEAT'][0] = int(self.dicGraphicsFromTo['INSTAGRAM_HEAT'][0])
        self.dicGraphicsFromTo['INSTAGRAM_HEAT'][1] = int(self.dicGraphicsFromTo['INSTAGRAM_HEAT'][1])
        self.dicGraphicsFilters['UPDATECOMBOS'] = False
        self.dicGraphicsFromTo['UPDATECOMBOS'] = False
        #self.pick_execute = 'comboheat'
        self.on_press('combotop')
        
    def comboHeatYouTube_select(self, event):      
        #print('comboHeatYoutube_select')
        self.dicGraphicsFilters['YOUTUBE_HEAT'] = self.comboHeatYouTube.get()
        self.dicGraphicsFilters['UPDATECOMBOS'] = False
        self.dicGraphicsFromTo['YOUTUBE_HEAT'] = [1,11]
        #self.dicGraphicsFromTo['UPDATECOMBOS'] = False
        self.pick_execute = 'comboheat'
        self.on_press('comboheat')
        
    def comboTopNYouTube_select(self, event):      
         #print('comboHeatInstagram_select')
        fromToValues = self.comboTopNYouTube.get()
        self.dicGraphicsFromTo['YOUTUBE_HEAT'][0], self.dicGraphicsFromTo['YOUTUBE_HEAT'][1] = fromToValues.split("-")
        self.dicGraphicsFromTo['YOUTUBE_HEAT'][0] = int(self.dicGraphicsFromTo['YOUTUBE_HEAT'][0])
        self.dicGraphicsFromTo['YOUTUBE_HEAT'][1] = int(self.dicGraphicsFromTo['YOUTUBE_HEAT'][1])
        self.dicGraphicsFilters['UPDATECOMBOS'] = False
        self.dicGraphicsFromTo['UPDATECOMBOS'] = False
        #self.pick_execute = 'comboheat'
        self.on_press('combotop')
        
    def home(self):
        #print('home')
        print(self.listdicGraphicsFilters[0])
        self.dicGraphicsFiltersListToDic(self.listdicGraphicsFilters[0])
        for l in range(0,len(self.listdicGraphicsFilters) -1):
            self.listdicGraphicsFilters.pop()
        self.combosGeraisSet()
        self.dicGraphicsFromTo['TWITTER_HEAT']  = [1,11]
        self.dicGraphicsFromTo['INSTAGRAM_HEAT'] = [1,11]
        self.dicGraphicsFromTo['YOUTUBE_HEAT'] = [1,11]
        self.dicGraphicsFromTo['UPDATECOMBOS'] = True
        
        self.pick_execute = 'home'
        self.on_press('home')
        
#     def comboGraphValueSet (self):
#         print(self.dicGraphicsFilters['TWITTER_GRAPHVALUE'])
#         if self.dicGraphicsFilters['TWITTER_GRAPHVALUE'] == "TOTALS":
#             self.comboGraphValueOptTwitter.current(0)
#         else:
#             self.comboGraphValueOptTwitter.current(1)
        
        
    def combosGeraisSet(self):
        #print('combosGeraisSet INI')
        for combo in self.listCombosGerais:
            if combo['values'] != ():
                combo.current(0)
        self.comboSocialMedia.current(0)
        self.comboQtdYears.current(0)
        
        #print('combosGeraisSet FIM')
            
        
    def dicToList(self,dic):
        listElements = []
        for k,v in dic.items():
            if 'BAR' in k and k !='GERAL_BAR':
                listElements.append(v.get())
            else:
                listElements.append(v)
        return list(listElements)
    
    def dicGraphicsFiltersListToDic(self,listFilter):
        i = 0;
        for k in self.dicGraphicsFilters:            
            if 'BAR' in k and k !='GERAL_BAR':
                self.dicGraphicsFilters[k].set(listFilter[i])
            else:
                self.dicGraphicsFilters[k] = listFilter[i]            
            i +=1
        
    def showhideGraphs(self):
        #print('showhideGraphs')
        self.frames_dic['GERAL'].pack_forget()
        self.frames_dic['TWITTER'].pack_forget()
        self.frames_dic['INSTAGRAM'].pack_forget()
        self.frames_dic['YOUTUBE'].pack_forget()
        #print(self.geral_show.get())
        
        if self.geral_show.get():
            if self.horiz_vert_show.get():
                self.frames_dic['GERAL'].pack(side="top",expand=True,fill='both', pady=2)
                self.frames_dic['GERAL_PIE'].pack(side="left",expand=True,fill='both')
                self.frames_dic['GERAL_BAR'].pack(side="left",expand=True,fill='both')
            else:
                self.frames_dic['GERAL'].pack(side="left", expand=True, fill='both', pady=2)
                self.frames_dic['GERAL_PIE'].pack(side="top",expand=True,fill='both')
                self.frames_dic['GERAL_BAR'].pack(side="top",expand=True,fill='both')
            
        if self.twitter_show.get():
            if self.horiz_vert_show.get():
                self.frames_dic['TWITTER'].pack(side="top",expand=True,fill='both')
            else:
                self.frames_dic['TWITTER'].pack(side="left",expand=True,fill='both', pady=2)
                            
        if self.instagram_show.get():
            if self.horiz_vert_show.get():
                self.frames_dic['INSTAGRAM'].pack(side="top",expand=True,fill='both')
            else:
                self.frames_dic['INSTAGRAM'].pack(side="left",expand=True,fill='both', pady=2)
            
        if self.youtube_show.get():
            if self.horiz_vert_show.get():
                self.frames_dic['YOUTUBE'].pack(side="top",expand=True,fill='both')
            else:
                self.frames_dic['YOUTUBE'].pack(side="left",expand=True,fill='both', pady=2)
        
        #self.GraphicPrepare()        
        #self.Graphics()
            
    def showLineBarsGraphs(self):
        #print('showLineBarsGraphs') 
        
        self.frames_dic['TWITTER_HEAT'].pack_forget()
        self.frames_dic['TWITTER_FILTER'].pack_forget()
        self.frames_dic['INSTAGRAM_HEAT'].pack_forget()
        self.frames_dic['INSTAGRAM_FILTER'].pack_forget()
        self.frames_dic['YOUTUBE_HEAT'].pack_forget()
        self.frames_dic['YOUTUBE_FILTER'].pack_forget()
        
        if self.horiz_vert_show.get():
            
            if self.bar_line_show.get():                
                self.frames_dic['TWITTER_LINE'].pack_forget()
                self.frames_dic['TWITTER_BAR'].pack(side="left",expand=True,fill='both')
                self.frames_dic['INSTAGRAM_LINE'].pack_forget()
                self.frames_dic['INSTAGRAM_BAR'].pack(side="left",expand=True,fill='both')
                self.frames_dic['YOUTUBE_LINE'].pack_forget()
                self.frames_dic['YOUTUBE_BAR'].pack(side="left",expand=True,fill='both')
            else:
                self.frames_dic['TWITTER_BAR'].pack_forget()
                self.frames_dic['TWITTER_LINE'].pack(side="left",expand=True,fill='both')
                self.frames_dic['INSTAGRAM_BAR'].pack_forget()
                self.frames_dic['INSTAGRAM_LINE'].pack(side="left",expand=True,fill='both')
                self.frames_dic['YOUTUBE_BAR'].pack_forget()
                self.frames_dic['YOUTUBE_LINE'].pack(side="left",expand=True,fill='both')
                
            self.frames_dic['TWITTER_HEAT'].pack(side="left",expand=True,fill='both')
            self.frames_dic['TWITTER_FILTER'].pack(side="top",expand=True,fill='both')
            self.frames_dic['INSTAGRAM_HEAT'].pack(side="left",expand=True,fill='both')
            self.frames_dic['INSTAGRAM_FILTER'].pack(side="top",expand=True,fill='both')
            self.frames_dic['YOUTUBE_HEAT'].pack(side="left",expand=True,fill='both')
            self.frames_dic['YOUTUBE_FILTER'].pack(side="top",expand=True,fill='both')
            
        else:
            
            if self.bar_line_show.get():
                self.frames_dic['TWITTER_LINE'].pack_forget()
                self.frames_dic['TWITTER_BAR'].pack(side="top",expand=True,fill='both')
                self.frames_dic['INSTAGRAM_LINE'].pack_forget()
                self.frames_dic['INSTAGRAM_BAR'].pack(side="top",expand=True,fill='both')
                self.frames_dic['YOUTUBE_LINE'].pack_forget()
                self.frames_dic['YOUTUBE_BAR'].pack(side="top",expand=True,fill='both')
            else:
                self.frames_dic['TWITTER_BAR'].pack_forget()
                self.frames_dic['TWITTER_LINE'].pack(side="top",expand=True,fill='both')
                self.frames_dic['INSTAGRAM_BAR'].pack_forget()
                self.frames_dic['INSTAGRAM_LINE'].pack(side="top",expand=True,fill='both')
                self.frames_dic['YseOUTUBE_BAR'].pack_forget()
                self.frames_dic['YOUTUBE_LINE'].pack(side="top",expand=True,fill='both')
                
            self.frames_dic['TWITTER_FILTER'].pack(side="left",expand=False,fill='y')
            self.frames_dic['TWITTER_HEAT'].pack(side="top",expand=True,fill='both')
            self.frames_dic['INSTAGRAM_FILTER'].pack(side="left",expand=False,fill='y')
            self.frames_dic['INSTAGRAM_HEAT'].pack(side="top",expand=True,fill='both')
            self.frames_dic['YOUTUBE_FILTER'].pack(side="left",expand=False,fill='y')
            self.frames_dic['YOUTUBE_HEAT'].pack(side="top",expand=True,fill='both')            
            
    def showHorizVert(self):
        #print('self.horiz_vert_show')                
        
        self.frames_dic['GERAL'].pack_forget()
        self.frames_dic['GERAL_PIE'].pack_forget()
        self.frames_dic['GERAL_BAR'].pack_forget()

        self.frames_dic['TWITTER'].pack_forget()
        self.frames_dic['INSTAGRAM'].pack_forget()
        self.frames_dic['YOUTUBE'].pack_forget()

        self.frames_dic['TWITTER_BAR'].pack_forget()
        self.frames_dic['TWITTER_HEAT'].pack_forget()
        self.frames_dic['TWITTER_FILTER'].pack_forget()

        self.frames_dic['INSTAGRAM_BAR'].pack_forget()
        self.frames_dic['INSTAGRAM_HEAT'].pack_forget()
        self.frames_dic['INSTAGRAM_FILTER'].pack_forget() 

        self.frames_dic['YOUTUBE_BAR'].pack_forget()
        self.frames_dic['YOUTUBE_HEAT'].pack_forget()                            
        self.frames_dic['YOUTUBE_FILTER'].pack_forget()
        
        if self.horiz_vert_show.get():
            
            if self.geral_show.get():
                self.frames_dic['GERAL'].pack(side="top",expand=True,fill='both', pady=2)
                self.frames_dic['GERAL_PIE'].pack(side="left",expand=True,fill='both')
                self.frames_dic['GERAL_BAR'].pack(side="left",expand=True,fill='both')
            
            if self.twitter_show.get():
                self.frames_dic['TWITTER'].pack(side="top",expand=True,fill='both', pady=2)
                
            if self.instagram_show.get():
                self.frames_dic['INSTAGRAM'].pack(side="top",expand=True,fill='both', pady=2)
                
            if self.youtube_show.get():
                self.frames_dic['YOUTUBE'].pack(side="top",expand=True,fill='both', pady=2)
            
            #self.frames_dic['TWITTER_BAR'].pack(side="left",expand=True,fill='both')
            
            if self.bar_line_show.get():
                self.frames_dic['TWITTER_LINE'].pack_forget()                
                self.frames_dic['TWITTER_BAR'].pack(side="left",expand=True,fill='both')
                self.frames_dic['INSTAGRAM_LINE'].pack_forget()
                self.frames_dic['INSTAGRAM_BAR'].pack(side="left",expand=True,fill='both')
                self.frames_dic['YOUTUBE_LINE'].pack_forget()
                self.frames_dic['YOUTUBE_BAR'].pack(side="left",expand=True,fill='both')
            else:
                self.frames_dic['TWITTER_BAR'].pack_forget()
                self.frames_dic['TWITTER_LINE'].pack(side="left",expand=True,fill='both')
                self.frames_dic['INSTAGRAM_BAR'].pack_forget()
                self.frames_dic['INSTAGRAM_LINE'].pack(side="left",expand=True,fill='both')
                self.frames_dic['YOUTUBE_BAR'].pack_forget()
                self.frames_dic['YOUTUBE_LINE'].pack(side="left",expand=True,fill='both')
                
            self.frames_dic['TWITTER_HEAT'].pack(side="left",expand=True,fill='both')
            self.frames_dic['TWITTER_FILTER'].pack(side="top",expand=True,fill='both')            
            
            #self.frames_dic['INSTAGRAM_BAR'].pack(side="left",expand=True,fill='both')
            self.frames_dic['INSTAGRAM_HEAT'].pack(side="left",expand=True,fill='both')
            self.frames_dic['INSTAGRAM_FILTER'].pack(side="top",expand=True,fill='both') 
            
            #self.frames_dic['YOUTUBE_BAR'].pack(side="left",expand=True,fill='both')
            self.frames_dic['YOUTUBE_HEAT'].pack(side="left",expand=True,fill='both')                            
            self.frames_dic['YOUTUBE_FILTER'].pack(side="top",expand=True,fill='both')
            
        else:
            
            if self.geral_show.get():
                self.frames_dic['GERAL'].pack(side="left", expand=True, fill='both', pady=2)
                self.frames_dic['GERAL_PIE'].pack(side="top",expand=True,fill='both')
                self.frames_dic['GERAL_BAR'].pack(side="top",expand=True,fill='both')
            
            if self.twitter_show.get():
                self.frames_dic['TWITTER'].pack(side="left",expand=True,fill='both', pady=2)
            if self.instagram_show.get():
                self.frames_dic['INSTAGRAM'].pack(side="left",expand=True,fill='both', pady=2)
            if self.youtube_show.get():
                self.frames_dic['YOUTUBE'].pack(side="left",expand=True,fill='both', pady=2)
            
            if self.bar_line_show.get():
                self.frames_dic['TWITTER_LINE'].pack_forget()
                self.frames_dic['TWITTER_BAR'].pack(side="top",expand=True,fill='both')
                self.frames_dic['INSTAGRAM_LINE'].pack_forget()
                self.frames_dic['INSTAGRAM_BAR'].pack(side="top",expand=True,fill='both')
                self.frames_dic['YOUTUBE_LINE'].pack_forget()
                self.frames_dic['YOUTUBE_BAR'].pack(side="top",expand=True,fill='both')
            else:
                self.frames_dic['TWITTER_BAR'].pack_forget()
                self.frames_dic['TWITTER_LINE'].pack(side="top",expand=True,fill='both')
                self.frames_dic['INSTAGRAM_BAR'].pack_forget()
                self.frames_dic['INSTAGRAM_LINE'].pack(side="top",expand=True,fill='both')
                self.frames_dic['YOUTUBE_BAR'].pack_forget()
                self.frames_dic['YOUTUBE_LINE'].pack(side="top",expand=True,fill='both')
                
            self.frames_dic['TWITTER_FILTER'].pack(side="left",expand=False,fill='y')
            self.frames_dic['TWITTER_HEAT'].pack(side="top",expand=True,fill='both')
            
            #self.frames_dic['INSTAGRAM_BAR'].pack(side="top",expand=True,fill='both')
            self.frames_dic['INSTAGRAM_FILTER'].pack(side="left",expand=False,fill='y')  
            self.frames_dic['INSTAGRAM_HEAT'].pack(side="top",expand=True,fill='both')
            
            #self.frames_dic['YOUTUBE_BAR'].pack(side="top",expand=True,fill='both')
            self.frames_dic['YOUTUBE_FILTER'].pack(side="left",expand=False,fill='y')
            self.frames_dic['YOUTUBE_HEAT'].pack(side="top",expand=True,fill='both')
            
            
    def show_statistics(self):
        dicData = {'TWITTER_STATS'     : self.dicDF_StatsSent['TWITTER_BAR'],
                   'INSTAGRAM_STATS'   : self.dicDF_StatsSent['INSTAGRAM_BAR'],
                   'YOUTUBE_STATS'     : self.dicDF_StatsSent['YOUTUBE_BAR'],
                   'DATATIPO'          : self.dicGraphicsFilters['DATATIPO'],#'MES', 
                   'DATAVAL'           : self.dicGraphicsFilters['DATAVAL'],#'2009',
                   'SOCIALMEDIA'       : self.dicGraphicsFilters['SOCIALMEDIA'],#'',
                   'SOCIALMEDIATITLE'  : self.title#'',
                  }
        #print(dicData)
        self.newWindow = Toplevel(self.master)        
        self.statisticsgui = TKInterStatistics(self.newWindow,dicData)
        
    def show_sent_anal_youtube(self):
        dicData = {'DATATIPO'          : self.dicGraphicsFilters['DATATIPO'],#'MES', 
                   'DATAVAL'           : self.dicGraphicsFilters['DATAVAL'],#'2009',
                   'SOCIALMEDIA'       : 'YouTube',#'',
                   'SOCIALMEDIATITLE'  : self.title#''
                  }
        self.newWindow = Toplevel(self.master)        
        self.sentanalysisgui = TKInterSentAnalisys(self.newWindow,dicData)
        
    def show_sent_anal_instagram(self):
        dicData = {'DATATIPO'         : self.dicGraphicsFilters['DATATIPO'],#'MES', 
                   'DATAVAL'          : self.dicGraphicsFilters['DATAVAL'],#'2009',
                   'SOCIALMEDIA'      : 'Instagram',
                   'SOCIALMEDIATITLE' : self.title#'',
                  }
        self.newWindow = Toplevel(self.master)        
        self.sentanalysisgui = TKInterSentAnalisys(self.newWindow,dicData)

root = tk.Tk()

sns.set_style("darkgrid", {"axes.edgecolor":  ".8", 'axes.facecolor': 'white', 'grid.color': '.8'})
db_name = ''

#profile_p = sys.argv[1]

#profile_p = "netflix2"
profile_p = "amazonprime"
#profile_p = "nilewilson"

#print(profile_p)
#user_p = sys.argv[2]

client = ('mongodb://localhost/socialmediadb_' + profile_p)
print('Connecting database')
con = Conexao (client)  
print('Loading database')
dfAnoCountOrig, dfMesCountOrig, dfDiaCountOrig,  dfHoraCountOrig, dfMinCountOrig = dataFrameRetrieveCount()
print('Loading database.')
dfAnoMaxOrigTwitter, dfMesMaxOrigTwitter, dfDiaMaxOrigTwitter, dfHoraMaxOrigTwitter, dfMinMaxOrigTwitter = dataFrameRetrieveMaxTwitter()
print('Loading database..')
dfAnoMaxOrigYouTube, dfMesMaxOrigYouTube, dfDiaMaxOrigYouTube, dfHoraMaxOrigYouTube, dfMinMaxOrigYouTube = dataFrameRetrieveMaxYouTube()
print('Loading database...')
dfAnoMaxOrigInstagram, dfMesMaxOrigInstagram, dfDiaMaxOrigInstagram, dfHoraMaxOrigInstagram, dfMinMaxOrigInstagram = dataFrameRetrieveMaxInstagram()
print('Loading database....')
print('Database Loaded')
if profile_p == "amazonprime":
    db_name = "Amazon Prime Video"
elif profile_p == "netflix2":
    db_name = "Netflix"
elif profile_p == "nilewilson":
    db_name = "Nile Wilson"
    
profile = profile_p
my_gui = TKInterGUI(root, db_name)
my_gui.home()

root.mainloop()

Connecting database
Loading database
Loading database.
Loading database..
Loading database...
Loading database....
Database Loaded
['ANO', '', '', 'ALL', [2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008], 'GERAL_BAR', 'TWITTER_BAR_LIKE', 'INSTAGRAM_BAR_LIKE', 'YOUTUBE_BAR_LIKE', '2020', '2020', '2020', True]
self.on_press_ini


C:\Users\Joana\Anaconda3\lib\site-packages\matplotlib\backends\backend_agg.py:240: RuntimeWarning: Glyph 127774 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\Users\Joana\Anaconda3\lib\site-packages\matplotlib\backends\backend_agg.py:203: RuntimeWarning: Glyph 127774 missing from current font.
  font.set_text(s, 0, flags=flags)


self.on_press_fim
['ANO', '', '', 'ALL', [2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008], 'GERAL_BAR', 'TWITTER_BAR_LIKE', 'INSTAGRAM_BAR_LIKE', 'YOUTUBE_BAR_LIKE', '2020', '2020', '2020', True]
self.on_press_ini
self.on_press_fim
['ANO', '', '', 'ALL', [2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008], 'GERAL_BAR', 'TWITTER_BAR_LIKE', 'INSTAGRAM_BAR_LIKE', 'YOUTUBE_BAR_LIKE', '2020', '2020', '2020', True]
self.on_press_ini
self.on_press_fim


C:\Users\Joana\Anaconda3\lib\site-packages\matplotlib\backends\backend_agg.py:240: RuntimeWarning: Glyph 9752 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\Users\Joana\Anaconda3\lib\site-packages\matplotlib\backends\backend_agg.py:240: RuntimeWarning: Glyph 65039 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\Users\Joana\Anaconda3\lib\site-packages\matplotlib\backends\backend_agg.py:203: RuntimeWarning: Glyph 9752 missing from current font.
  font.set_text(s, 0, flags=flags)
C:\Users\Joana\Anaconda3\lib\site-packages\matplotlib\backends\backend_agg.py:203: RuntimeWarning: Glyph 65039 missing from current font.
  font.set_text(s, 0, flags=flags)
C:\Users\Joana\Anaconda3\lib\site-packages\matplotlib\backends\backend_agg.py:240: RuntimeWarning: Glyph 127774 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\Users\Joana\Anaconda3\lib\site-packages\matplotlib\backends\backend_agg.py:203: RuntimeWarning: Glyph 127774 missing from curre

Couldn't find 12.404165457927343!
Couldn't find 12.436110305284895!
Couldn't find 12.436110305284895!


['JAN', 'FEB']